In [1]:
import numpy as np
import os
import pandas as pd
import arcpy

# Import the required ArcGIS API for Python modules
import arcgis
from arcgis.gis import GIS
gis = arcgis.GIS()
from arcgis.geoanalytics import manage_data

from IPython.display import display, Markdown

In [17]:
dirWork         = os.getcwd()
year            = '2018'
dirInput        = os.path.join(dirWork, r'data\data' + year                    )
dirIntermediate = os.path.join(dirWork, r'intermediate\intermediate' + year    )
dirResults      = os.path.join(dirWork, r'results\results' + year              )

print("Working Directory: "      + dirWork        )
print("Input Directory: "        + dirInput       )
print("Intermediate Directory: " + dirIntermediate)
print("Results Directory: "      + dirResults     )

Working Directory: c:\Users\cday\Documents\projects\Census-LEHD-Commute-Patterns
Input Directory: c:\Users\cday\Documents\projects\Census-LEHD-Commute-Patterns\data\data2018
Intermediate Directory: c:\Users\cday\Documents\projects\Census-LEHD-Commute-Patterns\intermediate\intermediate2018
Results Directory: c:\Users\cday\Documents\projects\Census-LEHD-Commute-Patterns\results\results2018


In [12]:
dirIntermediate

'c:\\Users\\cday\\Documents\\projects\\Census-LEHD-Commute-Patterns\\intermediate\\intermediate2018'

In [53]:
#input filenames
fname_CityTownship_csv        = os.path.join(dirInput, 'citytownship.csv'                                         )
fname_CityTownship_shp        = os.path.join(dirInput, 'Municipalities_Township' + r'\Municipalities_Township.shp') 
fname_LEHDCommutePatterns_csv = os.path.join(dirInput, 'lehd_at_blockgroup_level' + year + '.csv'                 )
fname_UofUManualAllocation_csv= os.path.join(dirInput, '490351108004_Manual_Allocation.csv'                       )
fname_BlockGroups_shp         = os.path.join(dirInput, 'tl_2020_49_bg' + r'\tl_2020_49_bg.shp'                    )
fname_Counties_AGRC_shp       = os.path.join(dirInput, 'Utah_County_Boundaries-shp' + r'\Counties.shp'            )
fname_LEHDCommutePatterns_csv

#intermediate
fname_CityTownshipCounty_BlockGroups_shp = os.path.join(dirIntermediate, 'citytownshipcounty_blockgroup_intersect.shp')
fname_CityTownshipWCodes_shp             = os.path.join(dirIntermediate, 'Municipalities_Townships_withCodes.shp')
fname_CountyWCodes_shp                   = os.path.join(dirIntermediate, 'Counties_withCodes.shp')
fname_CityTownshipWCodesUnionCounty_shp  = os.path.join(dirIntermediate, 'Municipalities_Townships_withCodes_Counties.shp')
fname_CityTownshipCountyWCodes_shp       = os.path.join(dirIntermediate, 'Municipalities_Townships_Counties_withCodes.shp')

#display(dirIntermediate)

#results geodatabase

fname_CityTownshipJSON = os.path.join(dirResults, "citytownshipdata.json")

gdb_LEHDCommutePatterns = "LEHDCommutePatterns.gdb"
fname_LEHDCommutePatterns_gdb = os.path.join(dirResults, gdb_LEHDCommutePatterns)

fname_CityTownshipWCodes = os.path.join(fname_LEHDCommutePatterns_gdb, 'Municipalities_Townships')
#fname_CityTownshipCounty = os.path.join(fname_LEHDCommutePatterns_gdb, 'Municipalities_Townships_Counties')

fname_BlockGroupsWCommutePatterns_Number = os.path.join(fname_LEHDCommutePatterns_gdb, 'BlockGroup_CommutePatterns_Number')
fname_BlockGroupsWCommutePatterns_Percent_SA = os.path.join(fname_LEHDCommutePatterns_gdb, 'BlockGroup_CommutePatterns_Percent_SelectedArea')
fname_BlockGroupsWCommutePatterns_Percent_MU = os.path.join(fname_LEHDCommutePatterns_gdb, 'BlockGroup_CommutePatterns_Percent_MapUnit')

fname_TractsWCommutePatterns_Number = os.path.join(fname_LEHDCommutePatterns_gdb, 'Tract_CommutePatterns_Number')
fname_TractsWCommutePatterns_Percent_SA = os.path.join(fname_LEHDCommutePatterns_gdb, 'Tract_CommutePatterns_Percent_SelectedArea')
fname_TractsWCommutePatterns_Percent_MU = os.path.join(fname_LEHDCommutePatterns_gdb, 'Tract_CommutePatterns_Percent_MapUnit')

fname_CitiesWCommutePatterns_Number = os.path.join(fname_LEHDCommutePatterns_gdb, 'City_CommutePatterns_Number')
fname_CitiesWCommutePatterns_Percent_SA = os.path.join(fname_LEHDCommutePatterns_gdb, 'City_CommutePatterns_Percent_SelectedArea')
fname_CitiesWCommutePatterns_Percent_MU = os.path.join(fname_LEHDCommutePatterns_gdb, 'City_CommutePatterns_Percent_MapUnit')

fname_Counties = os.path.join(fname_LEHDCommutePatterns_gdb, 'Counties')

def deleteIfExists(obj):
    if arcpy.Exists(obj): arcpy.Delete_management(obj)

#if results geodatabase doesn't exist, create it
print("Checking if " + fname_LEHDCommutePatterns_gdb + " exists...")
if not arcpy.Exists(os.path.join(dirResults, gdb_LEHDCommutePatterns)):
    print("Creating " + fname_LEHDCommutePatterns_gdb + "...")
    arcpy.management.CreateFileGDB(dirResults, gdb_LEHDCommutePatterns)
else:
    print(gdb_LEHDCommutePatterns + " exists...")

#print(AnalysisAreasInput)

Checking if c:\Users\cday\Documents\projects\Census-LEHD-Commute-Patterns\results\results2018\LEHDCommutePatterns.gdb exists...
LEHDCommutePatterns.gdb exists...


# Create Cities/Townships Feature Class with 3-Digit Code and County FIPS

In [19]:
#read in city/township shapefile, SHORTDESC is common field between this and CODE3 dataset
sdf_CityTownship = pd.DataFrame.spatial.from_featureclass(fname_CityTownship_shp)
display(sdf_CityTownship)

,FID,COUNTYNBR,NAME,COUNTYSEAT,SHORTDESC,UPDATED,FIPS,ENTITYNBR,SALESTAXID,IMSCOLOR,MINNAME,POPLASTCEN,POPLASTEST,GNIS,GlobalID,SHAPE_Leng,SHAPE_Area,SHAPE
0,0,03,Newton,0,NEWTON,2018-09-19,54550,3100.0,047,2,,801,813,1430705,{1C320704-F3E4-4D4F-A542-F6FFE3C36900},0.071275,0.000240,"{""rings"": [[[-112.00138477499996, 41.855487523..."
1,1,12,Eureka,0,EUREKA,2018-09-19,24080,3010.0,009,2,,693,695,1437974,{E1F6AAAD-E49E-4820-A270-D3E7C2131A80},0.100706,0.000395,"{""rings"": [[[-112.09987515199998, 39.957670251..."
2,2,29,Huntsville,0,HUNTSVILLE,2014-10-15,37060,3030.0,019,4,Huntsville,602,638,1428949,{E9A934D8-FE13-4A6D-925E-73A9988E3F17},0.126158,0.000246,"{""rings"": [[[-111.76117535999998, 41.253396941..."
3,3,27,Springdale,0,SPRINGDALE,2014-10-15,71840,3100.0,023,2,Springdale,569,609,1432867,{E6E76C45-D141-47DF-9C96-D3D46F15A2B9},0.193621,0.001208,"{""rings"": [[[-112.97934207899999, 37.197581387..."
4,4,23,Grantsville,0,GRANTSVILLE,2020-06-05,31120,3010.0,023,2,,9152,11568,1428338,{5F5B6417-7EF9-49B5-A689-CBF42953DC9D},0.772545,0.010742,"{""rings"": [[[-112.37537669199997, 40.593511564..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253,253,18,COPPERTON METRO TOWNSHIP,0,Copperton,2016-12-27,15720,0.0,,0,Copperton,822,835,,{B3F64B90-565B-44B4-8C7B-87000672B734},0.067294,0.000086,"{""rings"": [[[-112.09934129299995, 40.562296682..."
254,254,18,KEARNS METRO TOWNSHIP,0,Kearns,2016-12-27,40470,0.0,,0,Kearns,35769,36330,,{75A0B23A-775B-44DC-BCCA-4E504BF9E548},0.185692,0.001277,"{""rings"": [[[-111.98660579699998, 40.667567316..."
255,255,18,MAGNA METRO TOWNSHIP,0,Magna,2018-11-14,47290,0.0,,0,Magna,26513,26949,,{28F85E1F-72D3-43E5-BBF4-F85CC28898D5},0.756349,0.010342,"{""rings"": [[[-112.10619664199999, 40.691640141..."
256,256,18,EMIGRATION CANYON METRO TOWNSHIP,0,Emigration Canyon,2016-12-27,22875,0.0,,0,Emigration Canyon,1568,1592,,{29B91C07-4DE5-4395-8337-6E13304256BB},0.350202,0.005034,"{""rings"": [[[-111.74338063899995, 40.818777216..."


In [20]:
#rename METRO TOWNSHIPs to match CODE3 dataset
sdf_CityTownship.loc[sdf_CityTownship.NAME == 'WHITE CITY METRO TOWNSHIP'       , 'SHORTDESC'] = "WHITE CITY METRO TOWNSHIP"
sdf_CityTownship.loc[sdf_CityTownship.NAME == 'EMIGRATION CANYON METRO TOWNSHIP', 'SHORTDESC'] = "EMIGRATION CANYON METRO TOWNSHIP"
sdf_CityTownship.loc[sdf_CityTownship.NAME == 'COPPERTON METRO TOWNSHIP'        , 'SHORTDESC'] = "COPPERTON METRO TOWNSHIP"
sdf_CityTownship.loc[sdf_CityTownship.NAME == 'MAGNA METRO TOWNSHIP'            , 'SHORTDESC'] = "MAGNA METRO TOWNSHIP"
sdf_CityTownship.loc[sdf_CityTownship.NAME == 'KEARNS METRO TOWNSHIP'           , 'SHORTDESC'] = "KEARNS METRO TOWNSHIP"

#give townships a better name
sdf_CityTownship.loc[sdf_CityTownship.NAME == 'WHITE CITY METRO TOWNSHIP'       , 'NAME'] = "White City"
sdf_CityTownship.loc[sdf_CityTownship.NAME == 'EMIGRATION CANYON METRO TOWNSHIP', 'NAME'] = "Emigration Canyon"
sdf_CityTownship.loc[sdf_CityTownship.NAME == 'COPPERTON METRO TOWNSHIP'        , 'NAME'] = "Copperton"
sdf_CityTownship.loc[sdf_CityTownship.NAME == 'MAGNA METRO TOWNSHIP'            , 'NAME'] = "Magna"
sdf_CityTownship.loc[sdf_CityTownship.NAME == 'KEARNS METRO TOWNSHIP'           , 'NAME'] = "Kearns"

In [29]:
#read in township codes data
df_CityTownshipCodes = pd.read_csv(fname_CityTownship_csv)

#consolidate Hideout
df_CityTownshipCodes = df_CityTownshipCodes.loc[df_CityTownshipCodes.SHORTDESC != 'HIDEOUT (WASATCH)']
df_CityTownshipCodes = df_CityTownshipCodes.loc[df_CityTownshipCodes.SHORTDESC != 'HIDEOUT (SUMMIT)']

# get wfrc city/townships only
wfrc_towns  = ["AFK","ALA","ALP","BDL","BGM","BNT","BRT","CDF","CEN","CHA","CHL","CLF","CLI","CMT","COA","CWH","DAN","DRA","EAG","ELK","EMT","FAR","FCS","FFD","FRR","FTH","GLA","GOS","GRL","HAR","HDT","HDT","HEB","HER","HGH","HNF","HOL","HOO","HVL","IND","INT","KAY","KMS","KMT","LAY","LEH","LIN","MAP","MID","MLC","MMT","MRG","MSL","MUR","MWY","NOG","NSL","OGD","OKL","ORM","PAY","PGR","PLN","PRK","PRY","PVO","PVW","ROY","RVD","RVT","SAN","SAQ","SAR","SFK","SJC","SLC","SLM","SOG","SPV","SSL","SUN","SWE","SYR","TAY","TOO","UIN","VIN","WAT","WBG","WDL","WEB","WHT","WHV","WIL","WJC","WPT","WVC","WXC"]
df_CityTownshipCodes = df_CityTownshipCodes[df_CityTownshipCodes['CODE3'].isin(wfrc_towns)].reset_index(drop=True)

df_CityTownshipCodes

,SHORTDESC,CODE3
0,AMERICAN FORK,AFK
1,ALTA,ALA
2,ALPINE,ALP
3,BLUFFDALE (SL CO),BDL
4,BLUFFDALE (UTAH CO),BDL
...,...,...
96,WILLARD CITY,WIL
97,WEST JORDAN CITY,WJC
98,WEST POINT,WPT
99,WEST VALLEY CITY,WVC


In [30]:
#join spatial data frame to data frame with codes using SHORTDESC field as key between two
sdf_CityTownshipWCodes = pd.DataFrame.merge(sdf_CityTownship, df_CityTownshipCodes, on='SHORTDESC')

#export to intermediate location so that it can be used in dissolve function later (to remove cities that span counties)
sdf_CityTownshipWCodes.spatial.to_featureclass(fname_CityTownshipWCodes_shp)

display(sdf_CityTownshipWCodes)

,FID,COUNTYNBR,NAME,COUNTYSEAT,SHORTDESC,UPDATED,FIPS,ENTITYNBR,SALESTAXID,IMSCOLOR,MINNAME,POPLASTCEN,POPLASTEST,GNIS,GlobalID,SHAPE_Leng,SHAPE_Area,SHAPE,CODE3
0,2,29,Huntsville,0,HUNTSVILLE,2014-10-15,37060,3030.0,019,4,Huntsville,602,638,1428949,{E9A934D8-FE13-4A6D-925E-73A9988E3F17},0.126158,0.000246,"{""rings"": [[[-111.76117535999998, 41.253396941...",HVL
1,4,23,Grantsville,0,GRANTSVILLE,2020-06-05,31120,3010.0,023,2,,9152,11568,1428338,{5F5B6417-7EF9-49B5-A689-CBF42953DC9D},0.772545,0.010742,"{""rings"": [[[-112.37537669199997, 40.593511564...",GRL
2,5,18,Bluffdale,0,BLUFFDALE (SL CO),2018-11-15,06810,3020.0,019,4,,7693,14699,1425844,{E52CE320-1532-4A57-8DB9-DFA819FBB85B},0.303290,0.002868,"{""rings"": [[[-111.94473146099995, 40.497759794...",BDL
3,7,18,Herriman,0,HERRIMAN TOWN,2019-11-22,34970,3035.0,060,1,,22018,44877,1428675,{3E78F48D-80B9-44DA-AC9F-836A7B4F1B8E},0.617524,0.005943,"{""rings"": [[[-112.06032590299998, 40.481954434...",HER
4,13,18,Taylorsville,0,TAYLORSVILLE CITY,2014-09-15,75360,3105.0,142,4,Taylorsville,59989,60192,1433206,{3ADD19D5-EF23-4257-8048-E355AE283BC6},0.303444,0.002991,"{""rings"": [[[-111.92081851599994, 40.685958404...",TAY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,253,18,Copperton,0,COPPERTON METRO TOWNSHIP,2016-12-27,15720,0.0,,0,Copperton,822,835,,{B3F64B90-565B-44B4-8C7B-87000672B734},0.067294,0.000086,"{""rings"": [[[-112.09934129299995, 40.562296682...",CMT
97,254,18,Kearns,0,KEARNS METRO TOWNSHIP,2016-12-27,40470,0.0,,0,Kearns,35769,36330,,{75A0B23A-775B-44DC-BCCA-4E504BF9E548},0.185692,0.001277,"{""rings"": [[[-111.98660579699998, 40.667567316...",KMT
98,255,18,Magna,0,MAGNA METRO TOWNSHIP,2018-11-14,47290,0.0,,0,Magna,26513,26949,,{28F85E1F-72D3-43E5-BBF4-F85CC28898D5},0.756349,0.010342,"{""rings"": [[[-112.10619664199999, 40.691640141...",MMT
99,256,18,Emigration Canyon,0,EMIGRATION CANYON METRO TOWNSHIP,2016-12-27,22875,0.0,,0,Emigration Canyon,1568,1592,,{29B91C07-4DE5-4395-8337-6E13304256BB},0.350202,0.005034,"{""rings"": [[[-111.74338063899995, 40.818777216...",EMT


In [35]:
#dissolve by code, due to some cities spanning counties and having multiple records in codes and/or munis/townships
deleteIfExists(fname_CityTownshipWCodes)
arcpy.management.Dissolve(fname_CityTownshipWCodes_shp, fname_CityTownshipWCodes, ['CODE3'], [['NAME','FIRST'], ['COUNTYNBR','FIRST']])

sdf_CityTownshipWCodes = pd.DataFrame.spatial.from_featureclass(fname_CityTownshipWCodes)
sdf_CityTownshipWCodes = sdf_CityTownshipWCodes.rename(columns={"code3":"CODE3", "FIRST_name":"FIRST_NAME", "FIRST_countynbr":"FIRST_COUNTYNBR"})

display(sdf_CityTownshipWCodes)

,OBJECTID,CODE3,FIRST_NAME,FIRST_COUNTYNBR,SHAPE
0,1,AFK,American Fork,25,"{""rings"": [[[-111.77435121873373, 40.353624868..."
1,2,ALA,Alta,18,"{""rings"": [[[-111.59946851855727, 40.577431980..."
2,3,ALP,Alpine,25,"{""rings"": [[[-111.75455302160464, 40.479513910..."
3,4,BDL,Bluffdale,18,"{""rings"": [[[-111.9704071902396, 40.5008379065..."
4,5,BGM,Brigham City,02,"{""rings"": [[[-112.01898409573937, 41.535781784..."
...,...,...,...,...,...
92,93,WIL,Willard,02,"{""rings"": [[[-112.02999503963935, 41.391465965..."
93,94,WJC,West Jordan,18,"{""rings"": [[[-112.00475852158849, 40.638664299..."
94,95,WPT,West Point,06,"{""rings"": [[[-112.10779328273378, 41.143687479..."
95,96,WVC,West Valley City,18,"{""rings"": [[[-111.9626132723846, 40.7260972067..."


In [36]:
df_CityTownship_NameCode = sdf_CityTownshipWCodes[['FIRST_NAME','CODE3','FIRST_COUNTYNBR']].copy()
df_CityTownship_NameCode = df_CityTownship_NameCode.sort_values(by=['FIRST_NAME'])

df_CityTownship_NameCode.columns = ('NAME','CODE3','COUNTYNBR')

display(df_CityTownship_NameCode)

#df_CityTownship_NameCode.to_json(os.path.join(dirResults, "citytownshipdata.json"),orient='records')
#export after adding workers and residents columns from data

,NAME,CODE3,COUNTYNBR
2,Alpine,ALP,25
1,Alta,ALA,18
0,American Fork,AFK,25
3,Bluffdale,BDL,18
5,Bountiful,BNT,06
...,...,...,...
95,West Valley City,WVC,18
90,White City,WHT,18
92,Willard,WIL,02
88,Woodland Hills,WDL,25


In [37]:
#explicit definition of county for cross-county cities
df_CityTownship_NameCode.loc[((df_CityTownship_NameCode['CODE3'] == 'BDL')), 'COUNTYNBR'] = '18'
df_CityTownship_NameCode.loc[((df_CityTownship_NameCode['CODE3'] == 'DRA')), 'COUNTYNBR'] = '18'
df_CityTownship_NameCode.loc[((df_CityTownship_NameCode['CODE3'] == 'HDT')), 'COUNTYNBR'] = '26'
df_CityTownship_NameCode.loc[((df_CityTownship_NameCode['CODE3'] == 'PRK')), 'COUNTYNBR'] = '22'

#read in county dataset
sdf_CountiesAGRC = pd.DataFrame.spatial.from_featureclass(fname_Counties_AGRC_shp)
sdf_CountiesAGRC['FIPS'   ] = sdf_CountiesAGRC['FIPS'].astype(int)
sdf_CountiesAGRC['zCOFIPS'] = 'z' + sdf_CountiesAGRC['FIPS'].astype(str).str.zfill(2) #add 'z' instead of 0, so that columns names won't start with number, and so sorts at end
sdf_CountiesAGRC['CO_NAME'] = sdf_CountiesAGRC['NAME'].str.title() + ' County'
display(sdf_CountiesAGRC)

#join by COUNTYNBR to get CO_FIPS
df_CityTownship_NameCode_wCOFIPS = pd.DataFrame.merge(df_CityTownship_NameCode,sdf_CountiesAGRC,on='COUNTYNBR')
display(df_CityTownship_NameCode_wCOFIPS)


df_CityTownship_NameCode = df_CityTownship_NameCode_wCOFIPS[['NAME_x','CODE3','zCOFIPS']]
df_CityTownship_NameCode.columns = ('NAME','CODE3','zCOFIPS')

display(df_CityTownship_NameCode)

,FID,OBJECTID,COUNTYNBR,ENTITYNBR,ENTITYYR,NAME,FIPS,STATEPLANE,POP_LASTCE,POP_CURRES,GlobalID,FIPS_STR,COLOR4,SHAPE_Leng,SHAPE_Area,SHAPE,zCOFIPS,CO_NAME
0,0,1,03,2.010031e+09,2010.0,CACHE,5,North,112656,128289,{59BEE5FF-00D6-414A-8CD9-3B9CF5FA25FB},49005,2,3.071504,0.328661,"{""rings"": [[[-112.15617197899996, 41.997725941...",z05,Cache County
1,1,2,07,2.010071e+09,2010.0,DUCHESNE,13,Central,18607,19938,{AAE112BD-8D24-46EE-98CD-6689B87A14D1},49013,4,3.966961,0.891751,"{""rings"": [[[-110.25173670299995, 40.832347463...",z13,Duchesne County
2,2,3,06,2.010061e+09,2010.0,DAVIS,11,North,306479,355481,{66DDE457-AF53-4B6D-9A7D-25F29FCCF626},49011,3,2.259091,0.176033,"{""rings"": [[[-111.92092166399999, 41.138676415...",z11,Davis County
3,3,4,14,2.010141e+09,2010.0,MILLARD,27,Central,12503,13188,{DE7E933D-7233-4FBE-8D0C-5E0826EF05D7},49027,3,6.100361,1.843191,"{""rings"": [[[-112.23854898599996, 39.553633436...",z27,Millard County
4,4,5,26,2.010261e+09,2010.0,WASATCH,51,Central,23530,34091,{1BAA04F5-D976-4917-BBF2-BE9C0F868C55},49051,2,3.353038,0.331949,"{""rings"": [[[-111.39572506599995, 40.690187842...",z51,Wasatch County
5,5,6,02,2.010021e+09,2010.0,BOX ELDER,3,North,49975,56046,{2F4FBEBE-32DB-4A2D-A6D1-277679BDF193},49003,4,6.210604,1.880056,"{""rings"": [[[-113.47488592699995, 41.993309448...",z03,Box Elder County
6,6,7,29,2.010291e+09,2010.0,WEBER,57,North,231236,260213,{7F2E8776-C886-4F23-9D40-B861B1FB5391},49057,1,2.973528,0.183659,"{""rings"": [[[-111.92092166399999, 41.138676415...",z57,Weber County
7,7,8,12,2.010121e+09,2010.0,JUAB,23,Central,10246,12017,{7A6D2476-AE18-48E8-820B-CE97B805879F},49023,4,6.802360,0.926379,"{""rings"": [[[-112.17814760399995, 40.011304953...",z23,Juab County
8,8,9,20,2.010201e+09,2010.0,SANPETE,39,Central,27822,30939,{323210BE-8E14-49FE-96FE-1C571E744993},49039,2,3.238312,0.433777,"{""rings"": [[[-111.60347983999998, 39.812889279...",z39,Sanpete County
9,9,10,27,2.010271e+09,2010.0,WASHINGTON,53,South,138115,177556,{A13328EF-DD99-45B8-897C-A9FE2418E7CA},49053,1,3.591173,0.639603,"{""rings"": [[[-113.47458777999998, 37.607168207...",z53,Washington County


,NAME_x,CODE3,COUNTYNBR,FID,OBJECTID,ENTITYNBR,ENTITYYR,NAME_y,FIPS,STATEPLANE,POP_LASTCE,POP_CURRES,GlobalID,FIPS_STR,COLOR4,SHAPE_Leng,SHAPE_Area,SHAPE,zCOFIPS,CO_NAME
0,Alpine,ALP,25,28,29,2.010251e+09,2010.0,UTAH,49,Central,516564,636235,{33E4105B-187C-4C0B-8F1C-D091F44BEFB1},49049,3,5.073804,0.586227,"{'rings': [[[-111.59404929699997, 40.577075300...",z49,Utah County
1,American Fork,AFK,25,28,29,2.010251e+09,2010.0,UTAH,49,Central,516564,636235,{33E4105B-187C-4C0B-8F1C-D091F44BEFB1},49049,3,5.073804,0.586227,"{'rings': [[[-111.59404929699997, 40.577075300...",z49,Utah County
2,Cedar Fort,CDF,25,28,29,2.010251e+09,2010.0,UTAH,49,Central,516564,636235,{33E4105B-187C-4C0B-8F1C-D091F44BEFB1},49049,3,5.073804,0.586227,"{'rings': [[[-111.59404929699997, 40.577075300...",z49,Utah County
3,Cedar Hills,CHL,25,28,29,2.010251e+09,2010.0,UTAH,49,Central,516564,636235,{33E4105B-187C-4C0B-8F1C-D091F44BEFB1},49049,3,5.073804,0.586227,"{'rings': [[[-111.59404929699997, 40.577075300...",z49,Utah County
4,Eagle Mountain,EAG,25,28,29,2.010251e+09,2010.0,UTAH,49,Central,516564,636235,{33E4105B-187C-4C0B-8F1C-D091F44BEFB1},49049,3,5.073804,0.586227,"{'rings': [[[-111.59404929699997, 40.577075300...",z49,Utah County
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,West Haven,WHV,29,6,7,2.010291e+09,2010.0,WEBER,57,North,231236,260213,{7F2E8776-C886-4F23-9D40-B861B1FB5391},49057,1,2.973528,0.183659,"{'rings': [[[-111.92092166399999, 41.138676415...",z57,Weber County
93,Grantsville,GRL,23,17,18,2.010231e+09,2010.0,TOOELE,45,Central,58218,72259,{197FEA12-E2B4-4514-9A3C-963D8244A1E3},49045,2,6.105650,2.003307,"{'rings': [[[-112.44788691599996, 41.017030151...",z45,Tooele County
94,Tooele,TOO,23,17,18,2.010231e+09,2010.0,TOOELE,45,Central,58218,72259,{197FEA12-E2B4-4514-9A3C-963D8244A1E3},49045,2,6.105650,2.003307,"{'rings': [[[-112.44788691599996, 41.017030151...",z45,Tooele County
95,Morgan,MRG,15,25,26,2.010151e+09,2010.0,MORGAN,29,North,9469,12124,{08CCA506-439E-4F9E-8302-50A6B9468EF5},49029,2,2.862360,0.169567,"{'rings': [[[-111.51438719299995, 40.803633660...",z29,Morgan County


,NAME,CODE3,zCOFIPS
0,Alpine,ALP,z49
1,American Fork,AFK,z49
2,Cedar Fort,CDF,z49
3,Cedar Hills,CHL,z49
4,Eagle Mountain,EAG,z49
...,...,...,...
92,West Haven,WHV,z57
93,Grantsville,GRL,z45
94,Tooele,TOO,z45
95,Morgan,MRG,z29


# Add Rows for County Level Totals

In [72]:
#read in csv with commute pattern data
df_LEHDCommutePatterns = pd.read_csv(fname_LEHDCommutePatterns_csv)

In [73]:
#get all column codes for summing
dfColCodes = df_LEHDCommutePatterns.loc[1:1,'AFK_h':'TTL_w'].T #get all columns from AFK_x through TTL_y
dfColCodes = dfColCodes.reset_index()

#convert to list for ease of calcs in future steps
dColCodes = dfColCodes['index'].values.tolist()
dColCodes

['AFK_h',
 'ALA_h',
 'ALP_h',
 'BDL_h',
 'BGM_h',
 'BNT_h',
 'BRT_h',
 'CDF_h',
 'CEN_h',
 'CHA_h',
 'CHL_h',
 'CLF_h',
 'CLI_h',
 'CMT_h',
 'COA_h',
 'CWH_h',
 'DAN_h',
 'DRA_h',
 'EAG_h',
 'ELK_h',
 'EMT_h',
 'FAR_h',
 'FCS_h',
 'FFD_h',
 'FRR_h',
 'FTH_h',
 'GLA_h',
 'GOS_h',
 'GRL_h',
 'HAR_h',
 'HDT_h',
 'HEB_h',
 'HER_h',
 'HGH_h',
 'HNF_h',
 'HOL_h',
 'HOO_h',
 'HVL_h',
 'IND_h',
 'INT_h',
 'KAY_h',
 'KMS_h',
 'KMT_h',
 'LAY_h',
 'LEH_h',
 'LIN_h',
 'MAP_h',
 'MID_h',
 'MLC_h',
 'MMT_h',
 'MRG_h',
 'MSL_h',
 'MUR_h',
 'MWY_h',
 'NOG_h',
 'NSL_h',
 'OGD_h',
 'OKL_h',
 'ORM_h',
 'PAY_h',
 'PGR_h',
 'PLN_h',
 'PRK_h',
 'PRY_h',
 'PVO_h',
 'PVW_h',
 'ROY_h',
 'RVD_h',
 'RVT_h',
 'SAN_h',
 'SAQ_h',
 'SAR_h',
 'SFK_h',
 'SJC_h',
 'SLC_h',
 'SLM_h',
 'SOG_h',
 'SPV_h',
 'SSL_h',
 'SUN_h',
 'SWE_h',
 'SYR_h',
 'TAY_h',
 'TOO_h',
 'UIN_h',
 'VIN_h',
 'WAT_h',
 'WBG_h',
 'WDL_h',
 'WEB_h',
 'WHT_h',
 'WHV_h',
 'WIL_h',
 'WJC_h',
 'WPT_h',
 'WVC_h',
 'WXC_h',
 'TTL_h',
 'AFK_w',
 'ALA_w',


In [74]:
#normalize (melt) commute data so that columns become rows
df_LEHDCommutePatterns_Normalized = pd.melt(df_LEHDCommutePatterns, id_vars=['fakeid','GEOID'], value_vars=dColCodes)

#populate new fields with data from previous column names (ABC_x/y): CODE3 and Category (_x/_y)
df_LEHDCommutePatterns_Normalized['CODE3'   ] = df_LEHDCommutePatterns_Normalized['variable'].str[0:3]
df_LEHDCommutePatterns_Normalized['CATEGORY'] = df_LEHDCommutePatterns_Normalized['variable'].str[3:5]

#join with City/Township data using CODE3
df_LEHDCommutePatterns_Normalized_wCOFIPS = pd.DataFrame.merge(df_CityTownship_NameCode, df_LEHDCommutePatterns_Normalized, on='CODE3')
display(df_LEHDCommutePatterns_Normalized_wCOFIPS)

#get county totals for each GEOID by aggregating (groupby)
df_LEHDCommutePatterns_CountyTotals_Normalized = df_LEHDCommutePatterns_Normalized_wCOFIPS.groupby(['zCOFIPS','GEOID','CATEGORY'],as_index=False).agg({'value':np.sum})

#create column names for county level totals
df_LEHDCommutePatterns_CountyTotals_Normalized['zCOFIPS_wCat'] = df_LEHDCommutePatterns_CountyTotals_Normalized['zCOFIPS'] + df_LEHDCommutePatterns_CountyTotals_Normalized['CATEGORY']

#create pivot table to reverse normalization, returning structure to original format, but this time with county columns
df_LEHDCommutePatterns_CountyTotals = df_LEHDCommutePatterns_CountyTotals_Normalized.pivot(index='GEOID', columns='zCOFIPS_wCat')['value']

display(df_LEHDCommutePatterns_CountyTotals)

,NAME,CODE3,zCOFIPS,fakeid,GEOID,variable,value,CATEGORY
0,Alpine,ALP,z49,0,colSUM,ALP_h,4439,_h
1,Alpine,ALP,z49,0,490011001001,ALP_h,0,_h
2,Alpine,ALP,z49,1,490011001002,ALP_h,0,_h
3,Alpine,ALP,z49,2,490011001003,ALP_h,0,_h
4,Alpine,ALP,z49,3,490011002001,ALP_h,0,_h
...,...,...,...,...,...,...,...,...
391875,Santaquin,SAQ,z23,2014,490572112021,SAQ_w,0,_w
391876,Santaquin,SAQ,z23,2015,490572112022,SAQ_w,0,_w
391877,Santaquin,SAQ,z23,2016,490572112023,SAQ_w,0,_w
391878,Santaquin,SAQ,z23,2017,490572112024,SAQ_w,0,_w


zCOFIPS_wCat,z03_h,z03_w,z11_h,z11_w,z23_h,z23_w,z29_h,z29_w,z35_h,z35_w,z43_h,z43_w,z45_h,z45_w,z49_h,z49_w,z51_h,z51_w,z57_h,z57_w
GEOID,,,,,,,,,,,,,,,,,,,,
490011001001,0,0,5,0,0,0,1,1,20,20,0,0,0,0,13,10,0,1,5,3
490011001002,1,0,5,4,1,0,0,0,11,53,0,3,0,0,12,20,0,0,3,7
490011001003,1,0,12,11,1,0,0,0,59,62,0,1,3,0,18,13,2,1,5,3
490011002001,1,0,0,0,0,0,0,0,9,20,0,1,2,1,12,5,0,2,0,1
490011002002,0,0,0,5,0,1,0,0,1,27,0,1,0,0,3,23,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490572112022,15,8,202,268,0,0,12,7,42,297,1,3,4,0,29,60,2,1,354,592
490572112023,4,4,45,108,0,0,2,2,6,127,1,0,2,1,7,19,0,0,83,235
490572112024,5,4,22,81,0,0,0,0,4,106,0,1,0,0,3,15,1,0,71,213


In [75]:
#add county columns to original dataset
df_LEHDCommutePatternsWCounty = pd.DataFrame.merge(df_LEHDCommutePatterns, df_LEHDCommutePatterns_CountyTotals, on='GEOID')
df_LEHDCommutePatterns = df_LEHDCommutePatternsWCounty

df_LEHDCommutePatterns

,fakeid,GEOID,AFK_h,ALA_h,ALP_h,BDL_h,BGM_h,BNT_h,BRT_h,CDF_h,...,z43_h,z43_w,z45_h,z45_w,z49_h,z49_w,z51_h,z51_w,z57_h,z57_w
0,0,colSUM,12283,162,4439,6482,9819,19759,71,322,...,6918,13798,21883,9402,249752,233966,11737,7438,107909,100189
1,0,490011001001,0,0,0,0,0,0,0,0,...,0,0,0,0,13,10,0,1,5,3
2,1,490011001002,1,0,0,0,0,1,0,0,...,0,3,0,0,12,20,0,0,3,7
3,2,490011001003,1,0,0,2,1,2,0,0,...,0,1,3,0,18,13,2,1,5,3
4,3,490011002001,1,0,0,0,1,0,0,0,...,0,1,2,1,12,5,0,2,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015,2014,490572112021,1,0,1,0,5,3,0,0,...,2,1,3,1,15,10,2,1,197,138
2016,2015,490572112022,2,0,0,0,10,13,0,0,...,1,3,4,0,29,60,2,1,354,592
2017,2016,490572112023,0,0,0,0,3,0,0,0,...,1,0,2,1,7,19,0,0,83,235
2018,2017,490572112024,0,0,0,0,4,0,0,0,...,0,1,0,0,3,15,1,0,71,213


In [76]:
#remove UofU displaced block group
#superceeded by manual reallocation prior to data input prep
#
display(df_LEHDCommutePatterns[df_LEHDCommutePatterns['GEOID']=='490351108004'])

#
#df_LEHDCommutePatterns = df_LEHDCommutePatterns[df_LEHDCommutePatterns['GEOID'] != '490351108004']

#display(df_LEHDCommutePatterns[df_LEHDCommutePatterns['GEOID']=='490351108004'])

,fakeid,GEOID,AFK_h,ALA_h,ALP_h,BDL_h,BGM_h,BNT_h,BRT_h,CDF_h,...,z43_h,z43_w,z45_h,z45_w,z49_h,z49_w,z51_h,z51_w,z57_h,z57_w
631,630,490351108004,0,0,2,1,2,5,0,0,...,1,8,11,2,50,53,3,2,14,12


# Quick JSON export with some summary stats

In [77]:
#use row with "colSUM" as GEOID as total
df_LEHDCommutePatterns_ColSum = df_LEHDCommutePatterns[df_LEHDCommutePatterns['GEOID'] == 'colSUM']
df_LEHDCommutePatterns_ColSum

,fakeid,GEOID,AFK_h,ALA_h,ALP_h,BDL_h,BGM_h,BNT_h,BRT_h,CDF_h,...,z43_h,z43_w,z45_h,z45_w,z49_h,z49_w,z51_h,z51_w,z57_h,z57_w
0,0,colSUM,12283,162,4439,6482,9819,19759,71,322,...,6918,13798,21883,9402,249752,233966,11737,7438,107909,100189


In [79]:
#prepare area columns for json
df_people = df_LEHDCommutePatterns_ColSum.loc[:,'AFK_h':'z57_w'].T #transpose
df_people = df_people.reset_index()
df_people['CATEGORY'] = df_people['index'].str[3:5]
df_people['index']    = df_people['index'].str[:3]
df_people

,index,0,CATEGORY
0,AFK,12283,_h
1,ALA,162,_h
2,ALP,4439,_h
3,BDL,6482,_h
4,BGM,9819,_h
...,...,...,...
211,z49,233966,_w
212,z51,11737,_h
213,z51,7438,_w
214,z57,107909,_h


In [85]:
#prepare x columns
df_people_x = df_people[df_people['CATEGORY'] == '_h']
df_people_x.columns = ('CODE3','people_h','CATEGORY')

df_people_x['CODE3'] = df_people_x['CODE3'].str[:3]

df_people_x = df_people_x.drop_duplicates() #some duplicate columns in dataset
display(df_people_x)

#prepare y columns
df_people_y = df_people[df_people['CATEGORY'] == '_w']
df_people_y.columns = ('CODE3','people_w','CATEGORY')

df_people_y['CODE3'] = df_people_y['CODE3'].str[:3]

df_people_y = df_people_y.drop_duplicates() #some duplicate columns in dataset
display(df_people_y)

df_people_x = df_people_x.drop(['CATEGORY'], axis=1)
df_people_y = df_people_y.drop(['CATEGORY'], axis=1)

#combine city/township data with x and y
df_CityTownship_People = pd.DataFrame.merge(df_CityTownship_NameCode,df_people_x, on='CODE3', how='right')
df_CityTownship_People = pd.DataFrame.merge(df_CityTownship_People  ,df_people_y, on='CODE3', how='right')                                           


C:\Users\cday\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,CODE3,people_h,CATEGORY
0,AFK,12283,_h
1,ALA,162,_h
2,ALP,4439,_h
3,BDL,6482,_h
4,BGM,9819,_h
...,...,...,...
206,z43,6918,_h
208,z45,21883,_h
210,z49,249752,_h
212,z51,11737,_h


C:\Users\cday\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,CODE3,people_w,CATEGORY
98,AFK,22429,_w
99,ALA,1475,_w
100,ALP,2785,_w
101,BDL,4403,_w
102,BGM,8977,_w
...,...,...,...
207,z43,13798,_w
209,z45,9402,_w
211,z49,233966,_w
213,z51,7438,_w


In [86]:
#define county dataframe
df_CoNames = sdf_CountiesAGRC[['zCOFIPS','CO_NAME']]

#prepare dataframe for export to JSON, combining city/township
df_CityTownship_Export = pd.DataFrame.merge(df_CityTownship_People, df_CoNames, left_on='CODE3', right_on='zCOFIPS', how='left')

#for any null values, replace with zero(0)
df_CityTownship_Export = df_CityTownship_Export.fillna(0)

#Get city/township name and county name into one field
df_CityTownship_Export.loc[df_CityTownship_Export.CO_NAME != 0, 'NAME'] = df_CityTownship_Export['CO_NAME']

#filter by only 4 columns for export
df_CityTownship_Export = df_CityTownship_Export[['NAME','CODE3','people_h','people_w']]

#change TTL (total) row to Counties subhead for correct positioning in dropdown

#don't include total row
df_CityTownship_Export = df_CityTownship_Export[df_CityTownship_Export['CODE3'] != 'TTL']

#df_CityTownship_Export.loc[((df_CityTownship_Export['CODE3'] == 'TTL')), 'NAME' ] = '---COUNTIES---'
#df_CityTownship_Export.loc[((df_CityTownship_Export['CODE3'] == 'TTL')), 'CODE3'] = 'y000'

#create a first letter field for sorting, to preserve COUNTIES with (z..) at bottom of sort, so can't use name as first sort
#df_CityTownship_Export['firstletter'] = df_CityTownship_Export['CODE3'].str[0:1]
#df_CityTownship_Export = df_CityTownship_Export.sort_values(by=['firstletter','NAME'])
df_CityTownship_Export = df_CityTownship_Export.sort_values(by=['NAME'])

#filter again by only 4 columns for export
df_CityTownship_Export = df_CityTownship_Export[['NAME','CODE3','people_h','people_w']]

#rename columns with 'label' and 'value' to work better in javascript dojo convention
df_CityTownship_Export.columns = ('label','value','people_h','people_w')

#let's see what we got!!
display(df_CityTownship_Export)
    
#export to JSON
df_CityTownship_Export.to_json(fname_CityTownshipJSON,orient='records')

,label,value,people_h,people_w
2,Alpine,ALP,4439,2785
1,Alta,ALA,162,1475
0,American Fork,AFK,12283,22429
3,Bluffdale,BDL,6482,4403
5,Bountiful,BNT,19759,11439
...,...,...,...,...
95,West Valley City,WVC,62980,68427
90,White City,WHT,2967,386
92,Willard,WIL,1016,183
88,Woodland Hills,WDL,832,186


# Update ColCodes with Counties for summing

In [87]:
#get all column codes for summing
dfColCodesWCounty = df_LEHDCommutePatterns.loc[1:1,'AFK_h':'z57_w'].T
dfColCodesWCounty = dfColCodesWCounty.reset_index()
dColCodesWCounty = dfColCodesWCounty['index'].values.tolist()
dColCodesWCounty

['AFK_h',
 'ALA_h',
 'ALP_h',
 'BDL_h',
 'BGM_h',
 'BNT_h',
 'BRT_h',
 'CDF_h',
 'CEN_h',
 'CHA_h',
 'CHL_h',
 'CLF_h',
 'CLI_h',
 'CMT_h',
 'COA_h',
 'CWH_h',
 'DAN_h',
 'DRA_h',
 'EAG_h',
 'ELK_h',
 'EMT_h',
 'FAR_h',
 'FCS_h',
 'FFD_h',
 'FRR_h',
 'FTH_h',
 'GLA_h',
 'GOS_h',
 'GRL_h',
 'HAR_h',
 'HDT_h',
 'HEB_h',
 'HER_h',
 'HGH_h',
 'HNF_h',
 'HOL_h',
 'HOO_h',
 'HVL_h',
 'IND_h',
 'INT_h',
 'KAY_h',
 'KMS_h',
 'KMT_h',
 'LAY_h',
 'LEH_h',
 'LIN_h',
 'MAP_h',
 'MID_h',
 'MLC_h',
 'MMT_h',
 'MRG_h',
 'MSL_h',
 'MUR_h',
 'MWY_h',
 'NOG_h',
 'NSL_h',
 'OGD_h',
 'OKL_h',
 'ORM_h',
 'PAY_h',
 'PGR_h',
 'PLN_h',
 'PRK_h',
 'PRY_h',
 'PVO_h',
 'PVW_h',
 'ROY_h',
 'RVD_h',
 'RVT_h',
 'SAN_h',
 'SAQ_h',
 'SAR_h',
 'SFK_h',
 'SJC_h',
 'SLC_h',
 'SLM_h',
 'SOG_h',
 'SPV_h',
 'SSL_h',
 'SUN_h',
 'SWE_h',
 'SYR_h',
 'TAY_h',
 'TOO_h',
 'UIN_h',
 'VIN_h',
 'WAT_h',
 'WBG_h',
 'WDL_h',
 'WEB_h',
 'WHT_h',
 'WHV_h',
 'WIL_h',
 'WJC_h',
 'WPT_h',
 'WVC_h',
 'WXC_h',
 'TTL_h',
 'AFK_w',
 'ALA_w',


# Add Unincorporated County Areas to Cities/Townships

In [88]:
#filter county dataset with only what we want
sdf_CountiesWCodes = sdf_CountiesAGRC[['zCOFIPS','CO_NAME','SHAPE']]
sdf_CountiesWCodes.columns = ('CODE3','FIRST_NAME','SHAPE')
display(sdf_CountiesWCodes)

#export a couple county spatial dataframes
sdf_CountiesWCodes.spatial.to_featureclass(fname_CountyWCodes_shp) #as shapefile for future use
sdf_CountiesWCodes.spatial.to_featureclass(fname_Counties) #for gdb for use in web app

,CODE3,FIRST_NAME,SHAPE
0,z05,Cache County,"{""rings"": [[[-112.15617197899996, 41.997725941..."
1,z13,Duchesne County,"{""rings"": [[[-110.25173670299995, 40.832347463..."
2,z11,Davis County,"{""rings"": [[[-111.92092166399999, 41.138676415..."
3,z27,Millard County,"{""rings"": [[[-112.23854898599996, 39.553633436..."
4,z51,Wasatch County,"{""rings"": [[[-111.39572506599995, 40.690187842..."
5,z03,Box Elder County,"{""rings"": [[[-113.47488592699995, 41.993309448..."
6,z57,Weber County,"{""rings"": [[[-111.92092166399999, 41.138676415..."
7,z23,Juab County,"{""rings"": [[[-112.17814760399995, 40.011304953..."
8,z39,Sanpete County,"{""rings"": [[[-111.60347983999998, 39.812889279..."
9,z53,Washington County,"{""rings"": [[[-113.47458777999998, 37.607168207..."


C:\Users\cday\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


'c:\\Users\\cday\\Documents\\projects\\Census-LEHD-Commute-Patterns\\results\\results2018\\LEHDCommutePatterns.gdb\\Counties'

In [97]:
deleteIfExists(fname_CityTownshipWCodesUnionCounty_shp)

print ("Unioning city/township with county...")

#intersection parcels with analysis area types
arcpy.analysis.Union([fname_CityTownshipWCodes,fname_CountyWCodes_shp], fname_CityTownshipWCodesUnionCounty_shp)

sdf_CityTownshipWCodesCounty = pd.DataFrame.spatial.from_featureclass(fname_CityTownshipWCodesUnionCounty_shp)
sdf_CityTownshipWCodesCounty = sdf_CityTownshipWCodesCounty.rename(columns={"code3":"CODE3", "FIRST_name":"FIRST_NAME", "FIRST_coun":"FIRST_COUN", "first_na_1":"FIRST_NA_1", "code3_1":"CODE3_1"})
sdf_CityTownshipWCodesCounty

Unioning city/township with county...


,FID,FID_Munici,CODE3,FIRST_NAME,FIRST_COUN,Shape_Leng,Shape_Area,FID_Counti,Id,CODE3_1,FIRST_NA_1,SHAPE
0,0,-1,,,,0.000000,0.000000,0,0,z05,Cache County,"{""rings"": [[[-111.50779761699998, 41.999560240..."
1,1,-1,,,,0.000000,0.000000,1,0,z13,Duchesne County,"{""rings"": [[[-109.99900220099995, 40.813577918..."
2,2,-1,,,,0.000000,0.000000,2,0,z11,Davis County,"{""rings"": [[[-112.10299823793908, 41.152736052..."
3,3,-1,,,,0.000000,0.000000,3,0,z27,Millard County,"{""rings"": [[[-112.01601400799998, 39.314566632..."
4,4,-1,,,,0.000000,0.000000,4,0,z51,Wasatch County,"{""rings"": [[[-111.38703771390648, 40.657177632..."
...,...,...,...,...,...,...,...,...,...,...,...,...
154,154,75,SLC,Salt Lake City,18,1.346029,0.030613,27,0,z35,Salt Lake County,"{""rings"": [[[-112.10123724564913, 40.775409823..."
155,155,50,MMT,Magna,18,0.756349,0.010342,27,0,z35,Salt Lake County,"{""rings"": [[[-112.06310238736523, 40.703812554..."
156,156,96,WVC,West Valley City,18,0.614523,0.009884,27,0,z35,Salt Lake County,"{""rings"": [[[-112.06310238736523, 40.703812554..."
157,157,70,SAN,Sandy,18,1.239008,0.006653,27,0,z35,Salt Lake County,"{""rings"": [[[-111.86188875086515, 40.580804996..."


In [98]:
sdf_CityTownshipWCodesCounty.loc[((sdf_CityTownshipWCodesCounty['FID_Munici'] == -1)), 'CODE3'     ] = sdf_CityTownshipWCodesCounty['CODE3_1'   ]
sdf_CityTownshipWCodesCounty.loc[((sdf_CityTownshipWCodesCounty['FID_Munici'] == -1)), 'FIRST_NAME'] = sdf_CityTownshipWCodesCounty['FIRST_NA_1'] + ' Unincorporated'

sdf_CityTownshipWCodesCounty = sdf_CityTownshipWCodesCounty[['FID_Munici','CODE3','FIRST_NAME','SHAPE']]

sdf_CityTownshipWCodesCounty.spatial.to_featureclass(fname_CityTownshipCountyWCodes_shp)
sdf_CityTownshipWCodesCounty

C:\Users\cday\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,FID_Munici,CODE3,FIRST_NAME,SHAPE
0,-1,z05,Cache County Unincorporated,"{""rings"": [[[-111.50779761699998, 41.999560240..."
1,-1,z13,Duchesne County Unincorporated,"{""rings"": [[[-109.99900220099995, 40.813577918..."
2,-1,z11,Davis County Unincorporated,"{""rings"": [[[-112.10299823793908, 41.152736052..."
3,-1,z27,Millard County Unincorporated,"{""rings"": [[[-112.01601400799998, 39.314566632..."
4,-1,z51,Wasatch County Unincorporated,"{""rings"": [[[-111.38703771390648, 40.657177632..."
...,...,...,...,...
154,75,SLC,Salt Lake City,"{""rings"": [[[-112.10123724564913, 40.775409823..."
155,50,MMT,Magna,"{""rings"": [[[-112.06310238736523, 40.703812554..."
156,96,WVC,West Valley City,"{""rings"": [[[-112.06310238736523, 40.703812554..."
157,70,SAN,Sandy,"{""rings"": [[[-111.86188875086515, 40.580804996..."


# Calculate Percent

In [99]:
#read block group shapefile
sdf_BlockGroups = pd.DataFrame.spatial.from_featureclass(fname_BlockGroups_shp)
sdf_LEHDCommutePatterns = pd.DataFrame.merge(sdf_BlockGroups, df_LEHDCommutePatterns, on='GEOID')
display(sdf_LEHDCommutePatterns)

,FID,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,GEOID,NAMELSAD,MTFCC,FUNCSTAT,ALAND,...,z43_h,z43_w,z45_h,z45_w,z49_h,z49_w,z51_h,z51_w,z57_h,z57_w
0,0,49,035,111306,1,490351113061,Block Group 1,G5030,S,877837.0,...,12,11,14,2,117,70,6,1,15,14
1,1,49,035,111306,2,490351113062,Block Group 2,G5030,S,653262.0,...,0,1,0,3,5,36,0,2,4,6
2,2,49,035,111400,1,490351114001,Block Group 1,G5030,S,381555.0,...,0,9,1,3,6,37,1,0,5,7
3,3,49,035,111400,2,490351114002,Block Group 2,G5030,S,308108.0,...,1,5,3,1,5,16,0,0,2,9
4,4,49,035,111400,3,490351114003,Block Group 3,G5030,S,422651.0,...,0,11,1,0,1,40,0,0,0,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014,2015,49,057,200900,2,490572009002,Block Group 2,G5030,S,347700.0,...,1,4,2,0,3,28,0,1,122,329
2015,2016,49,057,201200,1,490572012001,Block Group 1,G5030,S,752190.0,...,2,1,1,2,9,28,2,1,264,252
2016,2017,49,057,201200,2,490572012002,Block Group 2,G5030,S,437903.0,...,3,0,12,1,88,27,5,1,595,210
2017,2018,49,057,201301,2,490572013012,Block Group 2,G5030,S,326205.0,...,2,0,0,3,4,20,0,1,76,254


In [100]:
pd.set_option('display.max_columns', None)
df_LEHDCommutePatterns.loc[1:,'AFK_h':]

,AFK_h,ALA_h,ALP_h,BDL_h,BGM_h,BNT_h,BRT_h,CDF_h,CEN_h,CHA_h,CHL_h,CLF_h,CLI_h,CMT_h,COA_h,CWH_h,DAN_h,DRA_h,EAG_h,ELK_h,EMT_h,FAR_h,FCS_h,FFD_h,FRR_h,FTH_h,GLA_h,GOS_h,GRL_h,HAR_h,HDT_h,HEB_h,HER_h,HGH_h,HNF_h,HOL_h,HOO_h,HVL_h,IND_h,INT_h,KAY_h,KMS_h,KMT_h,LAY_h,LEH_h,LIN_h,MAP_h,MID_h,MLC_h,MMT_h,MRG_h,MSL_h,MUR_h,MWY_h,NOG_h,NSL_h,OGD_h,OKL_h,ORM_h,PAY_h,PGR_h,PLN_h,PRK_h,PRY_h,PVO_h,PVW_h,ROY_h,RVD_h,RVT_h,SAN_h,SAQ_h,SAR_h,SFK_h,SJC_h,SLC_h,SLM_h,SOG_h,SPV_h,SSL_h,SUN_h,SWE_h,SYR_h,TAY_h,TOO_h,UIN_h,VIN_h,WAT_h,WBG_h,WDL_h,WEB_h,WHT_h,WHV_h,WIL_h,WJC_h,WPT_h,WVC_h,WXC_h,TTL_h,AFK_w,ALA_w,ALP_w,BDL_w,BGM_w,BNT_w,BRT_w,CDF_w,CEN_w,CHA_w,CHL_w,CLF_w,CLI_w,CMT_w,COA_w,CWH_w,DAN_w,DRA_w,EAG_w,ELK_w,EMT_w,FAR_w,FCS_w,FFD_w,FRR_w,FTH_w,GLA_w,GOS_w,GRL_w,HAR_w,HDT_w,HEB_w,HER_w,HGH_w,HNF_w,HOL_w,HOO_w,HVL_w,IND_w,INT_w,KAY_w,KMS_w,KMT_w,LAY_w,LEH_w,LIN_w,MAP_w,MID_w,MLC_w,MMT_w,MRG_w,MSL_w,MUR_w,MWY_w,NOG_w,NSL_w,OGD_w,OKL_w,ORM_w,PAY_w,PGR_w,PLN_w,PRK_w,PRY_w,PVO_w,PVW_w,ROY_w,RVD_w,RVT_w,SAN_w,SAQ_w,SAR_w,SFK_w,SJC_w,SLC_w,SLM_w,SOG_w,SPV_w,SSL_w,SUN_w,SWE_w,SYR_w,TAY_w,TOO_w,UIN_w,VIN_w,WAT_w,WBG_w,WDL_w,WEB_w,WHT_w,WHV_w,WIL_w,WJC_w,WPT_w,WVC_w,WXC_w,TTL_w,z03_h,z03_w,z11_h,z11_w,z23_h,z23_w,z29_h,z29_w,z35_h,z35_w,z43_h,z43_w,z45_h,z45_w,z49_h,z49_w,z51_h,z51_w,z57_h,z57_w
1,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,1,1,1,1,1,0,0,0,0,0,1,0,2,1,0,1,0,0,8,0,1,0,1,2,0,0,1,1,2,0,0,0,1,0,0,1,2,0,0,0,0,0,0,0,0,2,0,2,1,4,0,373,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,2,0,2,2,0,0,0,0,1,0,0,0,0,2,0,0,2,0,11,0,0,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,2,0,246,0,0,5,0,0,0,1,1,20,20,0,0,0,0,13,10,0,1,5,3
2,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,1,0,0,0,0,0,1,0,1,0,1,0,1,0,0,0,6,0,0,0,0,6,1,0,1,0,2,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,1,0,0,1,0,268,1,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,2,0,1,0,0,0,0,1,1,0,0,0,4,0,0,0,2,1,7,0,0,0,2,0,8,1,1,1,0,2,0,0,3,0,22,0,1,0,5,0,0,0,2,0,0,0,0,0,0,0,0,1,0,5,0,7,0,562,1,0,5,4,1,0,0,0,11,53,0,3,0,0,12,20,0,0,3,7
3,1,0,0,2,1,2,0,0,0,0,1,0,1,0,0,1,1,3,1,0,0,1,0,0,0,0,0,0,0,0,0,1,1,1,0,2,0,0,0,0,1,0,2,2,1,0,1,2,5,3,0,0,2,0,0,3,1,0,0,1,0,0,0,0,8,0,2,1,0,5,1,1,1,2,12,1,0,0,2,1,0,1,3,3,0,0,1,0,0,0,0,0,0,5,0,7,0,913,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,2,0,0,0,1,0,0,0,0,0,1,1,0,0,2,0,0,0,0,1,0,0,1,0,1,0,3,6,0,0,0,11,0,0,2,2,0,3,0,2,0,1,0,6,0,0,1,2,2,0,0,0,0,20,0,0,0,4,0,0,1,1,0,0,0,0,0,0,0,0,0,0,3,0,5,1,531,1,0,12,11,1,0,0,0,59,62,0,1,3,0,18,13,2,1,5,3
4,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,3,0,1,1,0,2,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,349,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,2,0,0,0,0,0,0,1,0,0,1,0,1,0,1,0,0,0,0,0,0,0,7,0,0,1,2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,2,0,6,0,217,1,0,0,0,0,0,0,0,9,20,0,1,2,1,12,5,0,2,0,1
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,30,2,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2,2,1,2,2,0,0,0,1,0,1,0,2,0,2,5,0,0,0,0,7,0,0,0,0,6,1,0,0,1,7,0,0,1,2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,290,0,0,0,5,0,1,0,0,1,27,0,1,0,0,3,23,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [101]:
sdf_LEHDCommutePatterns

,FID,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,GEOID,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,BGAcres,SHAPE,fakeid,AFK_h,ALA_h,ALP_h,BDL_h,BGM_h,BNT_h,BRT_h,CDF_h,CEN_h,CHA_h,CHL_h,CLF_h,CLI_h,CMT_h,COA_h,CWH_h,DAN_h,DRA_h,EAG_h,ELK_h,EMT_h,FAR_h,FCS_h,FFD_h,FRR_h,FTH_h,GLA_h,GOS_h,GRL_h,HAR_h,HDT_h,HEB_h,HER_h,HGH_h,HNF_h,HOL_h,HOO_h,HVL_h,IND_h,INT_h,KAY_h,KMS_h,KMT_h,LAY_h,LEH_h,LIN_h,MAP_h,MID_h,MLC_h,MMT_h,MRG_h,MSL_h,MUR_h,MWY_h,NOG_h,NSL_h,OGD_h,OKL_h,ORM_h,PAY_h,PGR_h,PLN_h,PRK_h,PRY_h,PVO_h,PVW_h,ROY_h,RVD_h,RVT_h,SAN_h,SAQ_h,SAR_h,SFK_h,SJC_h,SLC_h,SLM_h,SOG_h,SPV_h,SSL_h,SUN_h,SWE_h,SYR_h,TAY_h,TOO_h,UIN_h,VIN_h,WAT_h,WBG_h,WDL_h,WEB_h,WHT_h,WHV_h,WIL_h,WJC_h,WPT_h,WVC_h,WXC_h,TTL_h,AFK_w,ALA_w,ALP_w,BDL_w,BGM_w,BNT_w,BRT_w,CDF_w,CEN_w,CHA_w,CHL_w,CLF_w,CLI_w,CMT_w,COA_w,CWH_w,DAN_w,DRA_w,EAG_w,ELK_w,EMT_w,FAR_w,FCS_w,FFD_w,FRR_w,FTH_w,GLA_w,GOS_w,GRL_w,HAR_w,HDT_w,HEB_w,HER_w,HGH_w,HNF_w,HOL_w,HOO_w,HVL_w,IND_w,INT_w,KAY_w,KMS_w,KMT_w,LAY_w,LEH_w,LIN_w,MAP_w,MID_w,MLC_w,MMT_w,MRG_w,MSL_w,MUR_w,MWY_w,NOG_w,NSL_w,OGD_w,OKL_w,ORM_w,PAY_w,PGR_w,PLN_w,PRK_w,PRY_w,PVO_w,PVW_w,ROY_w,RVD_w,RVT_w,SAN_w,SAQ_w,SAR_w,SFK_w,SJC_w,SLC_w,SLM_w,SOG_w,SPV_w,SSL_w,SUN_w,SWE_w,SYR_w,TAY_w,TOO_w,UIN_w,VIN_w,WAT_w,WBG_w,WDL_w,WEB_w,WHT_w,WHV_w,WIL_w,WJC_w,WPT_w,WVC_w,WXC_w,TTL_w,z03_h,z03_w,z11_h,z11_w,z23_h,z23_w,z29_h,z29_w,z35_h,z35_w,z43_h,z43_w,z45_h,z45_w,z49_h,z49_w,z51_h,z51_w,z57_h,z57_w
0,0,49,035,111306,1,490351113061,Block Group 1,G5030,S,877837.0,0.0,+40.6094422,-111.8259365,216.918868,"{'rings': [[[-111.833821, 40.615462], [-111.83...",668,10,1,2,10,0,39,0,0,10,0,0,9,3,0,0,100,1,58,7,2,1,6,0,0,1,0,0,0,3,0,0,3,19,3,1,50,1,0,1,0,24,1,19,14,25,3,1,58,91,4,2,0,58,0,2,17,5,2,18,3,9,0,8,1,13,0,2,0,28,164,0,14,4,57,620,0,0,2,28,1,2,6,35,11,0,0,3,1,1,3,4,1,0,59,0,36,6,1922,5,5,0,1,0,1,2,0,1,0,0,2,1,0,0,62,0,30,0,0,0,2,0,0,0,0,0,0,0,0,0,1,5,0,0,32,0,0,0,0,3,0,4,11,23,3,0,35,70,2,0,0,71,0,0,5,9,0,12,1,6,0,11,0,12,0,1,1,6,76,0,3,2,24,254,0,1,2,33,0,0,1,17,2,0,1,0,0,0,1,0,2,0,29,0,65,1,989,1,0,140,29,0,0,2,0,1500,823,12,11,14,2,117,70,6,1,15,14
1,1,49,035,111306,2,490351113062,Block Group 2,G5030,S,653262.0,0.0,+40.6068923,-111.8144187,161.424738,"{'rings': [[[-111.821587, 40.607759], [-111.82...",669,0,0,0,0,0,1,0,0,2,0,0,0,0,0,0,9,0,2,2,1,0,1,0,0,0,0,0,0,0,0,0,0,4,0,0,3,1,0,0,0,0,0,0,1,0,0,1,1,4,0,0,0,3,0,1,0,2,0,1,0,0,0,0,0,0,0,0,0,2,7,0,0,0,2,11,0,0,0,2,0,0,0,2,0,0,0,0,0,0,0,0,0,0,2,0,1,0,74,3,5,0,0,0,6,0,0,1,0,0,1,0,0,0,31,0,17,0,0,0,1,0,0,0,0,0,0,2,0,0,1,2,0,0,7,0,0,0,0,0,0,2,4,14,3,0,24,31,0,0,0,36,1,0,3,6,0,3,0,1,0,1,0,11,0,0,0,2,42,0,0,0,9,132,0,0,0,15,0,0,0,12,1,0,1,0,0,0,0,0,0,0,19,0,32,2,506,0,0,5,18,0,0,0,0,55,418,0,1,0,3,5,36,0,2,4,6
2,2,49,035,111400,1,490351114001,Block Group 1,G5030,S,381555.0,0.0,+40.7219003,-111.8796932,94.284721,"{'rings': [[[-111.882588, 40.718416999999995],...",670,0,0,1,0,1,1,0,0,1,0,0,0,0,0,0,2,0,4,0,0,1,0,0,0,0,0,0,0,0,0,0,1,3,0,0,5,0,0,0,0,1,0,4,0,3,0,0,1,3,2,0,0,2,0,1,1,3,0,2,0,0,0,0,0,0,1,0,0,2,6,0,0,0,2,16,0,0,0,14,0,0,2,3,1,0,0,0,0,0,0,0,0,0,9,1,11,1,119,5,0,0,0,0,4,0,0,1,0,0,3,0,0,0,9,0,7,2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,3,2,7,3,0,9,64,1,0,0,25,0,0,8,3,0,4,0,2,0,9,0,9,0,2,1,4,30,0,1,3,14,206,0,1,1,42,0,0,0,13,3,0,0,0,0,0,0,0,0,0,14,0,31,4,569,1,0,8,23,0,0,0,0,90,474,0,9,1,3,6,37,1,0,5,7
3,3,49,035,111400,2,490351114002,Block Group 2,G5030,S,308108.0,0.0,+40.7155063,-111.8797286,76.134594,"{'rings': [[[-111.882623, 40.712824999999995],...",671,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,4,1,0,0,0,2,3,2,0,1,3,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,7,0,0,0,1,19,0,0,1,5,0,0,0,3,3,0,0,0,0,0,0,0,0,0,3,0,4,0,79,1,2,0,2,0,6,0,0,0,0,0,0,0,0,0,11,0,7,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,4,0,3,3,0,6,31,4,0,0,23,0,0,5,7,0,4,0,1,0,5,0,3,0,1,0,1,17,0,0,0,9,109,0,0,1,31,0,0,0,11,1,0,0,0,0,0,0,0,0,0,10,0,22,2,356,1,0,5,13,0,0,0,0,58,303,1,5,3,1,5,16,0,0,2,9
4,4,49,035,111400,3,490351114003,Block Group 3,G5030,S,422651.0,0.0,+40.7087113,-1

In [103]:
df_LEHDCommutePatterns.iloc[0,2:218]

AFK_h     12283
ALA_h       162
ALP_h      4439
BDL_h      6482
BGM_h      9819
          ...  
z49_w    233966
z51_h     11737
z51_w      7438
z57_h    107909
z57_w    100189
Name: 0, Length: 216, dtype: object

In [104]:
#initialize Percent_SA distribution dataframe
df_LEHDCommutePatterns_Percent_SA = df_LEHDCommutePatterns

#divide columns AFK_x and after by first row (totals) and columns 2:218
df_LEHDCommutePatterns_Percent_SA.loc[1:,'AFK_h':] = df_LEHDCommutePatterns.loc[1:,'AFK_h':].div(df_LEHDCommutePatterns.iloc[0,2:218])
df_LEHDCommutePatterns_Percent_SA.loc[1:,'AFK_h':] = df_LEHDCommutePatterns.loc[1:,'AFK_h':] * 100
df_LEHDCommutePatterns_Percent_SA

,fakeid,GEOID,AFK_h,ALA_h,ALP_h,BDL_h,BGM_h,BNT_h,BRT_h,CDF_h,CEN_h,CHA_h,CHL_h,CLF_h,CLI_h,CMT_h,COA_h,CWH_h,DAN_h,DRA_h,EAG_h,ELK_h,EMT_h,FAR_h,FCS_h,FFD_h,FRR_h,FTH_h,GLA_h,GOS_h,GRL_h,HAR_h,HDT_h,HEB_h,HER_h,HGH_h,HNF_h,HOL_h,HOO_h,HVL_h,IND_h,INT_h,KAY_h,KMS_h,KMT_h,LAY_h,LEH_h,LIN_h,MAP_h,MID_h,MLC_h,MMT_h,MRG_h,MSL_h,MUR_h,MWY_h,NOG_h,NSL_h,OGD_h,OKL_h,ORM_h,PAY_h,PGR_h,PLN_h,PRK_h,PRY_h,PVO_h,PVW_h,ROY_h,RVD_h,RVT_h,SAN_h,SAQ_h,SAR_h,SFK_h,SJC_h,SLC_h,SLM_h,SOG_h,SPV_h,SSL_h,SUN_h,SWE_h,SYR_h,TAY_h,TOO_h,UIN_h,VIN_h,WAT_h,WBG_h,WDL_h,WEB_h,WHT_h,WHV_h,WIL_h,WJC_h,WPT_h,WVC_h,WXC_h,TTL_h,AFK_w,ALA_w,ALP_w,BDL_w,BGM_w,BNT_w,BRT_w,CDF_w,CEN_w,CHA_w,CHL_w,CLF_w,CLI_w,CMT_w,COA_w,CWH_w,DAN_w,DRA_w,EAG_w,ELK_w,EMT_w,FAR_w,FCS_w,FFD_w,FRR_w,FTH_w,GLA_w,GOS_w,GRL_w,HAR_w,HDT_w,HEB_w,HER_w,HGH_w,HNF_w,HOL_w,HOO_w,HVL_w,IND_w,INT_w,KAY_w,KMS_w,KMT_w,LAY_w,LEH_w,LIN_w,MAP_w,MID_w,MLC_w,MMT_w,MRG_w,MSL_w,MUR_w,MWY_w,NOG_w,NSL_w,OGD_w,OKL_w,ORM_w,PAY_w,PGR_w,PLN_w,PRK_w,PRY_w,PVO_w,PVW_w,ROY_w,RVD_w,RVT_w,SAN_w,SAQ_w,SAR_w,SFK_w,SJC_w,SLC_w,SLM_w,SOG_w,SPV_w,SSL_w,SUN_w,SWE_w,SYR_w,TAY_w,TOO_w,UIN_w,VIN_w,WAT_w,WBG_w,WDL_w,WEB_w,WHT_w,WHV_w,WIL_w,WJC_w,WPT_w,WVC_w,WXC_w,TTL_w,z03_h,z03_w,z11_h,z11_w,z23_h,z23_w,z29_h,z29_w,z35_h,z35_w,z43_h,z43_w,z45_h,z45_w,z49_h,z49_w,z51_h,z51_w,z57_h,z57_w
0,0,colSUM,12283,162,4439,6482,9819,19759,71,322,7303,252,4420,13201,9986,462,759,16463,543,22021,14490,1554,737,9204,443,79,3038,2167,805,484,5114,3029,338,8019,16868,7738,358,14254,3663,278,119,50,12693,945,18712,31664,28872,5063,4592,16743,29657,14422,1852,926,25017,2250,9082,8249,39586,566,44362,9726,15899,3050,3847,2633,39581,4090,18679,4214,23541,47586,5590,13715,19747,32162,88595,4101,7846,16558,11575,2604,3068,12129,30535,16769,619,90,4503,166,832,3131,2967,5306,1016,58202,4689,62980,5059,1390165,22429,1475,2785,4403,8977,11439,60,52,5640,189,845,16589,2518,76,607,20362,183,37678,2057,310,135,9149,60,101,3082,446,162,57,1686,1616,7,5824,4452,3028,46,10117,674,74,13,0,8034,1153,4298,26964,33202,12517,1921,18321,48572,3019,1254,1821,44412,1168,2792,14796,58127,252,47620,6246,10685,845,11680,906,58416,2013,7599,7759,10200,47939,1068,3498,12722,29894,238330,1602,6794,12550,34040,1194,636,3763,20760,7716,260,975,3351,54,186,2169,386,3382,183,35776,1049,68427,4753,1411279,13468,10066,144906,109139,5590,1068,1852,1254,540214,683132,6918,13798,21883,9402,249752,233966,11737,7438,107909,100189
1,0,490011001001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01515,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.005928,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.005344,0.003158,0.0,0.0,0.021777,0.005973,0.003372,0.006934,0.053996,0.0,0.0,0.0,0.0,0.0,0.002526,0.0,0.004508,0.010282,0.0,0.032787,0.0,0.0,0.020212,0.0,0.005354,0.0,0.004248,0.004203,0.0,0.0,0.005064,0.003109,0.002257,0.0,0.0,0.0,0.008639,0.0,0.0,0.008245,0.00655,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.037693,0.0,0.003436,0.021327,0.006351,0.0,0.026831,0.004459,0.067797,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01717,0.0,0.0,0.0,0.009884,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.003012,0.0,0.0,0.0,0.0,0.0,0.079745,0.054915,0.0,0.0,0.0,0.0,0.003441,0.0,0.0042,0.03202,0.0,0.0,0.0,0.0,0.001712,0.0,0.0,0.0,0.0,0.004172,0.0,0.0,0.015721,0.0,0.004615,0.0,0.0,0.007968,0.005875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.002795,0.0,0.002923,0.0,0.017431,0.0,0.0,0.003451,0.0,0.0,0.0,0.053996,0.079745,0.003702,0.002928,0.0,0.0,0.0,0.0,0.005205,0.004274,0.0,0.013444,0.004634,0.002994
2,1,490011001002,0.008141,0.0,0.0,0.0,0.0,0.005061,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.010865,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.009474,0.0,0.0,0.0,0.0,0.003372,0.0,0.0,0.0,0.0,0.0,0.011011,0.0,0.002526,0.0,0.002254,0.0,0.00629,0.0,0.0,0.0,0.015159,0.0,0.0,0.0,0.0,0.012609,0.017889,0.0,0.005064,0.0,0.002257,0.0,0.0,0.006039,0.0,0.0,0.0,0.0,0

In [105]:
sdf_BlockGroups = pd.DataFrame.spatial.from_featureclass(fname_BlockGroups_shp)
sdf_LEHDCommutePatterns_Percent_SA = pd.DataFrame.merge(sdf_BlockGroups, df_LEHDCommutePatterns_Percent_SA, on='GEOID')
sdf_LEHDCommutePatterns_Percent_SA

,FID,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,GEOID,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,BGAcres,SHAPE,fakeid,AFK_h,ALA_h,ALP_h,BDL_h,BGM_h,BNT_h,BRT_h,CDF_h,CEN_h,CHA_h,CHL_h,CLF_h,CLI_h,CMT_h,COA_h,CWH_h,DAN_h,DRA_h,EAG_h,ELK_h,EMT_h,FAR_h,FCS_h,FFD_h,FRR_h,FTH_h,GLA_h,GOS_h,GRL_h,HAR_h,HDT_h,HEB_h,HER_h,HGH_h,HNF_h,HOL_h,HOO_h,HVL_h,IND_h,INT_h,KAY_h,KMS_h,KMT_h,LAY_h,LEH_h,LIN_h,MAP_h,MID_h,MLC_h,MMT_h,MRG_h,MSL_h,MUR_h,MWY_h,NOG_h,NSL_h,OGD_h,OKL_h,ORM_h,PAY_h,PGR_h,PLN_h,PRK_h,PRY_h,PVO_h,PVW_h,ROY_h,RVD_h,RVT_h,SAN_h,SAQ_h,SAR_h,SFK_h,SJC_h,SLC_h,SLM_h,SOG_h,SPV_h,SSL_h,SUN_h,SWE_h,SYR_h,TAY_h,TOO_h,UIN_h,VIN_h,WAT_h,WBG_h,WDL_h,WEB_h,WHT_h,WHV_h,WIL_h,WJC_h,WPT_h,WVC_h,WXC_h,TTL_h,AFK_w,ALA_w,ALP_w,BDL_w,BGM_w,BNT_w,BRT_w,CDF_w,CEN_w,CHA_w,CHL_w,CLF_w,CLI_w,CMT_w,COA_w,CWH_w,DAN_w,DRA_w,EAG_w,ELK_w,EMT_w,FAR_w,FCS_w,FFD_w,FRR_w,FTH_w,GLA_w,GOS_w,GRL_w,HAR_w,HDT_w,HEB_w,HER_w,HGH_w,HNF_w,HOL_w,HOO_w,HVL_w,IND_w,INT_w,KAY_w,KMS_w,KMT_w,LAY_w,LEH_w,LIN_w,MAP_w,MID_w,MLC_w,MMT_w,MRG_w,MSL_w,MUR_w,MWY_w,NOG_w,NSL_w,OGD_w,OKL_w,ORM_w,PAY_w,PGR_w,PLN_w,PRK_w,PRY_w,PVO_w,PVW_w,ROY_w,RVD_w,RVT_w,SAN_w,SAQ_w,SAR_w,SFK_w,SJC_w,SLC_w,SLM_w,SOG_w,SPV_w,SSL_w,SUN_w,SWE_w,SYR_w,TAY_w,TOO_w,UIN_w,VIN_w,WAT_w,WBG_w,WDL_w,WEB_w,WHT_w,WHV_w,WIL_w,WJC_w,WPT_w,WVC_w,WXC_w,TTL_w,z03_h,z03_w,z11_h,z11_w,z23_h,z23_w,z29_h,z29_w,z35_h,z35_w,z43_h,z43_w,z45_h,z45_w,z49_h,z49_w,z51_h,z51_w,z57_h,z57_w
0,0,49,035,111306,1,490351113061,Block Group 1,G5030,S,877837.0,0.0,+40.6094422,-111.8259365,216.918868,"{'rings': [[[-111.833821, 40.615462], [-111.83...",668,0.081413,0.617284,0.045055,0.154273,0.0,0.197378,0.0,0.0,0.13693,0.0,0.0,0.068177,0.030042,0.0,0.0,0.607423,0.184162,0.263385,0.048309,0.1287,0.135685,0.065189,0.0,0.0,0.032916,0.0,0.0,0.0,0.058662,0.0,0.0,0.037411,0.112639,0.03877,0.27933,0.350779,0.0273,0.0,0.840336,0.0,0.189081,0.10582,0.101539,0.044214,0.086589,0.059253,0.021777,0.346413,0.306842,0.027735,0.107991,0.0,0.231842,0.0,0.022022,0.206086,0.012631,0.353357,0.040575,0.030845,0.056607,0.0,0.207954,0.037979,0.032844,0.0,0.010707,0.0,0.118941,0.344639,0.0,0.102078,0.020256,0.177228,0.699814,0.0,0.0,0.012079,0.241901,0.038402,0.065189,0.049468,0.114623,0.065597,0.0,0.0,0.066622,0.60241,0.120192,0.095816,0.134816,0.018847,0.0,0.101371,0.0,0.057161,0.118601,0.138257,0.022293,0.338983,0.0,0.022712,0.0,0.008742,3.333333,0.0,0.01773,0.0,0.0,0.012056,0.039714,0.0,0.0,0.304489,0.0,0.079622,0.0,0.0,0.0,0.02186,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01717,0.112309,0.0,0.0,0.316299,0.0,0.0,0.0,NaN,0.037341,0.0,0.093067,0.040795,0.069273,0.023967,0.0,0.191038,0.144116,0.066247,0.0,0.0,0.159867,0.0,0.0,0.033793,0.015483,0.0,0.025199,0.01601,0.056153,0.0,0.094178,0.0,0.020542,0.0,0.01316,0.012888,0.058824,0.158535,0.0,0.085763,0.015721,0.080284,0.106575,0.0,0.014719,0.015936,0.096945,0.0,0.0,0.026575,0.081888,0.02592,0.0,0.102564,0.0,0.0,0.0,0.046104,0.0,0.059137,0.0,0.08106,0.0,0.094992,0.021039,0.070078,0.007425,0.0,0.096614,0.026572,0.0,0.0,0.107991,0.0,0.277668,0.120475,0.173461,0.079722,0.063977,0.021272,0.046846,0.029919,0.05112,0.013444,0.013901,0.013974
1,1,49,035,111306,2,490351113062,Block Group 2,G5030,S,653262.0,0.0,+40.6068923,-111.8144187,161.424738,"{'rings': [[[-111.821587, 40.607759], [-111.82...",669,0.0,0.0,0.0,0.0,0.0,0.005061,0.0,0.0,0.027386,0.0,0.0,0.0,0.0,0.0,0.0,0.054668,0.0,0.009082,0.013803,0.06435,0.0,0.010865,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.023714,0.0,0.0,0.021047,0.0273,0.0,0.0,0.0,0.0,0.0,0.0,0.003158,0.0,0.0,0.021777,0.005973,0.013488,0.0,0.0,0.0,0.011992,0.0,0.011011,0.0,0.005052,0.0,0.002254,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.008496,0.01471,0.0,0.0,0.0,0.006219,0.012416,0.0,0.0,0.0,0.017279,0.0,0.0,0.0,0.00655,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.003436,0.0,0.001588,0.0,0.005323,0.013376,0.338983,0.0,0.0,0.0,0.052452,0.0,0.0,0.01773,0.0,0.0,0.006028,0.0,0.0,0.0,0.152244,0.0,0.045119,0.0,0.0,0.0,0.01093,0.0,0.0,0.0,0.0,0.0,0.0,0.118624,0.0,0.0,0.01717,0.044924,0.0,0.0,0.06919,0.0,0.0,0

In [106]:
sdf_LEHDCommutePatterns_Percent_SA.sum()

FID                                                   2037947
STATEFP     4949494949494949494949494949494949494949494949...
COUNTYFP    0350350350350350350350570350350350350570570570...
TRACTCE     1113061113061114001114001114001114001114002002...
BLKGRPCE    1212356234221212321212234212111131323122431221...
                                  ...                        
z49_w                                                   100.0
z51_h                                                   100.0
z51_w                                                   100.0
z57_h                                                   100.0
z57_w                                                   100.0
Length: 231, dtype: object

In [32]:
##create map centered on Salt Lake
##map_areas = gis.map('Salt Lake')
#
##define sdf layer
##sdf_LEHDCommutePatterns_Percent_SA.spatial.plot(map_widget = map_areas,
#                            renderer_type='s',
#                            cmap = 'RdPu',  # use a red to purple color map
#                            symbol_type='simple',
#                            outline_style='s',
#                            outline_color=[0,0,0,255],
#                            line_width=1.0)
#
##display map
#map_areas

In [107]:
#export to GDB
deleteIfExists(fname_BlockGroupsWCommutePatterns_Number)
sdf_LEHDCommutePatterns.spatial.to_featureclass(fname_BlockGroupsWCommutePatterns_Number)


'c:\\Users\\cday\\Documents\\projects\\Census-LEHD-Commute-Patterns\\results\\results2018\\LEHDCommutePatterns.gdb\\BlockGroup_CommutePatterns_Number'

In [108]:
deleteIfExists(fname_BlockGroupsWCommutePatterns_Percent_SA)
sdf_LEHDCommutePatterns_Percent_SA.spatial.to_featureclass(fname_BlockGroupsWCommutePatterns_Percent_SA)

'c:\\Users\\cday\\Documents\\projects\\Census-LEHD-Commute-Patterns\\results\\results2018\\LEHDCommutePatterns.gdb\\BlockGroup_CommutePatterns_Percent_SelectedArea'

# Group by Tract

In [109]:
#input filenames
fname_Tracts_shp = os.path.join(dirInput, 'Utah_Census_Tracts_2020' + r'\CensusTracts2020.shp')

In [110]:
#read tract shapefile
sdf_Tracts = pd.DataFrame.spatial.from_featureclass(fname_Tracts_shp)
sdf_Tracts

,FID,OBJECTID,STATEFP20,COUNTYFP20,TRACTCE20,GEOID20,NAME20,NAMELSAD20,MTFCC20,FUNCSTAT20,ALAND20,AWATER20,INTPTLAT20,INTPTLON20,SHAPE_Leng,SHAPE_Area,SHAPE
0,0,1,49,049,000901,49049000901,9.01,Census Tract,G5020,S,1.686016e+06,0.0,+40.3051285,-111.6922978,7129.054778,2.902253e+06,"{""rings"": [[[-12434548.4857, 4911361.749799997..."
1,1,2,49,049,002500,49049002500,25,Census Tract,G5020,S,1.034541e+06,0.0,+40.2296149,-111.6497278,5588.845931,1.776891e+06,"{""rings"": [[[-12429589.868, 4898990.177699998]..."
2,2,3,49,049,003401,49049003401,34.01,Census Tract,G5020,S,9.085765e+06,23788.0,+40.0560227,-111.7122570,18878.825275,1.556674e+07,"{""rings"": [[[-12439397.559799999, 4872289.2984..."
3,3,4,49,049,010210,49049010210,102.10,Census Tract,G5020,S,1.581048e+07,0.0,+40.4005884,-111.7306856,26101.425584,2.729233e+07,"{""rings"": [[[-12442557.703499999, 4926512.4487..."
4,4,5,49,049,000505,49049000505,5.05,Census Tract,G5020,S,3.919712e+06,0.0,+40.3700595,-111.7070846,12562.893900,6.760193e+06,"{""rings"": [[[-12437368.7665, 4920654.352499999..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
711,711,712,49,035,111701,49035111701,1117.01,Census Tract,G5020,S,1.358256e+06,0.0,+40.6934670,-111.8826082,6277.757800,2.365082e+06,"{""rings"": [[[-12455347.0952, 4966754.840000004..."
712,712,713,49,035,111903,49035111903,1119.03,Census Tract,G5020,S,2.072253e+06,0.0,+40.6806830,-111.8627470,7592.371898,3.606962e+06,"{""rings"": [[[-12453471.3611, 4964642.738300003..."
713,713,714,49,035,111905,49035111905,1119.05,Census Tract,G5020,S,1.167718e+06,0.0,+40.6765749,-111.8744366,8615.733516,2.032283e+06,"{""rings"": [[[-12454528.1172, 4964343.751500003..."
714,714,715,49,035,113309,49035113309,1133.09,Census Tract,G5020,S,1.867612e+06,0.0,+40.6881369,-111.9693980,8501.284969,3.251486e+06,"{""rings"": [[[-12465233.158599999, 4967696.3889..."


In [111]:
#calculate tract stats
df_LEHDCommutePatterns_tract = sdf_LEHDCommutePatterns.groupby(['STATEFP','COUNTYFP','TRACTCE'])[dColCodesWCounty].apply(lambda x : x.astype(int).sum())
df_LEHDCommutePatterns_tract = df_LEHDCommutePatterns_tract.reset_index()
display(df_LEHDCommutePatterns_tract)

#caclulate tract Percent_SA stats
df_LEHDCommutePatterns_Percent_SA_tract = sdf_LEHDCommutePatterns_Percent_SA.groupby(['STATEFP','COUNTYFP','TRACTCE'])[dColCodesWCounty].apply(lambda x : x.astype(float).sum())
df_LEHDCommutePatterns_Percent_SA_tract = df_LEHDCommutePatterns_Percent_SA_tract.reset_index()
display(df_LEHDCommutePatterns_Percent_SA_tract)


#df_test = df_LEHDCommutePatterns / df_LEHDCommutePatterns_ColSum

,STATEFP,COUNTYFP,TRACTCE,AFK_h,ALA_h,ALP_h,BDL_h,BGM_h,BNT_h,BRT_h,CDF_h,CEN_h,CHA_h,CHL_h,CLF_h,CLI_h,CMT_h,COA_h,CWH_h,DAN_h,DRA_h,EAG_h,ELK_h,EMT_h,FAR_h,FCS_h,FFD_h,FRR_h,FTH_h,GLA_h,GOS_h,GRL_h,HAR_h,HDT_h,HEB_h,HER_h,HGH_h,HNF_h,HOL_h,HOO_h,HVL_h,IND_h,INT_h,KAY_h,KMS_h,KMT_h,LAY_h,LEH_h,LIN_h,MAP_h,MID_h,MLC_h,MMT_h,MRG_h,MSL_h,MUR_h,MWY_h,NOG_h,NSL_h,OGD_h,OKL_h,ORM_h,PAY_h,PGR_h,PLN_h,PRK_h,PRY_h,PVO_h,PVW_h,ROY_h,RVD_h,RVT_h,SAN_h,SAQ_h,SAR_h,SFK_h,SJC_h,SLC_h,SLM_h,SOG_h,SPV_h,SSL_h,SUN_h,SWE_h,SYR_h,TAY_h,TOO_h,UIN_h,VIN_h,WAT_h,WBG_h,WDL_h,WEB_h,WHT_h,WHV_h,WIL_h,WJC_h,WPT_h,WVC_h,WXC_h,TTL_h,AFK_w,ALA_w,ALP_w,BDL_w,BGM_w,BNT_w,BRT_w,CDF_w,CEN_w,CHA_w,CHL_w,CLF_w,CLI_w,CMT_w,COA_w,CWH_w,DAN_w,DRA_w,EAG_w,ELK_w,EMT_w,FAR_w,FCS_w,FFD_w,FRR_w,FTH_w,GLA_w,GOS_w,GRL_w,HAR_w,HDT_w,HEB_w,HER_w,HGH_w,HNF_w,HOL_w,HOO_w,HVL_w,IND_w,INT_w,KAY_w,KMS_w,KMT_w,LAY_w,LEH_w,LIN_w,MAP_w,MID_w,MLC_w,MMT_w,MRG_w,MSL_w,MUR_w,MWY_w,NOG_w,NSL_w,OGD_w,OKL_w,ORM_w,PAY_w,PGR_w,PLN_w,PRK_w,PRY_w,PVO_w,PVW_w,ROY_w,RVD_w,RVT_w,SAN_w,SAQ_w,SAR_w,SFK_w,SJC_w,SLC_w,SLM_w,SOG_w,SPV_w,SSL_w,SUN_w,SWE_w,SYR_w,TAY_w,TOO_w,UIN_w,VIN_w,WAT_w,WBG_w,WDL_w,WEB_w,WHT_w,WHV_w,WIL_w,WJC_w,WPT_w,WVC_w,WXC_w,TTL_w,z03_h,z03_w,z11_h,z11_w,z23_h,z23_w,z29_h,z29_w,z35_h,z35_w,z43_h,z43_w,z45_h,z45_w,z49_h,z49_w,z51_h,z51_w,z57_h,z57_w
0,49,001,100100,2,0,0,2,1,3,0,0,0,0,1,2,1,0,0,1,1,3,1,0,0,2,0,0,0,0,0,0,0,0,0,1,2,1,0,2,0,0,0,0,1,0,3,6,1,0,2,3,7,4,1,0,2,0,1,3,3,0,3,2,1,1,0,0,22,0,3,1,1,13,2,1,3,3,16,1,0,1,3,1,0,2,6,3,0,0,1,0,1,0,0,3,1,7,1,12,0,1554,3,1,1,0,0,1,0,0,1,0,0,2,0,0,0,1,0,1,0,0,0,2,0,0,0,1,0,0,0,0,0,2,4,0,0,3,0,0,0,0,3,0,1,1,1,1,0,4,7,0,1,1,15,0,0,2,6,1,12,2,2,0,3,0,15,1,1,2,2,6,0,0,5,0,53,0,1,1,11,0,0,1,3,0,0,0,0,0,0,0,0,1,0,9,0,14,1,1339,2,0,22,15,2,0,1,1,90,135,0,4,3,0,43,43,2,2,13,13
1,49,001,100200,1,0,0,0,2,0,0,0,0,0,1,0,0,0,0,2,0,2,1,0,0,0,0,0,0,0,0,0,1,0,0,0,3,0,0,0,0,0,0,0,0,0,1,0,3,0,0,2,2,1,0,0,1,0,0,0,0,1,2,0,0,0,0,1,2,0,0,0,0,1,1,1,3,0,2,1,0,2,1,0,0,0,3,2,0,0,0,0,0,0,0,0,0,3,0,0,0,1050,4,1,3,1,0,1,0,0,0,1,0,1,0,0,0,2,0,4,0,0,0,3,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,1,1,0,0,4,3,1,2,7,0,0,0,6,0,1,0,3,0,4,9,0,0,3,0,21,0,2,1,0,8,1,0,0,2,36,0,0,2,6,0,0,0,1,2,0,0,0,0,0,1,0,0,0,3,0,16,1,953,3,0,0,8,1,1,0,0,24,95,1,4,3,3,17,52,0,2,0,8
2,49,003,960100,0,0,0,0,56,4,0,0,4,0,0,7,4,0,0,1,0,3,1,0,0,2,0,0,6,0,0,0,1,4,0,2,3,1,0,0,6,0,0,0,3,0,4,12,2,1,0,0,0,5,2,1,0,0,18,1,43,0,1,2,1,5,0,9,0,11,8,7,1,1,0,3,0,3,14,1,3,1,1,1,2,7,4,5,0,0,3,0,0,1,1,5,2,3,3,6,1,926,5,3,0,2,144,4,0,0,5,0,0,31,4,0,0,10,0,16,0,0,0,2,0,0,7,0,0,0,0,2,0,4,1,3,0,3,0,0,0,0,8,0,2,29,3,0,0,12,12,0,2,4,17,0,5,4,96,1,18,0,1,2,1,7,14,2,8,4,2,27,1,1,10,6,74,0,7,3,7,0,1,2,4,3,0,1,3,0,0,0,0,4,1,12,0,38,1,1365,67,152,52,91,0,1,2,2,50,248,0,2,6,3,14,59,2,4,120,144
3,49,003,960200,1,0,0,0,185,2,0,0,1,0,0,11,6,0,0,0,0,3,0,0,0,1,0,0,5,1,1,0,4,0,0,3,2,1,0,0,1,0,0,0,5,0,1,16,2,0,2,0,3,0,2,1,6,0,16,1,57,0,2,4,0,5,1,13,2,7,24,7,0,1,0,0,3,1,14,0,5,2,1,2,0,8,3,2,0,0,8,0,1,0,0,4,6,5,3,4,0,1528,14,0,1,5,418,11,0,0,7,1,0,49,11,0,0,4,3,36,3,1,0,6,0,0,22,0,0,0,2,8,0,4,3,1,0,2,2,0,0,0,9,0,3,58,17,5,0,16,26,4,5,16,36,0,8,23,228,0,23,5,8,1,5,32,27,16,21,19,6,34,0,3,1,16,198,1,15,9,23,0,4,4,16,10,1,2,6,0,0,2,1,9,4,24,1,53,7,3222,204,454,57,192,0,0,2,5,44,506,1,5,6,12,21,121,3,8,140,372
4,49,003,960301,0,0,0,0,79,3,0,0,0,0,0,4,1,0,0,3,0,2,1,0,0,0,0,0,4,1,0,0,4,3,0,0,0,1,0,2,1,1,0,0,4,0,2,7,0,0,0,2,1,1,2,1,3,0,5,4,15,0,9,0,1,2,3,21,6,5,2,3,2,2,1,0,3,2,1,0,1,2,0,0,1,2,1,2,0,0,4,0,0,1,0,2,9,7,1,1,0,1212,12,0,2,2,236,4,0,0,2,0,0,47,4,0,0,4,0,16,2,0,2,6,0,0,7,0,0,0,0,4,0,6,2,6,0,1,0,0,0,0,4,0,1,28,4,4,0,3,10,2,0,5,19,0,4,8,145,0,13,2,2,3,1,20,13,4,13,13,9,20,0,3,2,8,98,0,6,4,20,1,0,1,13,4,0,0,4,0,0,0,0,8,3,12,5,54,3,2039,109,259,29,113,1,0,2,0,32,296,3,1,6,4,23,69,0,6,49,216
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

,STATEFP,COUNTYFP,TRACTCE,AFK_h,ALA_h,ALP_h,BDL_h,BGM_h,BNT_h,BRT_h,CDF_h,CEN_h,CHA_h,CHL_h,CLF_h,CLI_h,CMT_h,COA_h,CWH_h,DAN_h,DRA_h,EAG_h,ELK_h,EMT_h,FAR_h,FCS_h,FFD_h,FRR_h,FTH_h,GLA_h,GOS_h,GRL_h,HAR_h,HDT_h,HEB_h,HER_h,HGH_h,HNF_h,HOL_h,HOO_h,HVL_h,IND_h,INT_h,KAY_h,KMS_h,KMT_h,LAY_h,LEH_h,LIN_h,MAP_h,MID_h,MLC_h,MMT_h,MRG_h,MSL_h,MUR_h,MWY_h,NOG_h,NSL_h,OGD_h,OKL_h,ORM_h,PAY_h,PGR_h,PLN_h,PRK_h,PRY_h,PVO_h,PVW_h,ROY_h,RVD_h,RVT_h,SAN_h,SAQ_h,SAR_h,SFK_h,SJC_h,SLC_h,SLM_h,SOG_h,SPV_h,SSL_h,SUN_h,SWE_h,SYR_h,TAY_h,TOO_h,UIN_h,VIN_h,WAT_h,WBG_h,WDL_h,WEB_h,WHT_h,WHV_h,WIL_h,WJC_h,WPT_h,WVC_h,WXC_h,TTL_h,AFK_w,ALA_w,ALP_w,BDL_w,BGM_w,BNT_w,BRT_w,CDF_w,CEN_w,CHA_w,CHL_w,CLF_w,CLI_w,CMT_w,COA_w,CWH_w,DAN_w,DRA_w,EAG_w,ELK_w,EMT_w,FAR_w,FCS_w,FFD_w,FRR_w,FTH_w,GLA_w,GOS_w,GRL_w,HAR_w,HDT_w,HEB_w,HER_w,HGH_w,HNF_w,HOL_w,HOO_w,HVL_w,IND_w,INT_w,KAY_w,KMS_w,KMT_w,LAY_w,LEH_w,LIN_w,MAP_w,MID_w,MLC_w,MMT_w,MRG_w,MSL_w,MUR_w,MWY_w,NOG_w,NSL_w,OGD_w,OKL_w,ORM_w,PAY_w,PGR_w,PLN_w,PRK_w,PRY_w,PVO_w,PVW_w,ROY_w,RVD_w,RVT_w,SAN_w,SAQ_w,SAR_w,SFK_w,SJC_w,SLC_w,SLM_w,SOG_w,SPV_w,SSL_w,SUN_w,SWE_w,SYR_w,TAY_w,TOO_w,UIN_w,VIN_w,WAT_w,WBG_w,WDL_w,WEB_w,WHT_w,WHV_w,WIL_w,WJC_w,WPT_w,WVC_w,WXC_w,TTL_w,z03_h,z03_w,z11_h,z11_w,z23_h,z23_w,z29_h,z29_w,z35_h,z35_w,z43_h,z43_w,z45_h,z45_w,z49_h,z49_w,z51_h,z51_w,z57_h,z57_w
0,49,001,100100,0.016283,0.0,0.000000,0.030855,0.010184,0.015183,0.0,0.0,0.000000,0.000000,0.022624,0.015150,0.010014,0.0,0.000000,0.006074,0.184162,0.013623,0.006901,0.00000,0.000000,0.021730,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.012470,0.011857,0.012923,0.000000,0.014031,0.000000,0.000000,0.0,0.0,0.007878,0.00000,0.016032,0.018949,0.003464,0.000000,0.043554,0.017918,0.023603,0.027735,0.053996,0.000000,0.007995,0.000000,0.011011,0.036368,0.007578,0.000000,0.006763,0.020563,0.006290,0.032787,0.000000,0.000000,0.055582,0.000000,0.016061,0.023730,0.004248,0.027319,0.035778,0.007291,0.015192,0.009328,0.018060,0.024384,0.000000,0.006039,0.025918,0.038402,0.000000,0.016489,0.019650,0.017890,0.000000,0.0,0.022207,0.00000,0.120192,0.000000,0.000000,0.056540,0.098425,0.012027,0.021327,0.019054,0.000000,0.111785,0.013376,0.067797,0.035907,0.000000,0.000000,0.008742,0.0,0.0,0.017730,0.000000,0.000000,0.012056,0.000000,0.0,0.000000,0.004911,0.000000,0.002654,0.000000,0.000000,0.000000,0.021860,0.0,0.0,0.000000,0.224215,0.0,0.0,0.000000,0.000000,0.0,0.034341,0.089847,0.000000,0.0,0.029653,0.000000,0.000000,0.0,0.0,0.037341,0.000000,0.023267,0.003709,0.003012,0.007989,0.000000,0.021833,0.014412,0.000000,0.079745,0.054915,0.033775,0.000000,0.000000,0.013517,0.010322,0.396825,0.025199,0.032020,0.018718,0.000000,0.025685,0.000000,0.025678,0.049677,0.013160,0.025777,0.019608,0.012516,0.000000,0.000000,0.039302,0.000000,0.022238,0.000000,0.014719,0.007968,0.032315,0.000000,0.000000,0.026575,0.014451,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.029568,0.000000,0.025157,0.000000,0.020460,0.021039,0.094878,0.014850,0.000000,0.015182,0.013744,0.035778,0.000000,0.053996,0.079745,0.016660,0.019762,0.000000,0.028990,0.013709,0.000000,0.017217,0.018379,0.017040,0.026889,0.012047,0.012975
1,49,001,100200,0.008141,0.0,0.000000,0.000000,0.020369,0.000000,0.0,0.0,0.000000,0.000000,0.022624,0.000000,0.000000,0.0,0.000000,0.012148,0.000000,0.009082,0.006901,0.00000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.019554,0.000000,0.0,0.000000,0.017785,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.005344,0.000000,0.010391,0.000000,0.000000,0.011945,0.006744,0.006934,0.000000,0.000000,0.003997,0.000000,0.000000,0.000000,0.000000,0.176678,0.004508,0.000000,0.000000,0.000000,0.000000,0.037979,0.005053,0.000000,0.000000,0.000000,0.000000,0.002101,0.017889,0.007291,0.015192,0.000000,0.002257,0.024384,0.000000,0.012079,0.008639,0.000000,0.000000,0.000000,0.009825,0.011927,0.000000,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.005154,0.000000,0.000000,0.000000,

In [112]:
df_LEHDCommutePatterns_Percent_SA_tract.sum()

STATEFP     4949494949494949494949494949494949494949494949...
COUNTYFP    0010010030030030030030030030030030030030030050...
TRACTCE     1001001002009601009602009603019603029604009605...
AFK_h                                                   100.0
ALA_h                                                   100.0
                                  ...                        
z49_w                                                   100.0
z51_h                                                   100.0
z51_w                                                   100.0
z57_h                                                   100.0
z57_w                                                   100.0
Length: 219, dtype: object

In [113]:
sdf_LEHDCommutePatterns_tract = pd.DataFrame.merge(sdf_Tracts, df_LEHDCommutePatterns_tract, left_on=['STATEFP20','COUNTYFP20','TRACTCE20'], right_on=['STATEFP','COUNTYFP',"TRACTCE"])
display(sdf_LEHDCommutePatterns_tract)

sdf_LEHDCommutePatterns_Percent_SA_tract = pd.DataFrame.merge(sdf_Tracts, df_LEHDCommutePatterns_Percent_SA_tract, left_on=['STATEFP20','COUNTYFP20','TRACTCE20'], right_on=['STATEFP','COUNTYFP',"TRACTCE"])
display(sdf_LEHDCommutePatterns_Percent_SA_tract)

,FID,OBJECTID,STATEFP20,COUNTYFP20,TRACTCE20,GEOID20,NAME20,NAMELSAD20,MTFCC20,FUNCSTAT20,ALAND20,AWATER20,INTPTLAT20,INTPTLON20,SHAPE_Leng,SHAPE_Area,SHAPE,STATEFP,COUNTYFP,TRACTCE,AFK_h,ALA_h,ALP_h,BDL_h,BGM_h,BNT_h,BRT_h,CDF_h,CEN_h,CHA_h,CHL_h,CLF_h,CLI_h,CMT_h,COA_h,CWH_h,DAN_h,DRA_h,EAG_h,ELK_h,EMT_h,FAR_h,FCS_h,FFD_h,FRR_h,FTH_h,GLA_h,GOS_h,GRL_h,HAR_h,HDT_h,HEB_h,HER_h,HGH_h,HNF_h,HOL_h,HOO_h,HVL_h,IND_h,INT_h,KAY_h,KMS_h,KMT_h,LAY_h,LEH_h,LIN_h,MAP_h,MID_h,MLC_h,MMT_h,MRG_h,MSL_h,MUR_h,MWY_h,NOG_h,NSL_h,OGD_h,OKL_h,ORM_h,PAY_h,PGR_h,PLN_h,PRK_h,PRY_h,PVO_h,PVW_h,ROY_h,RVD_h,RVT_h,SAN_h,SAQ_h,SAR_h,SFK_h,SJC_h,SLC_h,SLM_h,SOG_h,SPV_h,SSL_h,SUN_h,SWE_h,SYR_h,TAY_h,TOO_h,UIN_h,VIN_h,WAT_h,WBG_h,WDL_h,WEB_h,WHT_h,WHV_h,WIL_h,WJC_h,WPT_h,WVC_h,WXC_h,TTL_h,AFK_w,ALA_w,ALP_w,BDL_w,BGM_w,BNT_w,BRT_w,CDF_w,CEN_w,CHA_w,CHL_w,CLF_w,CLI_w,CMT_w,COA_w,CWH_w,DAN_w,DRA_w,EAG_w,ELK_w,EMT_w,FAR_w,FCS_w,FFD_w,FRR_w,FTH_w,GLA_w,GOS_w,GRL_w,HAR_w,HDT_w,HEB_w,HER_w,HGH_w,HNF_w,HOL_w,HOO_w,HVL_w,IND_w,INT_w,KAY_w,KMS_w,KMT_w,LAY_w,LEH_w,LIN_w,MAP_w,MID_w,MLC_w,MMT_w,MRG_w,MSL_w,MUR_w,MWY_w,NOG_w,NSL_w,OGD_w,OKL_w,ORM_w,PAY_w,PGR_w,PLN_w,PRK_w,PRY_w,PVO_w,PVW_w,ROY_w,RVD_w,RVT_w,SAN_w,SAQ_w,SAR_w,SFK_w,SJC_w,SLC_w,SLM_w,SOG_w,SPV_w,SSL_w,SUN_w,SWE_w,SYR_w,TAY_w,TOO_w,UIN_w,VIN_w,WAT_w,WBG_w,WDL_w,WEB_w,WHT_w,WHV_w,WIL_w,WJC_w,WPT_w,WVC_w,WXC_w,TTL_w,z03_h,z03_w,z11_h,z11_w,z23_h,z23_w,z29_h,z29_w,z35_h,z35_w,z43_h,z43_w,z45_h,z45_w,z49_h,z49_w,z51_h,z51_w,z57_h,z57_w
0,0,1,49,049,000901,49049000901,9.01,Census Tract,G5020,S,1.686016e+06,0.0,+40.3051285,-111.6922978,7129.054778,2.902253e+06,"{'rings': [[[-12434548.4857, 4911361.749799997...",49,049,000901,65,0,21,11,14,16,0,1,2,3,18,9,6,2,1,10,0,26,43,4,0,2,0,1,2,3,2,0,15,2,1,14,18,24,0,2,0,0,0,0,8,0,28,20,76,31,16,10,19,33,1,0,25,7,4,5,23,0,537,35,107,1,3,2,305,3,11,3,19,36,23,40,84,36,93,9,7,74,9,2,1,7,34,61,0,1,3,0,5,2,1,3,0,65,1,129,4,2864,147,1,8,9,3,6,0,0,3,0,8,6,0,0,0,22,1,76,12,3,0,4,0,0,0,0,0,0,0,0,0,9,8,15,0,11,0,0,0,0,3,0,1,16,172,123,6,15,34,0,0,1,48,4,1,8,20,0,602,9,78,0,5,0,412,0,4,2,12,59,2,14,43,43,146,5,3,70,31,0,0,3,12,3,0,10,1,0,0,0,0,3,0,23,0,45,6,2543,16,3,88,55,23,2,1,0,606,596,4,5,76,3,1499,1737,25,14,62,35
1,1,2,49,049,002500,49049002500,25,Census Tract,G5020,S,1.034541e+06,0.0,+40.2296149,-111.6497278,5588.845931,1.776891e+06,"{'rings': [[[-12429589.868, 4898990.177699998]...",49,049,002500,30,0,2,3,0,1,0,0,0,0,5,0,1,0,0,1,1,6,16,9,0,1,0,0,0,0,2,1,1,0,0,5,3,15,0,2,1,0,1,0,1,1,7,4,44,12,26,2,4,0,1,0,6,1,1,2,4,0,118,46,29,0,1,0,222,0,1,0,6,2,24,8,115,6,7,14,0,87,1,0,0,1,4,4,0,1,1,0,5,0,0,0,0,10,0,5,0,1025,63,0,5,3,4,5,0,0,0,0,3,0,0,0,0,2,2,17,4,2,0,7,0,2,1,0,0,0,0,0,0,3,3,6,0,1,0,0,0,0,2,1,0,10,97,51,6,9,8,0,0,1,13,1,0,6,12,0,275,14,57,0,1,0,540,1,3,1,3,18,1,5,62,10,84,2,1,72,13,1,0,2,4,5,0,0,0,0,1,1,0,1,0,4,1,25,2,1631,0,4,11,37,24,1,1,0,75,217,2,2,5,5,807,1267,8,6,8,21
2,2,3,49,049,003401,49049003401,34.01,Census Tract,G5020,S,9.085765e+06,23788.0,+40.0560227,-111.7122570,18878.825275,1.556674e+07,"{'rings': [[[-12439397.559799999, 4872289.2984...",49,049,003401,9,0,1,0,2,3,0,0,1,0,0,2,0,0,1,0,2,2,15,30,0,1,0,0,0,0,17,11,0,0,0,2,2,0,0,0,1,0,0,0,1,0,0,3,16,3,20,3,0,0,1,0,2,0,1,0,8,0,26,257,6,0,1,1,60,0,3,1,2,6,97,7,176,6,4,65,2,84,0,0,0,3,5,6,0,0,0,1,9,0,0,0,0,8,0,4,2,1174,52,0,1,3,1,7,0,2,3,0,0,5,0,0,1,5,1,23,3,1,0,3,0,0,1,0,7,1,1,0,0,11,4,4,0,1,0,0,0,0,2,0,2,14,69,43,15,11,25,3,0,1,25,1,0,7,12,0,164,285,21,0,5,1,232,0,2,2,7,33,15,6,175,15,133,31,4,133,23,0,1,4,16,5,0,8,0,0,0,0,0,0,0,28,1,48,3,1923,3,2,16,50,97,15,1,0,44,405,2,6,6,6,812,1253,5,13,16,22
3,3,4,49,049,010210,49049010210,102.10,Census Tract,G5020,S,1.581048e+07,0.0,+40.4005884,-111.7306856,26101.425584,2.729233e+07,"{'rings': [[[-12442557.703499999, 4926512.4487...",49,049,010210,26,0,9,1,1,0,0,0,1,0,64,2,2,0,0,1,0,6,10,0,0,1,0,0,0,0,1,1,0,0,0,1,0,21,0,3,0,0,0,0,0,0,1,2,31,3,2,2,8,0,0,0,7,0,1,1,5,0,35,2,68,1,0,0,25,0,1,0,2,17,0,6,11,5,11,2,1,7,2,0,0,0,7,1,0,0,0,0,0,0,2,0,0,7,0,6,0,451,239,1,35,10,2,7,0,0

,FID,OBJECTID,STATEFP20,COUNTYFP20,TRACTCE20,GEOID20,NAME20,NAMELSAD20,MTFCC20,FUNCSTAT20,ALAND20,AWATER20,INTPTLAT20,INTPTLON20,SHAPE_Leng,SHAPE_Area,SHAPE,STATEFP,COUNTYFP,TRACTCE,AFK_h,ALA_h,ALP_h,BDL_h,BGM_h,BNT_h,BRT_h,CDF_h,CEN_h,CHA_h,CHL_h,CLF_h,CLI_h,CMT_h,COA_h,CWH_h,DAN_h,DRA_h,EAG_h,ELK_h,EMT_h,FAR_h,FCS_h,FFD_h,FRR_h,FTH_h,GLA_h,GOS_h,GRL_h,HAR_h,HDT_h,HEB_h,HER_h,HGH_h,HNF_h,HOL_h,HOO_h,HVL_h,IND_h,INT_h,KAY_h,KMS_h,KMT_h,LAY_h,LEH_h,LIN_h,MAP_h,MID_h,MLC_h,MMT_h,MRG_h,MSL_h,MUR_h,MWY_h,NOG_h,NSL_h,OGD_h,OKL_h,ORM_h,PAY_h,PGR_h,PLN_h,PRK_h,PRY_h,PVO_h,PVW_h,ROY_h,RVD_h,RVT_h,SAN_h,SAQ_h,SAR_h,SFK_h,SJC_h,SLC_h,SLM_h,SOG_h,SPV_h,SSL_h,SUN_h,SWE_h,SYR_h,TAY_h,TOO_h,UIN_h,VIN_h,WAT_h,WBG_h,WDL_h,WEB_h,WHT_h,WHV_h,WIL_h,WJC_h,WPT_h,WVC_h,WXC_h,TTL_h,AFK_w,ALA_w,ALP_w,BDL_w,BGM_w,BNT_w,BRT_w,CDF_w,CEN_w,CHA_w,CHL_w,CLF_w,CLI_w,CMT_w,COA_w,CWH_w,DAN_w,DRA_w,EAG_w,ELK_w,EMT_w,FAR_w,FCS_w,FFD_w,FRR_w,FTH_w,GLA_w,GOS_w,GRL_w,HAR_w,HDT_w,HEB_w,HER_w,HGH_w,HNF_w,HOL_w,HOO_w,HVL_w,IND_w,INT_w,KAY_w,KMS_w,KMT_w,LAY_w,LEH_w,LIN_w,MAP_w,MID_w,MLC_w,MMT_w,MRG_w,MSL_w,MUR_w,MWY_w,NOG_w,NSL_w,OGD_w,OKL_w,ORM_w,PAY_w,PGR_w,PLN_w,PRK_w,PRY_w,PVO_w,PVW_w,ROY_w,RVD_w,RVT_w,SAN_w,SAQ_w,SAR_w,SFK_w,SJC_w,SLC_w,SLM_w,SOG_w,SPV_w,SSL_w,SUN_w,SWE_w,SYR_w,TAY_w,TOO_w,UIN_w,VIN_w,WAT_w,WBG_w,WDL_w,WEB_w,WHT_w,WHV_w,WIL_w,WJC_w,WPT_w,WVC_w,WXC_w,TTL_w,z03_h,z03_w,z11_h,z11_w,z23_h,z23_w,z29_h,z29_w,z35_h,z35_w,z43_h,z43_w,z45_h,z45_w,z49_h,z49_w,z51_h,z51_w,z57_h,z57_w
0,0,1,49,049,000901,49049000901,9.01,Census Tract,G5020,S,1.686016e+06,0.0,+40.3051285,-111.6922978,7129.054778,2.902253e+06,"{'rings': [[[-12434548.4857, 4911361.749799997...",49,049,000901,0.529187,0.000000,0.473080,0.169701,0.142581,0.080976,0.000000,0.310559,0.027386,1.190476,0.407240,0.068177,0.060084,0.432900,0.131752,0.060742,0.000000,0.118069,0.296756,0.257400,0.000000,0.021730,0.000000,1.265823,0.065833,0.138440,0.248447,0.000000,0.293312,0.066028,0.295858,0.174585,0.106711,0.310158,0.00000,0.014031,0.0000,0.0,0.000000,0.0,0.063027,0.00000,0.149637,0.063163,0.263231,0.612285,0.348432,0.059726,0.064066,0.228817,0.053996,0.000000,0.099932,0.311111,0.044043,0.060613,0.058101,0.000000,1.210495,0.359860,0.672998,0.032787,0.077983,0.075959,0.770572,0.073350,0.058890,0.071191,0.080710,0.075653,0.411449,0.291651,0.425381,0.111933,0.104972,0.219459,0.089217,0.446914,0.077754,0.076805,0.032595,0.057713,0.111348,0.363766,0.000000,1.111111,0.066622,0.00000,0.600962,0.063877,0.033704,0.056540,0.000000,0.111680,0.021327,0.204827,0.079067,0.206019,0.655401,0.067797,0.287253,0.204406,0.033419,0.052452,0.000000,0.000000,0.053191,0.000000,0.946746,0.036169,0.000000,0.0,0.000000,0.108044,0.546448,0.201709,0.583374,0.967742,0.0,0.043721,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.154533,0.179695,0.495376,0.0,0.108728,0.000000,0.0,0.0,0.0,0.037341,0.000000,0.023267,0.059338,0.518041,0.982664,0.312337,0.081873,0.069999,0.000000,0.000000,0.054915,0.108079,0.342466,0.035817,0.054069,0.034407,0.000000,1.264175,0.144092,0.729995,0.000000,0.042808,0.000000,0.705286,0.000000,0.052639,0.025777,0.117647,0.123073,0.187266,0.400229,0.337997,0.143842,0.061260,0.312110,0.044157,0.557769,0.091069,0.000000,0.000000,0.079724,0.057803,0.038880,0.0,1.025641,0.029842,0.000000,0.000000,0.000000,0.000000,0.088705,0.0,0.064289,0.000000,0.065764,0.126236,0.180191,0.118800,0.029803,0.060729,0.050394,0.411449,0.187266,0.053996,0.000000,0.112178,0.087245,0.057820,0.036237,0.347302,0.031908,0.600195,0.742416,0.213002,0.188223,0.057456,0.034934
1,1,2,49,049,002500,49049002500,25,Census Tract,G5020,S,1.034541e+06,0.0,+40.2296149,-111.6497278,5588.845931,1.776891e+06,"{'rings': [[[-12429589.868, 4898990.177699998]...",49,049,002500,0.244240,0.000000,0.045055,0.046282,0.000000,0.005061,0.000000,0.000000,0.000000,0.000000,0.113122,0.000000,0.010014,0.000000,0.000000,0.006074,0.184162,0.027247,0.110421,0.579151,0.000000,0.010865,0.000000,0.000000,0.000000,0.000000,0.248447,0.206612,0.0195

In [114]:
sdf_LEHDCommutePatterns_tract.sum()


FID                                                      255970
OBJECTID                                                 256686
STATEFP20     4949494949494949494949494949494949494949494949...
COUNTYFP20    0490490490490490490490510130490490490490490490...
TRACTCE20     0009010025000034010102100005050005070102099601...
                                    ...                        
z49_w                                                    233966
z51_h                                                     11737
z51_w                                                      7438
z57_h                                                    107909
z57_w                                                    100189
Length: 235, dtype: object

In [115]:
sdf_LEHDCommutePatterns_Percent_SA_tract.sum()

FID                                                      255970
OBJECTID                                                 256686
STATEFP20     4949494949494949494949494949494949494949494949...
COUNTYFP20    0490490490490490490490510130490490490490490490...
TRACTCE20     0009010025000034010102100005050005070102099601...
                                    ...                        
z49_w                                                     100.0
z51_h                                                     100.0
z51_w                                                     100.0
z57_h                                                     100.0
z57_w                                                     100.0
Length: 235, dtype: object

In [116]:
sdf_LEHDCommutePatterns_tract.spatial.to_featureclass(fname_TractsWCommutePatterns_Number)

'c:\\Users\\cday\\Documents\\projects\\Census-LEHD-Commute-Patterns\\results\\results2018\\LEHDCommutePatterns.gdb\\Tract_CommutePatterns_Number'

In [117]:
sdf_LEHDCommutePatterns_Percent_SA_tract.spatial.to_featureclass(fname_TractsWCommutePatterns_Percent_SA)

'c:\\Users\\cday\\Documents\\projects\\Census-LEHD-Commute-Patterns\\results\\results2018\\LEHDCommutePatterns.gdb\\Tract_CommutePatterns_Percent_SelectedArea'

# Group by Small District

# Group Rows by City

In [118]:

#verify that shape exists and sdf was created correctly and get spatial reference
#print(sdf_CityTownship.iloc[0].SHAPE.spatialReference)
#print(sdf_BlockGroups.iloc[0].SHAPE.spatialReference)
#
##change projection to 26912 to match TAZ Centroids
#sdf_CityTownship.spatial.project(26912)
#sdf_BlockGroups.spatial.project(26912)
#
#print(sdf_CityTownship.iloc[0].SHAPE.spatialReference)
#print(sdf_BlockGroups.iloc[0].SHAPE.spatialReference)

print ("Calculating blockgroup acreage...")

# Execute DeleteField
arcpy.DeleteField_management(fname_BlockGroups_shp, ['BGAcres'])

arcpy.AddField_management(fname_BlockGroups_shp, "BGAcres" , "DOUBLE")
arcpy.CalculateField_management(fname_BlockGroups_shp,
                                "BGAcres",
                                "!shape.area@acres!",
                                "PYTHON_9.3")
print ("Done calculating blockgroup acreage.")


print ("Intersecting blockgroup with city/township/county...")
#intersection parcels with analysis area types
deleteIfExists(fname_CityTownshipCounty_BlockGroups_shp)
arcpy.analysis.Intersect([fname_CityTownshipCountyWCodes_shp,fname_BlockGroups_shp], fname_CityTownshipCounty_BlockGroups_shp)


print ("Calculating piece area...")
arcpy.AddField_management(fname_CityTownshipCounty_BlockGroups_shp, "PieceAcres" , "DOUBLE")
arcpy.CalculateField_management(fname_CityTownshipCounty_BlockGroups_shp,
                                "PieceAcres",
                                "!shape.area@acres!",
                                "PYTHON_9.3")
print ("Calculating Percent_SA of blockgroup...")
arcpy.AddField_management(fname_CityTownshipCounty_BlockGroups_shp, "PortionBG" , "DOUBLE")
arcpy.CalculateField_management(fname_CityTownshipCounty_BlockGroups_shp,
                                "PortionBG",
                                "!PieceAcres!/!BGAcres!",
                                "PYTHON_9.3")


print ("Done")

Calculating blockgroup acreage...
Done calculating blockgroup acreage.
Intersecting blockgroup with city/township/county...
Calculating piece area...
Calculating Percent_SA of blockgroup...
Done


In [119]:
#read newly created shapefile
sdf_CityTownshipCounty_BlockGroups = pd.DataFrame.spatial.from_featureclass(fname_CityTownshipCounty_BlockGroups_shp)


In [120]:
sdf_LEHDCommutePatterns_joinCity = pd.DataFrame.merge(sdf_CityTownshipCounty_BlockGroups,sdf_LEHDCommutePatterns,on=['GEOID','STATEFP','COUNTYFP','TRACTCE','BLKGRPCE','NAMELSAD','MTFCC', 'FUNCSTAT','ALAND','AWATER','INTPTLAT','INTPTLON','BGAcres'])
sdf_LEHDCommutePatterns_joinCity

,FID_x,FID_Munici,Id,fid_muni_1,code3,first_name,FID_tl_202,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,GEOID,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,BGAcres,PieceAcres,PortionBG,SHAPE_x,FID_y,SHAPE_y,fakeid,AFK_h,ALA_h,ALP_h,BDL_h,BGM_h,BNT_h,BRT_h,CDF_h,CEN_h,CHA_h,CHL_h,CLF_h,CLI_h,CMT_h,COA_h,CWH_h,DAN_h,DRA_h,EAG_h,ELK_h,EMT_h,FAR_h,FCS_h,FFD_h,FRR_h,FTH_h,GLA_h,GOS_h,GRL_h,HAR_h,HDT_h,HEB_h,HER_h,HGH_h,HNF_h,HOL_h,HOO_h,HVL_h,IND_h,INT_h,KAY_h,KMS_h,KMT_h,LAY_h,LEH_h,LIN_h,MAP_h,MID_h,MLC_h,MMT_h,MRG_h,MSL_h,MUR_h,MWY_h,NOG_h,NSL_h,OGD_h,OKL_h,ORM_h,PAY_h,PGR_h,PLN_h,PRK_h,PRY_h,PVO_h,PVW_h,ROY_h,RVD_h,RVT_h,SAN_h,SAQ_h,SAR_h,SFK_h,SJC_h,SLC_h,SLM_h,SOG_h,SPV_h,SSL_h,SUN_h,SWE_h,SYR_h,TAY_h,TOO_h,UIN_h,VIN_h,WAT_h,WBG_h,WDL_h,WEB_h,WHT_h,WHV_h,WIL_h,WJC_h,WPT_h,WVC_h,WXC_h,TTL_h,AFK_w,ALA_w,ALP_w,BDL_w,BGM_w,BNT_w,BRT_w,CDF_w,CEN_w,CHA_w,CHL_w,CLF_w,CLI_w,CMT_w,COA_w,CWH_w,DAN_w,DRA_w,EAG_w,ELK_w,EMT_w,FAR_w,FCS_w,FFD_w,FRR_w,FTH_w,GLA_w,GOS_w,GRL_w,HAR_w,HDT_w,HEB_w,HER_w,HGH_w,HNF_w,HOL_w,HOO_w,HVL_w,IND_w,INT_w,KAY_w,KMS_w,KMT_w,LAY_w,LEH_w,LIN_w,MAP_w,MID_w,MLC_w,MMT_w,MRG_w,MSL_w,MUR_w,MWY_w,NOG_w,NSL_w,OGD_w,OKL_w,ORM_w,PAY_w,PGR_w,PLN_w,PRK_w,PRY_w,PVO_w,PVW_w,ROY_w,RVD_w,RVT_w,SAN_w,SAQ_w,SAR_w,SFK_w,SJC_w,SLC_w,SLM_w,SOG_w,SPV_w,SSL_w,SUN_w,SWE_w,SYR_w,TAY_w,TOO_w,UIN_w,VIN_w,WAT_w,WBG_w,WDL_w,WEB_w,WHT_w,WHV_w,WIL_w,WJC_w,WPT_w,WVC_w,WXC_w,TTL_w,z03_h,z03_w,z11_h,z11_w,z23_h,z23_w,z29_h,z29_w,z35_h,z35_w,z43_h,z43_w,z45_h,z45_w,z49_h,z49_w,z51_h,z51_w,z57_h,z57_w
0,0,0,0,-1,z05,Cache County Unincorporated,95,49,057,210101,1,490572101011,Block Group 1,G5030,S,86898873.0,19742.0,+41.3700825,-111.9334158,21478.059645,2.821747,0.000131,"{'rings': [[[-111.88525694249688, 41.426031954...",95,"{'rings': [[[-111.975799, 41.384468999999996],...",1918,0,0,0,0,0,0,0,0,0,0,0,4,2,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,2,2,0,0,0,0,0,7,0,0,0,0,0,0,0,0,2,0,2,0,18,0,0,0,0,2,0,0,1,3,2,1,1,0,0,0,0,2,0,0,2,0,0,0,1,1,1,0,0,0,0,0,0,0,0,2,0,1,0,2,1,84,3,0,0,0,6,2,0,0,2,0,0,7,0,0,0,1,0,5,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,2,1,0,0,4,0,2,8,1,0,0,1,2,0,1,4,3,0,12,2,68,0,5,0,0,0,0,1,1,8,6,6,0,4,0,0,1,4,42,0,9,0,5,0,0,0,3,1,0,0,8,0,0,0,0,2,0,2,0,6,1,347,0,7,17,27,0,0,0,1,11,80,0,0,0,1,1,11,0,1,37,128
1,238,5,0,-1,z03,Box Elder County Unincorporated,95,49,057,210101,1,490572101011,Block Group 1,G5030,S,86898873.0,19742.0,+41.3700825,-111.9334158,21478.059645,0.887017,0.000041,"{'rings': [[[-111.95909781387314, 41.433683592...",95,"{'rings': [[[-111.975799, 41.384468999999996],...",1918,0,0,0,0,0,0,0,0,0,0,0,4,2,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,2,2,0,0,0,0,0,7,0,0,0,0,0,0,0,0,2,0,2,0,18,0,0,0,0,2,0,0,1,3,2,1,1,0,0,0,0,2,0,0,2,0,0,0,1,1,1,0,0,0,0,0,0,0,0,2,0,1,0,2,1,84,3,0,0,0,6,2,0,0,2,0,0,7,0,0,0,1,0,5,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,2,1,0,0,4,0,2,8,1,0,0,1,2,0,1,4,3,0,12,2,68,0,5,0,0,0,0,1,1,8,6,6,0,4,0,0,1,4,42,0,9,0,5,0,0,0,3,1,0,0,8,0,0,0,0,2,0,2,0,6,1,347,0,7,17,27,0,0,0,1,11,80,0,0,0,1,1,11,0,1,37,128
2,288,6,0,-1,z57,Weber County Unincorporated,95,49,057,210101,1,490572101011,Block Group 1,G5030,S,86898873.0,19742.0,+41.3700825,-111.9334158,21478.059645,20832.493173,0.969943,"{'rings': [[[-111.95704997693178, 41.433486063...",95,"{'rings': [[[-111.975799, 41.384468999999996],...",1918,0,0,0,0,0,0,0,0,0,0,0,4,2,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,2,2,0,0,0,0,0,7,0,0,0,0,0,0,0,0,2,0,2,0,18,0,0,0,0,2,0,0,1,3,2,1,1,0,0,0,0,2,0,0,2,0,0,0,1,1,1,0,0,0,0,0,0,0,0,2,0,1,0,2,1,84,3,0,0,0,6,2,0,0,2,0,0,7,0,0,0,1,0,5,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,2,1,0,0,4,0,2,8,1,0,0,1,2,0,1,4,3,0,12,2,68,0,5,0,0,0,0,1,1,8,6,6,0,4,0,0,1,4,42,0,9,0,5,0,0,0,3,1,0,0,8,0,0,0,0,2,0,2,0,6,1,347,0,7,17,27,0,0,0,1,11,80,0,0,0,1,1,11,0,1,37,128
3,2156,90,0,55,NOG,North Ogden,95,49,057,210101,1,490572101011,Block Group 1,G5030,S,86898873.0,19742.0,+41.3700825,-111.9334158,21478.059645,641.857708,0.029884,"{'rings': [[[-111.93408914365247, 41.317315160...",95,"{'rings': [[[-111.975799, 41.384468999999996],...",1918,0,0,0,0,0,0,

In [122]:
my_list = list(sdf_LEHDCommutePatterns_joinCity)
my_list

['FID_x',
 'FID_Munici',
 'Id',
 'fid_muni_1',
 'code3',
 'first_name',
 'FID_tl_202',
 'STATEFP',
 'COUNTYFP',
 'TRACTCE',
 'BLKGRPCE',
 'GEOID',
 'NAMELSAD',
 'MTFCC',
 'FUNCSTAT',
 'ALAND',
 'AWATER',
 'INTPTLAT',
 'INTPTLON',
 'BGAcres',
 'PieceAcres',
 'PortionBG',
 'SHAPE_x',
 'FID_y',
 'SHAPE_y',
 'fakeid',
 'AFK_h',
 'ALA_h',
 'ALP_h',
 'BDL_h',
 'BGM_h',
 'BNT_h',
 'BRT_h',
 'CDF_h',
 'CEN_h',
 'CHA_h',
 'CHL_h',
 'CLF_h',
 'CLI_h',
 'CMT_h',
 'COA_h',
 'CWH_h',
 'DAN_h',
 'DRA_h',
 'EAG_h',
 'ELK_h',
 'EMT_h',
 'FAR_h',
 'FCS_h',
 'FFD_h',
 'FRR_h',
 'FTH_h',
 'GLA_h',
 'GOS_h',
 'GRL_h',
 'HAR_h',
 'HDT_h',
 'HEB_h',
 'HER_h',
 'HGH_h',
 'HNF_h',
 'HOL_h',
 'HOO_h',
 'HVL_h',
 'IND_h',
 'INT_h',
 'KAY_h',
 'KMS_h',
 'KMT_h',
 'LAY_h',
 'LEH_h',
 'LIN_h',
 'MAP_h',
 'MID_h',
 'MLC_h',
 'MMT_h',
 'MRG_h',
 'MSL_h',
 'MUR_h',
 'MWY_h',
 'NOG_h',
 'NSL_h',
 'OGD_h',
 'OKL_h',
 'ORM_h',
 'PAY_h',
 'PGR_h',
 'PLN_h',
 'PRK_h',
 'PRY_h',
 'PVO_h',
 'PVW_h',
 'ROY_h',
 'RVD_h',
 'RV

In [49]:
#for i in dColCodes:
#    display(sdf_LEHDCommutePatterns_joinCity[[i]])

In [123]:
df_LEHDCommutePatterns_CityCalcs = sdf_LEHDCommutePatterns_joinCity.copy()

for i in dColCodesWCounty:
    display(i)
    
    df_LEHDCommutePatterns_CityCalcs[i] = df_LEHDCommutePatterns_CityCalcs[i] * df_LEHDCommutePatterns_CityCalcs['PortionBG']


'AFK_h'

'ALA_h'

'ALP_h'

'BDL_h'

'BGM_h'

...

'z49_w'

'z51_h'

'z51_w'

'z57_h'

'z57_w'

In [127]:
df_LEHDCommutePatterns_CityCalcs = df_LEHDCommutePatterns_CityCalcs.rename(columns={"first_name":"FIRST_NAME", "code3":"CODE3"})
df_LEHDCommutePatterns_CityCalcs

,FID_x,FID_Munici,Id,fid_muni_1,CODE3,FIRST_NAME,FID_tl_202,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,GEOID,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,BGAcres,PieceAcres,PortionBG,SHAPE_x,FID_y,SHAPE_y,fakeid,AFK_h,ALA_h,ALP_h,BDL_h,BGM_h,BNT_h,BRT_h,CDF_h,CEN_h,CHA_h,CHL_h,CLF_h,CLI_h,CMT_h,COA_h,CWH_h,DAN_h,DRA_h,EAG_h,ELK_h,EMT_h,FAR_h,FCS_h,FFD_h,FRR_h,FTH_h,GLA_h,GOS_h,GRL_h,HAR_h,HDT_h,HEB_h,HER_h,HGH_h,HNF_h,HOL_h,HOO_h,HVL_h,IND_h,INT_h,KAY_h,KMS_h,KMT_h,LAY_h,LEH_h,LIN_h,MAP_h,MID_h,MLC_h,MMT_h,MRG_h,MSL_h,MUR_h,MWY_h,NOG_h,NSL_h,OGD_h,OKL_h,ORM_h,PAY_h,PGR_h,PLN_h,PRK_h,PRY_h,PVO_h,PVW_h,ROY_h,RVD_h,RVT_h,SAN_h,SAQ_h,SAR_h,SFK_h,SJC_h,SLC_h,SLM_h,SOG_h,SPV_h,SSL_h,SUN_h,SWE_h,SYR_h,TAY_h,TOO_h,UIN_h,VIN_h,WAT_h,WBG_h,WDL_h,WEB_h,WHT_h,WHV_h,WIL_h,WJC_h,WPT_h,WVC_h,WXC_h,TTL_h,AFK_w,ALA_w,ALP_w,BDL_w,BGM_w,BNT_w,BRT_w,CDF_w,CEN_w,CHA_w,CHL_w,CLF_w,CLI_w,CMT_w,COA_w,CWH_w,DAN_w,DRA_w,EAG_w,ELK_w,EMT_w,FAR_w,FCS_w,FFD_w,FRR_w,FTH_w,GLA_w,GOS_w,GRL_w,HAR_w,HDT_w,HEB_w,HER_w,HGH_w,HNF_w,HOL_w,HOO_w,HVL_w,IND_w,INT_w,KAY_w,KMS_w,KMT_w,LAY_w,LEH_w,LIN_w,MAP_w,MID_w,MLC_w,MMT_w,MRG_w,MSL_w,MUR_w,MWY_w,NOG_w,NSL_w,OGD_w,OKL_w,ORM_w,PAY_w,PGR_w,PLN_w,PRK_w,PRY_w,PVO_w,PVW_w,ROY_w,RVD_w,RVT_w,SAN_w,SAQ_w,SAR_w,SFK_w,SJC_w,SLC_w,SLM_w,SOG_w,SPV_w,SSL_w,SUN_w,SWE_w,SYR_w,TAY_w,TOO_w,UIN_w,VIN_w,WAT_w,WBG_w,WDL_w,WEB_w,WHT_w,WHV_w,WIL_w,WJC_w,WPT_w,WVC_w,WXC_w,TTL_w,z03_h,z03_w,z11_h,z11_w,z23_h,z23_w,z29_h,z29_w,z35_h,z35_w,z43_h,z43_w,z45_h,z45_w,z49_h,z49_w,z51_h,z51_w,z57_h,z57_w
0,0,0,0,-1,z05,Cache County Unincorporated,95,49,057,210101,1,490572101011,Block Group 1,G5030,S,86898873.0,19742.0,+41.3700825,-111.9334158,21478.059645,2.821747,0.000131,"{'rings': [[[-111.88525694249688, 41.426031954...",95,"{'rings': [[[-111.975799, 41.384468999999996],...",1918,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000526,0.000263,0.0,0.0,0.0,0.0,0.000131,0.0,0.0,0.0,0.000131,0.0,0.0,0.000131,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000131,0.0,0.0,0.0,0.000263,0.000263,0.0,0.0,0.0,0.0,0.0,0.000920,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000263,0.0,0.000263,0.0,0.002365,0.0,0.0,0.0,0.0,0.000263,0.0,0.0,0.000131,0.000394,0.000263,0.000131,0.000131,0.0,0.0,0.0,0.0,0.000263,0.0,0.0,0.000263,0.0,0.0,0.0,0.000131,0.000131,0.000131,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000263,0.0,0.000131,0.0,0.000263,0.000131,0.011036,0.000394,0.0,0.0,0.0,0.000788,0.000263,0.0,0.0,0.000263,0.0,0.0,0.000920,0.0,0.0,0.0,0.000131,0.0,0.000657,0.0,0.0,0.0,0.000131,0.0,0.0,0.000131,0.0,0.0,0.0,0.0,0.000131,0.0,0.000131,0.0,0.0,0.0,0.0,0.000263,0.000131,0.0,0.0,0.000526,0.0,0.000263,0.001051,0.000131,0.0,0.0,0.000131,0.000263,0.0,0.000131,0.000526,0.000394,0.0,0.001577,0.000263,0.008934,0.0,0.000657,0.0,0.0,0.0,0.0,0.000131,0.000131,0.001051,0.000788,0.000788,0.0,0.000526,0.0,0.0,0.000131,0.000526,0.005518,0.0,0.001182,0.0,0.000657,0.0,0.0,0.0,0.000394,0.000131,0.0,0.0,0.001051,0.0,0.0,0.0,0.0,0.000263,0.0,0.000263,0.0,0.000788,0.000131,0.045588,0.0,0.000920,0.002233,0.003547,0.0,0.0,0.0,0.000131,0.001445,0.010510,0.0,0.0,0.0,0.000131,0.000131,0.001445,0.0,0.000131,0.004861,0.016816
1,238,5,0,-1,z03,Box Elder County Unincorporated,95,49,057,210101,1,490572101011,Block Group 1,G5030,S,86898873.0,19742.0,+41.3700825,-111.9334158,21478.059645,0.887017,0.000041,"{'rings': [[[-111.95909781387314, 41.433683592...",95,"{'rings': [[[-111.975799, 41.384468999999996],...",1918,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000165,0.000083,0.0,0.0,0.0,0.0,0.000041,0.0,0.0,0.0,0.000041,0.0,0.0,0.000041,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000041,0.0,0.0,0.0,0.000083,0.000083,0.0,0.0,0.0,0.0,0.0,0.000289,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000083,0.0,0.000083,0.0,0.000743,0.0,0.0,0.0,0.0,0.000083,0.0,0.0,0.000041,0.000124,0.000083,0.000041,0.000041,0.0,0.0,0.0,0.0,0.000083,0.0,0.0,0.000083,0.0,0.0,0.0,0.000041,0.000041,0.000041,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000083,0.0,0.000041,0.0,0.000083,0.000041,0.003469,0.000124,0.0,0.0,0.0,0.000248,0.000083,0.0,0.0,0.000083,0.0,0.0,0.000289,0.0,0.0,0.0,0.000041,0.

In [128]:
df_LEHDCommutePatterns_CityCalcsSum = df_LEHDCommutePatterns_CityCalcs.groupby(['CODE3','FIRST_NAME'], as_index=False)[dColCodesWCounty].apply(lambda x : x.astype(int).sum())
df_LEHDCommutePatterns_CityCalcsSum = df_LEHDCommutePatterns_CityCalcsSum.reset_index()
df_LEHDCommutePatterns_CityCalcsSum

,index,CODE3,FIRST_NAME,AFK_h,ALA_h,ALP_h,BDL_h,BGM_h,BNT_h,BRT_h,CDF_h,CEN_h,CHA_h,CHL_h,CLF_h,CLI_h,CMT_h,COA_h,CWH_h,DAN_h,DRA_h,EAG_h,ELK_h,EMT_h,FAR_h,FCS_h,FFD_h,FRR_h,FTH_h,GLA_h,GOS_h,GRL_h,HAR_h,HDT_h,HEB_h,HER_h,HGH_h,HNF_h,HOL_h,HOO_h,HVL_h,IND_h,INT_h,KAY_h,KMS_h,KMT_h,LAY_h,LEH_h,LIN_h,MAP_h,MID_h,MLC_h,MMT_h,MRG_h,MSL_h,MUR_h,MWY_h,NOG_h,NSL_h,OGD_h,OKL_h,ORM_h,PAY_h,PGR_h,PLN_h,PRK_h,PRY_h,PVO_h,PVW_h,ROY_h,RVD_h,RVT_h,SAN_h,SAQ_h,SAR_h,SFK_h,SJC_h,SLC_h,SLM_h,SOG_h,SPV_h,SSL_h,SUN_h,SWE_h,SYR_h,TAY_h,TOO_h,UIN_h,VIN_h,WAT_h,WBG_h,WDL_h,WEB_h,WHT_h,WHV_h,WIL_h,WJC_h,WPT_h,WVC_h,WXC_h,TTL_h,AFK_w,ALA_w,ALP_w,BDL_w,BGM_w,BNT_w,BRT_w,CDF_w,CEN_w,CHA_w,CHL_w,CLF_w,CLI_w,CMT_w,COA_w,CWH_w,DAN_w,DRA_w,EAG_w,ELK_w,EMT_w,FAR_w,FCS_w,FFD_w,FRR_w,FTH_w,GLA_w,GOS_w,GRL_w,HAR_w,HDT_w,HEB_w,HER_w,HGH_w,HNF_w,HOL_w,HOO_w,HVL_w,IND_w,INT_w,KAY_w,KMS_w,KMT_w,LAY_w,LEH_w,LIN_w,MAP_w,MID_w,MLC_w,MMT_w,MRG_w,MSL_w,MUR_w,MWY_w,NOG_w,NSL_w,OGD_w,OKL_w,ORM_w,PAY_w,PGR_w,PLN_w,PRK_w,PRY_w,PVO_w,PVW_w,ROY_w,RVD_w,RVT_w,SAN_w,SAQ_w,SAR_w,SFK_w,SJC_w,SLC_w,SLM_w,SOG_w,SPV_w,SSL_w,SUN_w,SWE_w,SYR_w,TAY_w,TOO_w,UIN_w,VIN_w,WAT_w,WBG_w,WDL_w,WEB_w,WHT_w,WHV_w,WIL_w,WJC_w,WPT_w,WVC_w,WXC_w,TTL_w,z03_h,z03_w,z11_h,z11_w,z23_h,z23_w,z29_h,z29_w,z35_h,z35_w,z43_h,z43_w,z45_h,z45_w,z49_h,z49_w,z51_h,z51_w,z57_h,z57_w
0,0,AFK,American Fork,1679,0,286,126,43,95,1,7,34,2,312,46,40,2,2,91,1,449,756,20,3,36,0,1,7,6,8,2,35,7,1,86,257,550,0,75,10,0,0,0,63,3,76,156,1992,319,132,94,124,88,5,1,139,19,23,19,135,1,2307,202,1453,8,6,5,1312,9,61,9,364,432,115,788,499,406,379,76,28,506,54,6,10,36,155,132,0,2,18,0,3,2,18,12,3,427,13,311,20,21026,1689,3,107,47,8,19,1,1,9,1,29,8,3,0,1,104,1,441,34,0,0,8,0,0,0,0,0,0,1,0,0,33,37,169,0,16,3,0,0,0,3,0,5,51,1379,531,17,73,148,1,0,0,197,8,3,32,81,1,1352,28,596,0,12,1,1375,0,15,13,39,280,9,108,142,205,764,11,5,176,115,0,0,4,66,15,0,21,1,0,0,0,1,3,1,119,0,212,7,11975,58,10,694,207,115,9,5,0,4277,3046,25,16,178,16,13418,7935,134,47,406,161
1,1,ALA,Alta,1,2,0,0,0,0,1,0,0,0,0,0,0,0,0,40,0,8,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,0,0,14,0,0,0,0,0,0,2,0,1,0,0,13,19,1,0,0,11,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,3,42,0,0,0,6,41,0,0,0,5,0,0,0,7,1,0,0,0,0,0,0,2,0,0,11,0,7,0,297,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,1,0,0,3,8,0,0,0,6,0,0,0,0,0,0,0,0,0,2,0,1,0,0,0,0,9,0,0,0,2,26,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,5,0,115,0,0,6,3,0,0,0,0,252,93,0,2,1,0,12,5,1,0,1,0
2,2,ALP,Alpine,114,0,302,10,3,11,0,0,2,0,57,1,5,0,0,21,1,33,80,2,2,2,0,0,1,1,2,0,84,0,0,0,21,129,1,13,2,0,0,0,1,0,6,14,163,31,7,3,34,7,2,0,17,2,1,3,14,0,195,12,117,1,1,0,109,0,10,0,33,63,13,53,22,37,30,7,4,33,6,1,1,2,14,268,0,0,2,0,1,1,6,2,0,25,1,26,4,2682,273,0,273,11,1,10,0,0,1,0,18,4,1,4,0,54,0,135,10,0,0,2,0,0,0,0,0,0,3,1,0,8,5,131,0,25,0,0,0,0,7,2,1,16,427,138,7,46,94,3,0,1,116,2,2,10,35,0,296,9,168,1,18,0,341,1,4,4,24,140,1,23,20,88,374,6,2,30,64,0,1,4,45,8,0,1,0,0,0,1,0,3,0,55,0,111,2,4137,3,1,64,79,13,1,2,0,457,1468,3,21,354,13,1490,2226,5,11,43,69
3,3,BDL,Bluffdale,50,0,11,394,3,17,0,4,4,0,10,9,4,3,0,26,1,198,133,0,0,5,0,1,0,0,1,1,9,1,0,11,163,30,0,26,0,0,0,0,1,0,80,23,159,12,6,64,65,33,0,0,78,1,3,12,29,0,101,18,27,3,0,0,84,0,14,2,296,228,14,84,37,179,132,10,2,24,23,0,2,8,128,25,0,0,1,0,1,0,8,4,0,277,1,181,1,4093,139,0,9,386,0,21,0,0,15,0,1,5,1,0,0,147,0,574,13,0,0,0,0,0,0,0,0,0,1,0,0,1,51,5,0,50,0,0,0,0,2,0,18,14,294,50,0,175,214,11,0,0,298,0,0,30,27,0,206,4,26,0,5,0,121,0,3,6,227,459,0,10,10,368,1000,0,2,10,180,0,0,2,104,7,0,1,0,0,0,7,0,0,0,350,1,310,7,6335,3,0,123,142,14,0,0,0,2670,5006,3,5,38,9,884,949,21,3,81,56
4,4,BGM,Brigham City,6,0,2,2,2282,18,0,0,5,0,5,55,28,0,2,9,0,13,4,1,1,6,0,0,30,5,2,1,3,41,1,9,3,6,2,13,23,2,0,0,22,2,16,86,13,4,2,7,11,7,9,8,16,1,146,10,467,1,24,4,9,31,4,459,26,80,148,32,13,27,3,6,10,13,77,2,53,15,9,8,4,33,14,27,3,0,35,0,0,3,3,34,92,24,15,46,11,7081,48,0,4,7,2452,50,0,0,24,0,3,159,19,0,2,27,0,72,4,0,1,29,0,0,76,1,0,0,5,28,0,15,5,6,0,21,4,0,0,0,34,5,9,208,39,21,2,34,72,6,6,32,111,1,69,76,1028,0,

In [129]:
df_LEHDCommutePatterns_CityCalcsSum.sum()

index                                                      7875
CODE3         AFKALAALPBDLBGMBNTBRTCDFCENCHACHLCLFCLICMTCOAC...
FIRST_NAME    American ForkAltaAlpineBluffdaleBrigham CityBo...
AFK_h                                                     11694
ALA_h                                                       105
                                    ...                        
z49_w                                                    232929
z51_h                                                     11163
z51_w                                                      6750
z57_h                                                    107034
z57_w                                                     99203
Length: 219, dtype: object

In [131]:
sdf_LEHDCommutePatterns_joinCity_Percent_SA = pd.DataFrame.merge(sdf_CityTownshipCounty_BlockGroups,sdf_LEHDCommutePatterns_Percent_SA,on=['GEOID','STATEFP','COUNTYFP','TRACTCE','BLKGRPCE','NAMELSAD','MTFCC', 'FUNCSTAT','ALAND','AWATER','INTPTLAT','INTPTLON','BGAcres'])
sdf_LEHDCommutePatterns_joinCity_Percent_SA = sdf_LEHDCommutePatterns_joinCity_Percent_SA.rename(columns={"first_name":"FIRST_NAME", "code3":"CODE3"})
df_LEHDCommutePatterns_CityCalcs_Percent_SA = sdf_LEHDCommutePatterns_joinCity_Percent_SA.copy()

for i in dColCodesWCounty:    
    df_LEHDCommutePatterns_CityCalcs_Percent_SA[i] = df_LEHDCommutePatterns_CityCalcs_Percent_SA[i] * df_LEHDCommutePatterns_CityCalcs_Percent_SA['PortionBG']


display(df_LEHDCommutePatterns_CityCalcs_Percent_SA.sum())
    
df_LEHDCommutePatterns_CityCalcs_Percent_SA = df_LEHDCommutePatterns_CityCalcs_Percent_SA.fillna(0)
   

df_LEHDCommutePatterns_CityCalcsSum_Percent_SA = df_LEHDCommutePatterns_CityCalcs_Percent_SA.groupby(['CODE3','FIRST_NAME'], as_index=False)[dColCodesWCounty].apply(lambda x : x.astype(float).sum())
df_LEHDCommutePatterns_CityCalcsSum_Percent_SA = df_LEHDCommutePatterns_CityCalcsSum_Percent_SA.reset_index()
df_LEHDCommutePatterns_CityCalcsSum_Percent_SA
    
display(df_LEHDCommutePatterns_CityCalcsSum_Percent_SA.sum())

FID_x                                                   6578904
FID_Munici                                               251195
Id                                                            0
fid_muni_1                                               139499
CODE3         z05z03z57NOGz05z05z05z05z05z05z05z05z05z05z03z...
                                    ...                        
z49_w                                                 99.999924
z51_h                                                  99.99998
z51_w                                                 99.999888
z57_h                                                 99.999987
z57_w                                                 99.999845
Length: 240, dtype: object

index                                                      7875
CODE3         AFKALAALPBDLBGMBNTBRTCDFCENCHACHLCLFCLICMTCOAC...
FIRST_NAME    American ForkAltaAlpineBluffdaleBrigham CityBo...
AFK_h                                                 99.999992
ALA_h                                                100.000006
                                    ...                        
z49_w                                                 99.999924
z51_h                                                  99.99998
z51_w                                                 99.999888
z57_h                                                 99.999987
z57_w                                                 99.999845
Length: 219, dtype: object

In [132]:
sdf_LEHDCommutePatterns_City_Number = pd.DataFrame.merge(sdf_CityTownshipWCodes,df_LEHDCommutePatterns_CityCalcsSum,on=['CODE3','FIRST_NAME'])
sdf_LEHDCommutePatterns_City_Percent_SA = pd.DataFrame.merge(sdf_CityTownshipWCodes,df_LEHDCommutePatterns_CityCalcsSum_Percent_SA,on=['CODE3','FIRST_NAME'])

In [133]:
sdf_LEHDCommutePatterns_City_Number.spatial.to_featureclass(fname_CitiesWCommutePatterns_Number)

'c:\\Users\\cday\\Documents\\projects\\Census-LEHD-Commute-Patterns\\results\\results2018\\LEHDCommutePatterns.gdb\\City_CommutePatterns_Number'

In [134]:
sdf_LEHDCommutePatterns_City_Percent_SA.spatial.to_featureclass(fname_CitiesWCommutePatterns_Percent_SA)

'c:\\Users\\cday\\Documents\\projects\\Census-LEHD-Commute-Patterns\\results\\results2018\\LEHDCommutePatterns.gdb\\City_CommutePatterns_Percent_SelectedArea'

In [135]:
sdf_LEHDCommutePatterns_City_Number

,OBJECTID,CODE3,FIRST_NAME,FIRST_COUNTYNBR,SHAPE,index,AFK_h,ALA_h,ALP_h,BDL_h,BGM_h,BNT_h,BRT_h,CDF_h,CEN_h,CHA_h,CHL_h,CLF_h,CLI_h,CMT_h,COA_h,CWH_h,DAN_h,DRA_h,EAG_h,ELK_h,EMT_h,FAR_h,FCS_h,FFD_h,FRR_h,FTH_h,GLA_h,GOS_h,GRL_h,HAR_h,HDT_h,HEB_h,HER_h,HGH_h,HNF_h,HOL_h,HOO_h,HVL_h,IND_h,INT_h,KAY_h,KMS_h,KMT_h,LAY_h,LEH_h,LIN_h,MAP_h,MID_h,MLC_h,MMT_h,MRG_h,MSL_h,MUR_h,MWY_h,NOG_h,NSL_h,OGD_h,OKL_h,ORM_h,PAY_h,PGR_h,PLN_h,PRK_h,PRY_h,PVO_h,PVW_h,ROY_h,RVD_h,RVT_h,SAN_h,SAQ_h,SAR_h,SFK_h,SJC_h,SLC_h,SLM_h,SOG_h,SPV_h,SSL_h,SUN_h,SWE_h,SYR_h,TAY_h,TOO_h,UIN_h,VIN_h,WAT_h,WBG_h,WDL_h,WEB_h,WHT_h,WHV_h,WIL_h,WJC_h,WPT_h,WVC_h,WXC_h,TTL_h,AFK_w,ALA_w,ALP_w,BDL_w,BGM_w,BNT_w,BRT_w,CDF_w,CEN_w,CHA_w,CHL_w,CLF_w,CLI_w,CMT_w,COA_w,CWH_w,DAN_w,DRA_w,EAG_w,ELK_w,EMT_w,FAR_w,FCS_w,FFD_w,FRR_w,FTH_w,GLA_w,GOS_w,GRL_w,HAR_w,HDT_w,HEB_w,HER_w,HGH_w,HNF_w,HOL_w,HOO_w,HVL_w,IND_w,INT_w,KAY_w,KMS_w,KMT_w,LAY_w,LEH_w,LIN_w,MAP_w,MID_w,MLC_w,MMT_w,MRG_w,MSL_w,MUR_w,MWY_w,NOG_w,NSL_w,OGD_w,OKL_w,ORM_w,PAY_w,PGR_w,PLN_w,PRK_w,PRY_w,PVO_w,PVW_w,ROY_w,RVD_w,RVT_w,SAN_w,SAQ_w,SAR_w,SFK_w,SJC_w,SLC_w,SLM_w,SOG_w,SPV_w,SSL_w,SUN_w,SWE_w,SYR_w,TAY_w,TOO_w,UIN_w,VIN_w,WAT_w,WBG_w,WDL_w,WEB_w,WHT_w,WHV_w,WIL_w,WJC_w,WPT_w,WVC_w,WXC_w,TTL_w,z03_h,z03_w,z11_h,z11_w,z23_h,z23_w,z29_h,z29_w,z35_h,z35_w,z43_h,z43_w,z45_h,z45_w,z49_h,z49_w,z51_h,z51_w,z57_h,z57_w
0,1,AFK,American Fork,25,"{""rings"": [[[-111.77435121873373, 40.353624868...",0,1679,0,286,126,43,95,1,7,34,2,312,46,40,2,2,91,1,449,756,20,3,36,0,1,7,6,8,2,35,7,1,86,257,550,0,75,10,0,0,0,63,3,76,156,1992,319,132,94,124,88,5,1,139,19,23,19,135,1,2307,202,1453,8,6,5,1312,9,61,9,364,432,115,788,499,406,379,76,28,506,54,6,10,36,155,132,0,2,18,0,3,2,18,12,3,427,13,311,20,21026,1689,3,107,47,8,19,1,1,9,1,29,8,3,0,1,104,1,441,34,0,0,8,0,0,0,0,0,0,1,0,0,33,37,169,0,16,3,0,0,0,3,0,5,51,1379,531,17,73,148,1,0,0,197,8,3,32,81,1,1352,28,596,0,12,1,1375,0,15,13,39,280,9,108,142,205,764,11,5,176,115,0,0,4,66,15,0,21,1,0,0,0,1,3,1,119,0,212,7,11975,58,10,694,207,115,9,5,0,4277,3046,25,16,178,16,13418,7935,134,47,406,161
1,2,ALA,Alta,18,"{""rings"": [[[-111.59946851855727, 40.577431980...",1,1,2,0,0,0,0,1,0,0,0,0,0,0,0,0,40,0,8,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,0,0,14,0,0,0,0,0,0,2,0,1,0,0,13,19,1,0,0,11,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,3,42,0,0,0,6,41,0,0,0,5,0,0,0,7,1,0,0,0,0,0,0,2,0,0,11,0,7,0,297,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,1,0,0,3,8,0,0,0,6,0,0,0,0,0,0,0,0,0,2,0,1,0,0,0,0,9,0,0,0,2,26,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,5,0,115,0,0,6,3,0,0,0,0,252,93,0,2,1,0,12,5,1,0,1,0
2,3,ALP,Alpine,25,"{""rings"": [[[-111.75455302160464, 40.479513910...",2,114,0,302,10,3,11,0,0,2,0,57,1,5,0,0,21,1,33,80,2,2,2,0,0,1,1,2,0,84,0,0,0,21,129,1,13,2,0,0,0,1,0,6,14,163,31,7,3,34,7,2,0,17,2,1,3,14,0,195,12,117,1,1,0,109,0,10,0,33,63,13,53,22,37,30,7,4,33,6,1,1,2,14,268,0,0,2,0,1,1,6,2,0,25,1,26,4,2682,273,0,273,11,1,10,0,0,1,0,18,4,1,4,0,54,0,135,10,0,0,2,0,0,0,0,0,0,3,1,0,8,5,131,0,25,0,0,0,0,7,2,1,16,427,138,7,46,94,3,0,1,116,2,2,10,35,0,296,9,168,1,18,0,341,1,4,4,24,140,1,23,20,88,374,6,2,30,64,0,1,4,45,8,0,1,0,0,0,1,0,3,0,55,0,111,2,4137,3,1,64,79,13,1,2,0,457,1468,3,21,354,13,1490,2226,5,11,43,69
3,4,BDL,Bluffdale,18,"{""rings"": [[[-111.9704071902396, 40.5008379065...",3,50,0,11,394,3,17,0,4,4,0,10,9,4,3,0,26,1,198,133,0,0,5,0,1,0,0,1,1,9,1,0,11,163,30,0,26,0,0,0,0,1,0,80,23,159,12,6,64,65,33,0,0,78,1,3,12,29,0,101,18,27,3,0,0,84,0,14,2,296,228,14,84,37,179,132,10,2,24,23,0,2,8,128,25,0,0,1,0,1,0,8,4,0,277,1,181,1,4093,139,0,9,386,0,21,0,0,15,0,1,5,1,0,0,147,0,574,13,0,0,0,0,0,0,0,0,0,1,0,0,1,51,5,0,50,0,0,0,0,2,0,18,14,294,50,0,175,214,11,0,0,298,0,0,30,27,0,206,4,26,0,5,0,121,0,3,6,227,459,0,10,10,368,1000,0,2,10,180,0,0,2,104,7,0,1,0,0,0,7,0,0,0,350,1,310,7,6335,3,0,123,142,14,0,0,0,2670,5006,3,5,38,9,884,949,21,3,81,56
4,5,BGM,Brigham City,02,"{""rings"": [[[-112.01898409573937, 41.535781784...",4,6,0,2,2,2282,18,0,0,5,0,5,55,28,0,2,9,0,13,4,1,1,6,0,0,30,5,2,1,3,41

# Calculate Percents by Map Units

In [136]:
# prepare column list

#display(dColCodesWCounty)
dColCodesWCounty = dColCodesWCounty.copy()

dColCodesWCounty_x = [x for x in dColCodesWCounty if "w" not in x]
dColCodesWCounty_y = [x for x in dColCodesWCounty if "h" not in x]

#remove and append to make sure at end
dColCodesWCounty_x.remove('TTL_h')
dColCodesWCounty_y.remove('TTL_w')
dColCodesWCounty_x.append('TTL_h')
dColCodesWCounty_y.append('TTL_w')

display(dColCodesWCounty_x)
display(dColCodesWCounty_y)

dColCodesNoTotal_x = [x for x in dColCodes if "w" not in x]
dColCodesNoTotal_x.remove('TTL_h')

dColCodesNoTotal_x = [x for x in dColCodes if "h" not in x]
dColCodesNoTotal_x.remove('TTL_w')

['AFK_h',
 'ALA_h',
 'ALP_h',
 'BDL_h',
 'BGM_h',
 'BNT_h',
 'BRT_h',
 'CDF_h',
 'CEN_h',
 'CHA_h',
 'CHL_h',
 'CLF_h',
 'CLI_h',
 'CMT_h',
 'COA_h',
 'CWH_h',
 'DAN_h',
 'DRA_h',
 'EAG_h',
 'ELK_h',
 'EMT_h',
 'FAR_h',
 'FCS_h',
 'FFD_h',
 'FRR_h',
 'FTH_h',
 'GLA_h',
 'GOS_h',
 'GRL_h',
 'HAR_h',
 'HDT_h',
 'HEB_h',
 'HER_h',
 'HGH_h',
 'HNF_h',
 'HOL_h',
 'HOO_h',
 'HVL_h',
 'IND_h',
 'INT_h',
 'KAY_h',
 'KMS_h',
 'KMT_h',
 'LAY_h',
 'LEH_h',
 'LIN_h',
 'MAP_h',
 'MID_h',
 'MLC_h',
 'MMT_h',
 'MRG_h',
 'MSL_h',
 'MUR_h',
 'MWY_h',
 'NOG_h',
 'NSL_h',
 'OGD_h',
 'OKL_h',
 'ORM_h',
 'PAY_h',
 'PGR_h',
 'PLN_h',
 'PRK_h',
 'PRY_h',
 'PVO_h',
 'PVW_h',
 'ROY_h',
 'RVD_h',
 'RVT_h',
 'SAN_h',
 'SAQ_h',
 'SAR_h',
 'SFK_h',
 'SJC_h',
 'SLC_h',
 'SLM_h',
 'SOG_h',
 'SPV_h',
 'SSL_h',
 'SUN_h',
 'SWE_h',
 'SYR_h',
 'TAY_h',
 'TOO_h',
 'UIN_h',
 'VIN_h',
 'WAT_h',
 'WBG_h',
 'WDL_h',
 'WEB_h',
 'WHT_h',
 'WHV_h',
 'WIL_h',
 'WJC_h',
 'WPT_h',
 'WVC_h',
 'WXC_h',
 'z03_h',
 'z11_h',
 'z23_h',


['AFK_w',
 'ALA_w',
 'ALP_w',
 'BDL_w',
 'BGM_w',
 'BNT_w',
 'BRT_w',
 'CDF_w',
 'CEN_w',
 'CHA_w',
 'CHL_w',
 'CLF_w',
 'CLI_w',
 'CMT_w',
 'COA_w',
 'CWH_w',
 'DAN_w',
 'DRA_w',
 'EAG_w',
 'ELK_w',
 'EMT_w',
 'FAR_w',
 'FCS_w',
 'FFD_w',
 'FRR_w',
 'FTH_w',
 'GLA_w',
 'GOS_w',
 'GRL_w',
 'HAR_w',
 'HDT_w',
 'HEB_w',
 'HER_w',
 'HGH_w',
 'HNF_w',
 'HOL_w',
 'HOO_w',
 'HVL_w',
 'IND_w',
 'INT_w',
 'KAY_w',
 'KMS_w',
 'KMT_w',
 'LAY_w',
 'LEH_w',
 'LIN_w',
 'MAP_w',
 'MID_w',
 'MLC_w',
 'MMT_w',
 'MRG_w',
 'MSL_w',
 'MUR_w',
 'MWY_w',
 'NOG_w',
 'NSL_w',
 'OGD_w',
 'OKL_w',
 'ORM_w',
 'PAY_w',
 'PGR_w',
 'PLN_w',
 'PRK_w',
 'PRY_w',
 'PVO_w',
 'PVW_w',
 'ROY_w',
 'RVD_w',
 'RVT_w',
 'SAN_w',
 'SAQ_w',
 'SAR_w',
 'SFK_w',
 'SJC_w',
 'SLC_w',
 'SLM_w',
 'SOG_w',
 'SPV_w',
 'SSL_w',
 'SUN_w',
 'SWE_w',
 'SYR_w',
 'TAY_w',
 'TOO_w',
 'UIN_w',
 'VIN_w',
 'WAT_w',
 'WBG_w',
 'WDL_w',
 'WEB_w',
 'WHT_w',
 'WHV_w',
 'WIL_w',
 'WJC_w',
 'WPT_w',
 'WVC_w',
 'WXC_w',
 'z03_w',
 'z11_w',
 'z23_w',


In [137]:
# block group
sdf_LEHDCommutePatterns_Percent_MU = sdf_LEHDCommutePatterns.copy()
sdf_LEHDCommutePatterns_tract_Percent_MU  = sdf_LEHDCommutePatterns_tract.copy()
sdf_LEHDCommutePatterns_City_Percent_MU  = sdf_LEHDCommutePatterns_City_Number.copy()

for x in dColCodesWCounty_x:   
    #display(x)
    sdf_LEHDCommutePatterns_Percent_MU      [x] = sdf_LEHDCommutePatterns_Percent_MU      [x] / sdf_LEHDCommutePatterns_Percent_MU      ['TTL_h'] * 100
    sdf_LEHDCommutePatterns_tract_Percent_MU[x] = sdf_LEHDCommutePatterns_tract_Percent_MU[x] / sdf_LEHDCommutePatterns_tract_Percent_MU['TTL_h'] * 100
    sdf_LEHDCommutePatterns_City_Percent_MU [x] = sdf_LEHDCommutePatterns_City_Percent_MU [x] / sdf_LEHDCommutePatterns_City_Percent_MU ['TTL_h'] * 100

for y in dColCodesWCounty_y:   
    #display(y)
    sdf_LEHDCommutePatterns_Percent_MU      [y] = sdf_LEHDCommutePatterns_Percent_MU      [y] / sdf_LEHDCommutePatterns_Percent_MU      ['TTL_w'] * 100
    sdf_LEHDCommutePatterns_tract_Percent_MU[y] = sdf_LEHDCommutePatterns_tract_Percent_MU[y] / sdf_LEHDCommutePatterns_tract_Percent_MU['TTL_w'] * 100
    sdf_LEHDCommutePatterns_City_Percent_MU [y] = sdf_LEHDCommutePatterns_City_Percent_MU [y] / sdf_LEHDCommutePatterns_City_Percent_MU ['TTL_w'] * 100
    

In [138]:
#check that everything is near 100 (subtracting out non-city areas)
display(sdf_LEHDCommutePatterns_Percent_MU      .loc[:,dColCodesNoTotal_x].sum(axis=1))
display(sdf_LEHDCommutePatterns_tract_Percent_MU.loc[:,dColCodesNoTotal_x].sum(axis=1))
display(sdf_LEHDCommutePatterns_City_Percent_MU .loc[:,dColCodesNoTotal_x].sum(axis=1))

0       96.056623
1       95.652174
2       97.188049
3       97.471910
4       97.443182
          ...    
2014    94.126506
2015    93.772242
2016    92.699115
2017    92.105263
2018    92.774566
Length: 2019, dtype: float64

0      96.342902
1      95.646842
2      92.147686
3      96.082546
4      96.892265
         ...    
711    96.519807
712    96.460573
713    96.696697
714    97.612958
715    20.510397
Length: 716, dtype: float64

0     91.849687
1     73.913043
2     89.968576
3     94.364641
4     79.517796
        ...    
92    64.492754
93    95.147803
94    88.397031
95    95.626333
96    90.059473
Length: 97, dtype: float64

In [139]:
#export to GDB
deleteIfExists(fname_BlockGroupsWCommutePatterns_Percent_MU)
deleteIfExists(fname_TractsWCommutePatterns_Percent_MU     )
deleteIfExists(fname_CitiesWCommutePatterns_Percent_MU     )

sdf_LEHDCommutePatterns_Percent_MU      .spatial.to_featureclass(fname_BlockGroupsWCommutePatterns_Percent_MU)
sdf_LEHDCommutePatterns_tract_Percent_MU.spatial.to_featureclass(fname_TractsWCommutePatterns_Percent_MU     )
sdf_LEHDCommutePatterns_City_Percent_MU .spatial.to_featureclass(fname_CitiesWCommutePatterns_Percent_MU     )

'c:\\Users\\cday\\Documents\\projects\\Census-LEHD-Commute-Patterns\\results\\results2018\\LEHDCommutePatterns.gdb\\City_CommutePatterns_Percent_MapUnit'

# Summarize by County

In [140]:
df_CityTownship_NameCode

,NAME,CODE3,zCOFIPS
0,Alpine,ALP,z49
1,American Fork,AFK,z49
2,Cedar Fort,CDF,z49
3,Cedar Hills,CHL,z49
4,Eagle Mountain,EAG,z49
...,...,...,...
92,West Haven,WHV,z57
93,Grantsville,GRL,z45
94,Tooele,TOO,z45
95,Morgan,MRG,z29


In [141]:
display(df_LEHDCommutePatterns_CityCalcsSum)
display(df_LEHDCommutePatterns_CityCalcsSum_Percent_SA)

,index,CODE3,FIRST_NAME,AFK_h,ALA_h,ALP_h,BDL_h,BGM_h,BNT_h,BRT_h,CDF_h,CEN_h,CHA_h,CHL_h,CLF_h,CLI_h,CMT_h,COA_h,CWH_h,DAN_h,DRA_h,EAG_h,ELK_h,EMT_h,FAR_h,FCS_h,FFD_h,FRR_h,FTH_h,GLA_h,GOS_h,GRL_h,HAR_h,HDT_h,HEB_h,HER_h,HGH_h,HNF_h,HOL_h,HOO_h,HVL_h,IND_h,INT_h,KAY_h,KMS_h,KMT_h,LAY_h,LEH_h,LIN_h,MAP_h,MID_h,MLC_h,MMT_h,MRG_h,MSL_h,MUR_h,MWY_h,NOG_h,NSL_h,OGD_h,OKL_h,ORM_h,PAY_h,PGR_h,PLN_h,PRK_h,PRY_h,PVO_h,PVW_h,ROY_h,RVD_h,RVT_h,SAN_h,SAQ_h,SAR_h,SFK_h,SJC_h,SLC_h,SLM_h,SOG_h,SPV_h,SSL_h,SUN_h,SWE_h,SYR_h,TAY_h,TOO_h,UIN_h,VIN_h,WAT_h,WBG_h,WDL_h,WEB_h,WHT_h,WHV_h,WIL_h,WJC_h,WPT_h,WVC_h,WXC_h,TTL_h,AFK_w,ALA_w,ALP_w,BDL_w,BGM_w,BNT_w,BRT_w,CDF_w,CEN_w,CHA_w,CHL_w,CLF_w,CLI_w,CMT_w,COA_w,CWH_w,DAN_w,DRA_w,EAG_w,ELK_w,EMT_w,FAR_w,FCS_w,FFD_w,FRR_w,FTH_w,GLA_w,GOS_w,GRL_w,HAR_w,HDT_w,HEB_w,HER_w,HGH_w,HNF_w,HOL_w,HOO_w,HVL_w,IND_w,INT_w,KAY_w,KMS_w,KMT_w,LAY_w,LEH_w,LIN_w,MAP_w,MID_w,MLC_w,MMT_w,MRG_w,MSL_w,MUR_w,MWY_w,NOG_w,NSL_w,OGD_w,OKL_w,ORM_w,PAY_w,PGR_w,PLN_w,PRK_w,PRY_w,PVO_w,PVW_w,ROY_w,RVD_w,RVT_w,SAN_w,SAQ_w,SAR_w,SFK_w,SJC_w,SLC_w,SLM_w,SOG_w,SPV_w,SSL_w,SUN_w,SWE_w,SYR_w,TAY_w,TOO_w,UIN_w,VIN_w,WAT_w,WBG_w,WDL_w,WEB_w,WHT_w,WHV_w,WIL_w,WJC_w,WPT_w,WVC_w,WXC_w,TTL_w,z03_h,z03_w,z11_h,z11_w,z23_h,z23_w,z29_h,z29_w,z35_h,z35_w,z43_h,z43_w,z45_h,z45_w,z49_h,z49_w,z51_h,z51_w,z57_h,z57_w
0,0,AFK,American Fork,1679,0,286,126,43,95,1,7,34,2,312,46,40,2,2,91,1,449,756,20,3,36,0,1,7,6,8,2,35,7,1,86,257,550,0,75,10,0,0,0,63,3,76,156,1992,319,132,94,124,88,5,1,139,19,23,19,135,1,2307,202,1453,8,6,5,1312,9,61,9,364,432,115,788,499,406,379,76,28,506,54,6,10,36,155,132,0,2,18,0,3,2,18,12,3,427,13,311,20,21026,1689,3,107,47,8,19,1,1,9,1,29,8,3,0,1,104,1,441,34,0,0,8,0,0,0,0,0,0,1,0,0,33,37,169,0,16,3,0,0,0,3,0,5,51,1379,531,17,73,148,1,0,0,197,8,3,32,81,1,1352,28,596,0,12,1,1375,0,15,13,39,280,9,108,142,205,764,11,5,176,115,0,0,4,66,15,0,21,1,0,0,0,1,3,1,119,0,212,7,11975,58,10,694,207,115,9,5,0,4277,3046,25,16,178,16,13418,7935,134,47,406,161
1,1,ALA,Alta,1,2,0,0,0,0,1,0,0,0,0,0,0,0,0,40,0,8,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,0,0,14,0,0,0,0,0,0,2,0,1,0,0,13,19,1,0,0,11,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,3,42,0,0,0,6,41,0,0,0,5,0,0,0,7,1,0,0,0,0,0,0,2,0,0,11,0,7,0,297,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,1,0,0,3,8,0,0,0,6,0,0,0,0,0,0,0,0,0,2,0,1,0,0,0,0,9,0,0,0,2,26,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,5,0,115,0,0,6,3,0,0,0,0,252,93,0,2,1,0,12,5,1,0,1,0
2,2,ALP,Alpine,114,0,302,10,3,11,0,0,2,0,57,1,5,0,0,21,1,33,80,2,2,2,0,0,1,1,2,0,84,0,0,0,21,129,1,13,2,0,0,0,1,0,6,14,163,31,7,3,34,7,2,0,17,2,1,3,14,0,195,12,117,1,1,0,109,0,10,0,33,63,13,53,22,37,30,7,4,33,6,1,1,2,14,268,0,0,2,0,1,1,6,2,0,25,1,26,4,2682,273,0,273,11,1,10,0,0,1,0,18,4,1,4,0,54,0,135,10,0,0,2,0,0,0,0,0,0,3,1,0,8,5,131,0,25,0,0,0,0,7,2,1,16,427,138,7,46,94,3,0,1,116,2,2,10,35,0,296,9,168,1,18,0,341,1,4,4,24,140,1,23,20,88,374,6,2,30,64,0,1,4,45,8,0,1,0,0,0,1,0,3,0,55,0,111,2,4137,3,1,64,79,13,1,2,0,457,1468,3,21,354,13,1490,2226,5,11,43,69
3,3,BDL,Bluffdale,50,0,11,394,3,17,0,4,4,0,10,9,4,3,0,26,1,198,133,0,0,5,0,1,0,0,1,1,9,1,0,11,163,30,0,26,0,0,0,0,1,0,80,23,159,12,6,64,65,33,0,0,78,1,3,12,29,0,101,18,27,3,0,0,84,0,14,2,296,228,14,84,37,179,132,10,2,24,23,0,2,8,128,25,0,0,1,0,1,0,8,4,0,277,1,181,1,4093,139,0,9,386,0,21,0,0,15,0,1,5,1,0,0,147,0,574,13,0,0,0,0,0,0,0,0,0,1,0,0,1,51,5,0,50,0,0,0,0,2,0,18,14,294,50,0,175,214,11,0,0,298,0,0,30,27,0,206,4,26,0,5,0,121,0,3,6,227,459,0,10,10,368,1000,0,2,10,180,0,0,2,104,7,0,1,0,0,0,7,0,0,0,350,1,310,7,6335,3,0,123,142,14,0,0,0,2670,5006,3,5,38,9,884,949,21,3,81,56
4,4,BGM,Brigham City,6,0,2,2,2282,18,0,0,5,0,5,55,28,0,2,9,0,13,4,1,1,6,0,0,30,5,2,1,3,41,1,9,3,6,2,13,23,2,0,0,22,2,16,86,13,4,2,7,11,7,9,8,16,1,146,10,467,1,24,4,9,31,4,459,26,80,148,32,13,27,3,6,10,13,77,2,53,15,9,8,4,33,14,27,3,0,35,0,0,3,3,34,92,24,15,46,11,7081,48,0,4,7,2452,50,0,0,24,0,3,159,19,0,2,27,0,72,4,0,1,29,0,0,76,1,0,0,5,28,0,15,5,6,0,21,4,0,0,0,34,5,9,208,39,21,2,34,72,6,6,32,111,1,69,76,1028,0,

,index,CODE3,FIRST_NAME,AFK_h,ALA_h,ALP_h,BDL_h,BGM_h,BNT_h,BRT_h,CDF_h,CEN_h,CHA_h,CHL_h,CLF_h,CLI_h,CMT_h,COA_h,CWH_h,DAN_h,DRA_h,EAG_h,ELK_h,EMT_h,FAR_h,FCS_h,FFD_h,FRR_h,FTH_h,GLA_h,GOS_h,GRL_h,HAR_h,HDT_h,HEB_h,HER_h,HGH_h,HNF_h,HOL_h,HOO_h,HVL_h,IND_h,INT_h,KAY_h,KMS_h,KMT_h,LAY_h,LEH_h,LIN_h,MAP_h,MID_h,MLC_h,MMT_h,MRG_h,MSL_h,MUR_h,MWY_h,NOG_h,NSL_h,OGD_h,OKL_h,ORM_h,PAY_h,PGR_h,PLN_h,PRK_h,PRY_h,PVO_h,PVW_h,ROY_h,RVD_h,RVT_h,SAN_h,SAQ_h,SAR_h,SFK_h,SJC_h,SLC_h,SLM_h,SOG_h,SPV_h,SSL_h,SUN_h,SWE_h,SYR_h,TAY_h,TOO_h,UIN_h,VIN_h,WAT_h,WBG_h,WDL_h,WEB_h,WHT_h,WHV_h,WIL_h,WJC_h,WPT_h,WVC_h,WXC_h,TTL_h,AFK_w,ALA_w,ALP_w,BDL_w,BGM_w,BNT_w,BRT_w,CDF_w,CEN_w,CHA_w,CHL_w,CLF_w,CLI_w,CMT_w,COA_w,CWH_w,DAN_w,DRA_w,EAG_w,ELK_w,EMT_w,FAR_w,FCS_w,FFD_w,FRR_w,FTH_w,GLA_w,GOS_w,GRL_w,HAR_w,HDT_w,HEB_w,HER_w,HGH_w,HNF_w,HOL_w,HOO_w,HVL_w,IND_w,INT_w,KAY_w,KMS_w,KMT_w,LAY_w,LEH_w,LIN_w,MAP_w,MID_w,MLC_w,MMT_w,MRG_w,MSL_w,MUR_w,MWY_w,NOG_w,NSL_w,OGD_w,OKL_w,ORM_w,PAY_w,PGR_w,PLN_w,PRK_w,PRY_w,PVO_w,PVW_w,ROY_w,RVD_w,RVT_w,SAN_w,SAQ_w,SAR_w,SFK_w,SJC_w,SLC_w,SLM_w,SOG_w,SPV_w,SSL_w,SUN_w,SWE_w,SYR_w,TAY_w,TOO_w,UIN_w,VIN_w,WAT_w,WBG_w,WDL_w,WEB_w,WHT_w,WHV_w,WIL_w,WJC_w,WPT_w,WVC_w,WXC_w,TTL_w,z03_h,z03_w,z11_h,z11_w,z23_h,z23_w,z29_h,z29_w,z35_h,z35_w,z43_h,z43_w,z45_h,z45_w,z49_h,z49_w,z51_h,z51_w,z57_h,z57_w
0,0,AFK,American Fork,13.783877,1.218193,6.693596,2.113599,0.544770,0.530338,1.408451,3.691058e+00,0.618671,2.177971,7.320558,0.430118,0.490782,0.981006,0.874791,0.609526,1.451567,2.095843,5.284967,1.791648,0.429131,4.994556e-01,0.598112,4.704245,3.060369e-01,0.362474,1.731732,1.299778,0.866158,0.350400,1.345840,1.175626,1.601020,7.231478,0.813910,0.596954,0.418693,0.413919,0.828863,0.752218,0.578916,0.909824,0.467546,0.526007,6.933829,6.523905e+00,3.111722,0.628528,0.459513,0.675366,0.516261,0.278380,0.597161,1.343069,0.327597,3.418806e-01,0.372089,0.722311,5.226877,2.216491,9.213036,5.045177e-01,0.288846,0.360857,3.340708,0.332482,3.921829e-01,0.364930,1.601944,0.935542,2.243960e+00,5.833462,2.590364,1.297377,0.443358,2.141178e+00,0.459334,3.107577,0.571153,0.513436,0.496702,3.789971e-01,0.544806,0.855986,0.271713,5.349500,0.585674,0.482125,1.158206,2.366404e-01,0.821339,0.383280,0.668538,0.749599,0.413144,0.513098,0.548552,1.513322,7.581500,0.487457,4.196949,1.279297,0.124643,0.265294,1.666667,2.471182,0.264115,0.946878,4.579197,0.077555,0.282283,0.000000,0.164745,0.568363,0.625523,1.197594,2.205569,1.245706,0.691051,1.538886e-01,0.000000,2.722201,3.501203e-02,0.108217,0.000000,3.333612,8.793877e-02,0.122008,0.000000,0.722369,1.064654,5.891179,0.000000,0.218100,0.520303,0.000000,0.000000,0.0,0.087289,0.105193,0.240585,0.229502,4.183504,4.320092,1.304506e+00,0.458487,0.325060,0.193528,0.000325,0.080735,0.473258,0.995585,0.140364,0.275002,0.157218,0.761488,2.864203,0.626639,5.682859,0.000000,0.182563,0.165785,2.374763,0.102946,0.320645,0.267974,0.494943,0.608292,1.578062,3.414142e+00,1.219887e+00,0.725611,0.324218,1.017961e+00,0.143230,1.474433,0.365985,0.082788,0.155423,2.025571e-01,0.374756,0.312888,0.000000,3.120291,0.097746,0.000000,0.532411,0.152280,7.557115e-01,1.854353e-01,1.048607,0.365221,0.141842,0.324514,0.265748,0.849375,0.518152,0.145144,0.487856,0.198265,2.243960e+00,1.578062,0.516261,0.000325,0.793965,0.447916,0.520398,0.184484,0.858363,0.272550,5.376422,3.397634,1.231786,0.761407,0.387433,0.169521
1,1,ALA,Alta,0.012547,1.408404,0.013234,0.009194,0.001994,0.004050,1.858568,0.000000e+00,0.005129,0.000000,0.009054,0.000000,0.003751,0.000000,0.000000,0.247654,0.000000,0.042412,0.002585,0.000003,0.078551,1.248724e-02,0.001923,0.000000,0.000000e+00,0.009036,0.000000,0.000000,0.000167,0.006183,0.057933,0.004885,0.013729,0.010564,0.000000,0.101776,0.000000,0.000000,0.000000,0.000000,0.007579,0.000000,0.015697,0.003011,0.005987,1.143658e-02,0.008345,0.082374,0.066399,0.013164,0.000000,0.000000,0.049073,0.025732,0.002156,4.644604e-03,0.000516,0.000000,0.004264,0.005953,0.008409,0.000000e+00,0.025008,0.000000,0.002907,0.000000

In [142]:
df_CitySum_wCounty_Join            = pd.DataFrame.merge(df_LEHDCommutePatterns_CityCalcsSum           , df_CityTownship_NameCode, on="CODE3", how="left")
df_CitySum_wCounty_Join_Percent_SA = pd.DataFrame.merge(df_LEHDCommutePatterns_CityCalcsSum_Percent_SA, df_CityTownship_NameCode, on="CODE3", how="left")

df_CitySum_wCounty_Join           .loc[df_CitySum_wCounty_Join           .CODE3.str[0] == 'z', 'zCOFIPS'] = df_CitySum_wCounty_Join           .CODE3
df_CitySum_wCounty_Join_Percent_SA.loc[df_CitySum_wCounty_Join_Percent_SA.CODE3.str[0] == 'z', 'zCOFIPS'] = df_CitySum_wCounty_Join_Percent_SA.CODE3

display(df_CitySum_wCounty_Join)
display(df_CitySum_wCounty_Join_Percent_SA)

,index,CODE3,FIRST_NAME,AFK_h,ALA_h,ALP_h,BDL_h,BGM_h,BNT_h,BRT_h,CDF_h,CEN_h,CHA_h,CHL_h,CLF_h,CLI_h,CMT_h,COA_h,CWH_h,DAN_h,DRA_h,EAG_h,ELK_h,EMT_h,FAR_h,FCS_h,FFD_h,FRR_h,FTH_h,GLA_h,GOS_h,GRL_h,HAR_h,HDT_h,HEB_h,HER_h,HGH_h,HNF_h,HOL_h,HOO_h,HVL_h,IND_h,INT_h,KAY_h,KMS_h,KMT_h,LAY_h,LEH_h,LIN_h,MAP_h,MID_h,MLC_h,MMT_h,MRG_h,MSL_h,MUR_h,MWY_h,NOG_h,NSL_h,OGD_h,OKL_h,ORM_h,PAY_h,PGR_h,PLN_h,PRK_h,PRY_h,PVO_h,PVW_h,ROY_h,RVD_h,RVT_h,SAN_h,SAQ_h,SAR_h,SFK_h,SJC_h,SLC_h,SLM_h,SOG_h,SPV_h,SSL_h,SUN_h,SWE_h,SYR_h,TAY_h,TOO_h,UIN_h,VIN_h,WAT_h,WBG_h,WDL_h,WEB_h,WHT_h,WHV_h,WIL_h,WJC_h,WPT_h,WVC_h,WXC_h,TTL_h,AFK_w,ALA_w,ALP_w,BDL_w,BGM_w,BNT_w,BRT_w,CDF_w,CEN_w,CHA_w,CHL_w,CLF_w,CLI_w,CMT_w,COA_w,CWH_w,DAN_w,DRA_w,EAG_w,ELK_w,EMT_w,FAR_w,FCS_w,FFD_w,FRR_w,FTH_w,GLA_w,GOS_w,GRL_w,HAR_w,HDT_w,HEB_w,HER_w,HGH_w,HNF_w,HOL_w,HOO_w,HVL_w,IND_w,INT_w,KAY_w,KMS_w,KMT_w,LAY_w,LEH_w,LIN_w,MAP_w,MID_w,MLC_w,MMT_w,MRG_w,MSL_w,MUR_w,MWY_w,NOG_w,NSL_w,OGD_w,OKL_w,ORM_w,PAY_w,PGR_w,PLN_w,PRK_w,PRY_w,PVO_w,PVW_w,ROY_w,RVD_w,RVT_w,SAN_w,SAQ_w,SAR_w,SFK_w,SJC_w,SLC_w,SLM_w,SOG_w,SPV_w,SSL_w,SUN_w,SWE_w,SYR_w,TAY_w,TOO_w,UIN_w,VIN_w,WAT_w,WBG_w,WDL_w,WEB_w,WHT_w,WHV_w,WIL_w,WJC_w,WPT_w,WVC_w,WXC_w,TTL_w,z03_h,z03_w,z11_h,z11_w,z23_h,z23_w,z29_h,z29_w,z35_h,z35_w,z43_h,z43_w,z45_h,z45_w,z49_h,z49_w,z51_h,z51_w,z57_h,z57_w,NAME,zCOFIPS
0,0,AFK,American Fork,1679,0,286,126,43,95,1,7,34,2,312,46,40,2,2,91,1,449,756,20,3,36,0,1,7,6,8,2,35,7,1,86,257,550,0,75,10,0,0,0,63,3,76,156,1992,319,132,94,124,88,5,1,139,19,23,19,135,1,2307,202,1453,8,6,5,1312,9,61,9,364,432,115,788,499,406,379,76,28,506,54,6,10,36,155,132,0,2,18,0,3,2,18,12,3,427,13,311,20,21026,1689,3,107,47,8,19,1,1,9,1,29,8,3,0,1,104,1,441,34,0,0,8,0,0,0,0,0,0,1,0,0,33,37,169,0,16,3,0,0,0,3,0,5,51,1379,531,17,73,148,1,0,0,197,8,3,32,81,1,1352,28,596,0,12,1,1375,0,15,13,39,280,9,108,142,205,764,11,5,176,115,0,0,4,66,15,0,21,1,0,0,0,1,3,1,119,0,212,7,11975,58,10,694,207,115,9,5,0,4277,3046,25,16,178,16,13418,7935,134,47,406,161,American Fork,z49
1,1,ALA,Alta,1,2,0,0,0,0,1,0,0,0,0,0,0,0,0,40,0,8,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,0,0,14,0,0,0,0,0,0,2,0,1,0,0,13,19,1,0,0,11,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,3,42,0,0,0,6,41,0,0,0,5,0,0,0,7,1,0,0,0,0,0,0,2,0,0,11,0,7,0,297,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,1,0,0,3,8,0,0,0,6,0,0,0,0,0,0,0,0,0,2,0,1,0,0,0,0,9,0,0,0,2,26,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,5,0,115,0,0,6,3,0,0,0,0,252,93,0,2,1,0,12,5,1,0,1,0,Alta,z35
2,2,ALP,Alpine,114,0,302,10,3,11,0,0,2,0,57,1,5,0,0,21,1,33,80,2,2,2,0,0,1,1,2,0,84,0,0,0,21,129,1,13,2,0,0,0,1,0,6,14,163,31,7,3,34,7,2,0,17,2,1,3,14,0,195,12,117,1,1,0,109,0,10,0,33,63,13,53,22,37,30,7,4,33,6,1,1,2,14,268,0,0,2,0,1,1,6,2,0,25,1,26,4,2682,273,0,273,11,1,10,0,0,1,0,18,4,1,4,0,54,0,135,10,0,0,2,0,0,0,0,0,0,3,1,0,8,5,131,0,25,0,0,0,0,7,2,1,16,427,138,7,46,94,3,0,1,116,2,2,10,35,0,296,9,168,1,18,0,341,1,4,4,24,140,1,23,20,88,374,6,2,30,64,0,1,4,45,8,0,1,0,0,0,1,0,3,0,55,0,111,2,4137,3,1,64,79,13,1,2,0,457,1468,3,21,354,13,1490,2226,5,11,43,69,Alpine,z49
3,3,BDL,Bluffdale,50,0,11,394,3,17,0,4,4,0,10,9,4,3,0,26,1,198,133,0,0,5,0,1,0,0,1,1,9,1,0,11,163,30,0,26,0,0,0,0,1,0,80,23,159,12,6,64,65,33,0,0,78,1,3,12,29,0,101,18,27,3,0,0,84,0,14,2,296,228,14,84,37,179,132,10,2,24,23,0,2,8,128,25,0,0,1,0,1,0,8,4,0,277,1,181,1,4093,139,0,9,386,0,21,0,0,15,0,1,5,1,0,0,147,0,574,13,0,0,0,0,0,0,0,0,0,1,0,0,1,51,5,0,50,0,0,0,0,2,0,18,14,294,50,0,175,214,11,0,0,298,0,0,30,27,0,206,4,26,0,5,0,121,0,3,6,227,459,0,10,10,368,1000,0,2,10,180,0,0,2,104,7,0,1,0,0,0,7,0,0,0,350,1,310,7,6335,3,0,123,142,14,0,0,0,2670,5006,3,5,38,9,884,949,21,3,81,56,Bluffdale,z35
4,4,BGM,Brigham City,6,0,2,2,2282,18,0,0,5,0,5,55,28,0,2,9,0,13,4,1,1,6,0,0,30,5,2,1,3,41,1,9,3,6,2,13,23,2,0,0,22,2,16,86,13,4,2,7,11,7,9,8,16,1,146,10,467,1,24,4,9,31,4,459,26,80,148,32,13,27,3,6,10,13,77,2,53,15,9,8,4,33,14,27,3,0,35,0,0,3,3,34,92,24,15,46,11,7081,48,0,4,7,2452,50,0,0,24,0,3,159,19,0,2,27,0,72,4,0,1,29,0,0,76,1,0,0,5,28,0,15,5,6

,index,CODE3,FIRST_NAME,AFK_h,ALA_h,ALP_h,BDL_h,BGM_h,BNT_h,BRT_h,CDF_h,CEN_h,CHA_h,CHL_h,CLF_h,CLI_h,CMT_h,COA_h,CWH_h,DAN_h,DRA_h,EAG_h,ELK_h,EMT_h,FAR_h,FCS_h,FFD_h,FRR_h,FTH_h,GLA_h,GOS_h,GRL_h,HAR_h,HDT_h,HEB_h,HER_h,HGH_h,HNF_h,HOL_h,HOO_h,HVL_h,IND_h,INT_h,KAY_h,KMS_h,KMT_h,LAY_h,LEH_h,LIN_h,MAP_h,MID_h,MLC_h,MMT_h,MRG_h,MSL_h,MUR_h,MWY_h,NOG_h,NSL_h,OGD_h,OKL_h,ORM_h,PAY_h,PGR_h,PLN_h,PRK_h,PRY_h,PVO_h,PVW_h,ROY_h,RVD_h,RVT_h,SAN_h,SAQ_h,SAR_h,SFK_h,SJC_h,SLC_h,SLM_h,SOG_h,SPV_h,SSL_h,SUN_h,SWE_h,SYR_h,TAY_h,TOO_h,UIN_h,VIN_h,WAT_h,WBG_h,WDL_h,WEB_h,WHT_h,WHV_h,WIL_h,WJC_h,WPT_h,WVC_h,WXC_h,TTL_h,AFK_w,ALA_w,ALP_w,BDL_w,BGM_w,BNT_w,BRT_w,CDF_w,CEN_w,CHA_w,CHL_w,CLF_w,CLI_w,CMT_w,COA_w,CWH_w,DAN_w,DRA_w,EAG_w,ELK_w,EMT_w,FAR_w,FCS_w,FFD_w,FRR_w,FTH_w,GLA_w,GOS_w,GRL_w,HAR_w,HDT_w,HEB_w,HER_w,HGH_w,HNF_w,HOL_w,HOO_w,HVL_w,IND_w,INT_w,KAY_w,KMS_w,KMT_w,LAY_w,LEH_w,LIN_w,MAP_w,MID_w,MLC_w,MMT_w,MRG_w,MSL_w,MUR_w,MWY_w,NOG_w,NSL_w,OGD_w,OKL_w,ORM_w,PAY_w,PGR_w,PLN_w,PRK_w,PRY_w,PVO_w,PVW_w,ROY_w,RVD_w,RVT_w,SAN_w,SAQ_w,SAR_w,SFK_w,SJC_w,SLC_w,SLM_w,SOG_w,SPV_w,SSL_w,SUN_w,SWE_w,SYR_w,TAY_w,TOO_w,UIN_w,VIN_w,WAT_w,WBG_w,WDL_w,WEB_w,WHT_w,WHV_w,WIL_w,WJC_w,WPT_w,WVC_w,WXC_w,TTL_w,z03_h,z03_w,z11_h,z11_w,z23_h,z23_w,z29_h,z29_w,z35_h,z35_w,z43_h,z43_w,z45_h,z45_w,z49_h,z49_w,z51_h,z51_w,z57_h,z57_w,NAME,zCOFIPS
0,0,AFK,American Fork,13.783877,1.218193,6.693596,2.113599,0.544770,0.530338,1.408451,3.691058e+00,0.618671,2.177971,7.320558,0.430118,0.490782,0.981006,0.874791,0.609526,1.451567,2.095843,5.284967,1.791648,0.429131,4.994556e-01,0.598112,4.704245,3.060369e-01,0.362474,1.731732,1.299778,0.866158,0.350400,1.345840,1.175626,1.601020,7.231478,0.813910,0.596954,0.418693,0.413919,0.828863,0.752218,0.578916,0.909824,0.467546,0.526007,6.933829,6.523905e+00,3.111722,0.628528,0.459513,0.675366,0.516261,0.278380,0.597161,1.343069,0.327597,3.418806e-01,0.372089,0.722311,5.226877,2.216491,9.213036,5.045177e-01,0.288846,0.360857,3.340708,0.332482,3.921829e-01,0.364930,1.601944,0.935542,2.243960e+00,5.833462,2.590364,1.297377,0.443358,2.141178e+00,0.459334,3.107577,0.571153,0.513436,0.496702,3.789971e-01,0.544806,0.855986,0.271713,5.349500,0.585674,0.482125,1.158206,2.366404e-01,0.821339,0.383280,0.668538,0.749599,0.413144,0.513098,0.548552,1.513322,7.581500,0.487457,4.196949,1.279297,0.124643,0.265294,1.666667,2.471182,0.264115,0.946878,4.579197,0.077555,0.282283,0.000000,0.164745,0.568363,0.625523,1.197594,2.205569,1.245706,0.691051,1.538886e-01,0.000000,2.722201,3.501203e-02,0.108217,0.000000,3.333612,8.793877e-02,0.122008,0.000000,0.722369,1.064654,5.891179,0.000000,0.218100,0.520303,0.000000,0.000000,0.0,0.087289,0.105193,0.240585,0.229502,4.183504,4.320092,1.304506e+00,0.458487,0.325060,0.193528,0.000325,0.080735,0.473258,0.995585,0.140364,0.275002,0.157218,0.761488,2.864203,0.626639,5.682859,0.000000,0.182563,0.165785,2.374763,0.102946,0.320645,0.267974,0.494943,0.608292,1.578062,3.414142e+00,1.219887e+00,0.725611,0.324218,1.017961e+00,0.143230,1.474433,0.365985,0.082788,0.155423,2.025571e-01,0.374756,0.312888,0.000000,3.120291,0.097746,0.000000,0.532411,0.152280,7.557115e-01,1.854353e-01,1.048607,0.365221,0.141842,0.324514,0.265748,0.849375,0.518152,0.145144,0.487856,0.198265,2.243960e+00,1.578062,0.516261,0.000325,0.793965,0.447916,0.520398,0.184484,0.858363,0.272550,5.376422,3.397634,1.231786,0.761407,0.387433,0.169521,American Fork,z49
1,1,ALA,Alta,0.012547,1.408404,0.013234,0.009194,0.001994,0.004050,1.858568,0.000000e+00,0.005129,0.000000,0.009054,0.000000,0.003751,0.000000,0.000000,0.247654,0.000000,0.042412,0.002585,0.000003,0.078551,1.248724e-02,0.001923,0.000000,0.000000e+00,0.009036,0.000000,0.000000,0.000167,0.006183,0.057933,0.004885,0.013729,0.010564,0.000000,0.101776,0.000000,0.000000,0.000000,0.000000,0.007579,0.000000,0.015697,0.003011,0.005987,1.143658e-02,0.008345,0.082374,0.066399,0.013164,0.000000,0.000000,0.049073,0.025732,0.002156,4.644604e-03,0.000516,0.000000,0.004264,0.005953,0.008409,0.000000e+00,0.02

In [143]:
dColCodesWCounty

['AFK_h',
 'ALA_h',
 'ALP_h',
 'BDL_h',
 'BGM_h',
 'BNT_h',
 'BRT_h',
 'CDF_h',
 'CEN_h',
 'CHA_h',
 'CHL_h',
 'CLF_h',
 'CLI_h',
 'CMT_h',
 'COA_h',
 'CWH_h',
 'DAN_h',
 'DRA_h',
 'EAG_h',
 'ELK_h',
 'EMT_h',
 'FAR_h',
 'FCS_h',
 'FFD_h',
 'FRR_h',
 'FTH_h',
 'GLA_h',
 'GOS_h',
 'GRL_h',
 'HAR_h',
 'HDT_h',
 'HEB_h',
 'HER_h',
 'HGH_h',
 'HNF_h',
 'HOL_h',
 'HOO_h',
 'HVL_h',
 'IND_h',
 'INT_h',
 'KAY_h',
 'KMS_h',
 'KMT_h',
 'LAY_h',
 'LEH_h',
 'LIN_h',
 'MAP_h',
 'MID_h',
 'MLC_h',
 'MMT_h',
 'MRG_h',
 'MSL_h',
 'MUR_h',
 'MWY_h',
 'NOG_h',
 'NSL_h',
 'OGD_h',
 'OKL_h',
 'ORM_h',
 'PAY_h',
 'PGR_h',
 'PLN_h',
 'PRK_h',
 'PRY_h',
 'PVO_h',
 'PVW_h',
 'ROY_h',
 'RVD_h',
 'RVT_h',
 'SAN_h',
 'SAQ_h',
 'SAR_h',
 'SFK_h',
 'SJC_h',
 'SLC_h',
 'SLM_h',
 'SOG_h',
 'SPV_h',
 'SSL_h',
 'SUN_h',
 'SWE_h',
 'SYR_h',
 'TAY_h',
 'TOO_h',
 'UIN_h',
 'VIN_h',
 'WAT_h',
 'WBG_h',
 'WDL_h',
 'WEB_h',
 'WHT_h',
 'WHV_h',
 'WIL_h',
 'WJC_h',
 'WPT_h',
 'WVC_h',
 'WXC_h',
 'TTL_h',
 'AFK_w',
 'ALA_w',


In [144]:
df_CountySum            = df_CitySum_wCounty_Join           .groupby(['zCOFIPS'])[dColCodesWCounty].apply(lambda x : x.astype(int  ).sum())
df_CountySum_Percent_SA = df_CitySum_wCounty_Join_Percent_SA.groupby(['zCOFIPS'])[dColCodesWCounty].apply(lambda x : x.astype(float).sum())

df_CountySum            = df_CountySum           .reset_index()
df_CountySum_Percent_SA = df_CountySum_Percent_SA.reset_index()

display(df_CountySum)
display(df_CountySum_Percent_SA)

,zCOFIPS,AFK_h,ALA_h,ALP_h,BDL_h,BGM_h,BNT_h,BRT_h,CDF_h,CEN_h,CHA_h,CHL_h,CLF_h,CLI_h,CMT_h,COA_h,CWH_h,DAN_h,DRA_h,EAG_h,ELK_h,EMT_h,FAR_h,FCS_h,FFD_h,FRR_h,FTH_h,GLA_h,GOS_h,GRL_h,HAR_h,HDT_h,HEB_h,HER_h,HGH_h,HNF_h,HOL_h,HOO_h,HVL_h,IND_h,INT_h,KAY_h,KMS_h,KMT_h,LAY_h,LEH_h,LIN_h,MAP_h,MID_h,MLC_h,MMT_h,MRG_h,MSL_h,MUR_h,MWY_h,NOG_h,NSL_h,OGD_h,OKL_h,ORM_h,PAY_h,PGR_h,PLN_h,PRK_h,PRY_h,PVO_h,PVW_h,ROY_h,RVD_h,RVT_h,SAN_h,SAQ_h,SAR_h,SFK_h,SJC_h,SLC_h,SLM_h,SOG_h,SPV_h,SSL_h,SUN_h,SWE_h,SYR_h,TAY_h,TOO_h,UIN_h,VIN_h,WAT_h,WBG_h,WDL_h,WEB_h,WHT_h,WHV_h,WIL_h,WJC_h,WPT_h,WVC_h,WXC_h,TTL_h,AFK_w,ALA_w,ALP_w,BDL_w,BGM_w,BNT_w,BRT_w,CDF_w,CEN_w,CHA_w,CHL_w,CLF_w,CLI_w,CMT_w,COA_w,CWH_w,DAN_w,DRA_w,EAG_w,ELK_w,EMT_w,FAR_w,FCS_w,FFD_w,FRR_w,FTH_w,GLA_w,GOS_w,GRL_w,HAR_w,HDT_w,HEB_w,HER_w,HGH_w,HNF_w,HOL_w,HOO_w,HVL_w,IND_w,INT_w,KAY_w,KMS_w,KMT_w,LAY_w,LEH_w,LIN_w,MAP_w,MID_w,MLC_w,MMT_w,MRG_w,MSL_w,MUR_w,MWY_w,NOG_w,NSL_w,OGD_w,OKL_w,ORM_w,PAY_w,PGR_w,PLN_w,PRK_w,PRY_w,PVO_w,PVW_w,ROY_w,RVD_w,RVT_w,SAN_w,SAQ_w,SAR_w,SFK_w,SJC_w,SLC_w,SLM_w,SOG_w,SPV_w,SSL_w,SUN_w,SWE_w,SYR_w,TAY_w,TOO_w,UIN_w,VIN_w,WAT_w,WBG_w,WDL_w,WEB_w,WHT_w,WHV_w,WIL_w,WJC_w,WPT_w,WVC_w,WXC_w,TTL_w,z03_h,z03_w,z11_h,z11_w,z23_h,z23_w,z29_h,z29_w,z35_h,z35_w,z43_h,z43_w,z45_h,z45_w,z49_h,z49_w,z51_h,z51_w,z57_h,z57_w
0,z01,2,0,0,2,2,3,0,0,0,0,1,1,1,0,0,3,1,4,1,0,0,2,0,0,0,0,0,0,1,0,0,1,3,1,0,2,0,0,0,0,1,0,3,5,3,0,1,3,7,3,0,0,2,0,1,3,2,1,4,1,1,0,0,1,22,0,2,1,0,13,3,1,4,2,16,1,0,2,3,1,0,1,8,4,0,0,1,0,1,0,0,2,1,9,0,11,0,2602,5,1,3,1,0,2,0,0,1,0,0,3,0,0,0,3,0,5,0,0,0,5,0,0,1,1,0,0,1,0,0,1,4,1,0,2,0,0,0,0,4,1,1,1,4,4,1,6,13,0,0,0,20,0,1,2,8,1,15,9,2,0,5,0,34,1,2,3,2,13,1,0,4,2,87,0,1,1,15,0,0,1,4,1,0,0,0,0,0,1,0,1,0,10,0,28,2,2290,4,0,21,23,3,1,0,0,112,228,1,7,5,2,58,93,2,2,12,19
1,z03,21,0,2,3,3892,46,0,0,10,0,10,129,62,0,2,21,0,28,9,1,1,10,0,0,83,12,4,1,27,117,2,25,11,11,2,28,58,3,0,0,50,4,29,221,32,5,5,29,34,19,20,19,40,5,339,20,976,1,68,10,23,78,14,883,61,164,302,78,28,73,5,14,23,36,169,2,99,35,17,19,12,93,48,87,3,0,74,0,1,4,6,99,242,70,34,104,15,19043,122,1,11,15,5106,109,0,0,58,1,6,501,52,0,2,77,2,223,14,1,3,81,0,0,196,2,3,0,8,79,0,49,11,18,0,40,9,1,0,0,110,5,19,562,114,42,4,99,190,15,14,99,314,4,146,172,2743,0,275,28,40,44,26,546,256,132,260,213,59,295,2,12,66,133,1555,3,171,65,185,20,12,45,143,59,3,3,104,1,0,8,5,123,80,243,23,597,49,26452,5040,5757,832,1924,5,2,20,14,878,4416,27,37,119,70,397,1189,34,61,2610,4466
2,z05,74,0,32,21,727,226,1,2,88,7,25,176,101,0,16,70,1,109,74,7,6,99,5,0,46,27,3,6,67,38,4,125,54,54,6,63,46,6,2,1,149,9,74,350,126,33,21,72,127,65,38,15,116,33,119,87,700,9,282,85,101,54,58,157,260,64,268,63,103,230,30,55,100,132,428,22,116,93,52,32,30,106,151,264,11,0,55,1,2,32,13,59,47,248,47,240,57,50124,310,3,27,26,969,208,1,0,112,2,14,328,95,1,3,155,4,406,21,7,0,123,1,0,67,4,2,0,18,96,0,155,34,21,1,74,10,1,0,0,117,4,53,756,322,103,12,214,390,42,26,37,688,14,96,246,1584,1,949,46,94,19,121,60,1063,109,209,301,154,557,12,40,116,263,2812,10,175,75,369,15,19,67,370,149,6,35,67,0,1,38,11,68,12,412,19,860,66,59032,935,1048,1627,2295,30,12,38,26,2406,8030,104,132,331,169,1473,3349,175,177,1681,2924
3,z07,13,1,7,1,11,5,0,0,4,0,6,6,5,0,6,8,1,14,19,1,0,3,3,0,2,0,2,1,3,1,0,23,5,15,3,14,2,0,0,1,3,2,14,17,47,4,3,11,27,6,1,1,17,6,4,7,24,2,45,19,14,2,8,2,70,3,11,3,12,25,10,21,32,11,69,10,5,35,6,0,0,5,11,18,0,1,5,0,0,1,2,1,1,44,3,37,2,7702,93,1,16,7,13,10,0,1,3,1,6,12,4,0,0,31,0,141,7,1,0,3,0,0,3,0,2,0,2,0,1,36,12,7,0,18,0,0,0,0,8,4,2,62,68,41,19,55,64,2,0,8,93,2,5,28,110,0,161,44,15,0,12,1,273,0,8,18,25,96,11,9,55,65,569,5,5,44,43,4,1,7,108,23,0,6,0,0,0,2,1,14,1,63,3,180,5,9531,14,15,70,175,10,11,1,0,364,1636,25,16,21,26,403,919,31,44,68,188
4,z09,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,0,1,0,214,1,0,0,0,0,1,0,0,0,0,0,2,0,0,0,1,0,2,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,2,0,0,0,0,0,0,1,3,4,0,1,

,zCOFIPS,AFK_h,ALA_h,ALP_h,BDL_h,BGM_h,BNT_h,BRT_h,CDF_h,CEN_h,CHA_h,CHL_h,CLF_h,CLI_h,CMT_h,COA_h,CWH_h,DAN_h,DRA_h,EAG_h,ELK_h,EMT_h,FAR_h,FCS_h,FFD_h,FRR_h,FTH_h,GLA_h,GOS_h,GRL_h,HAR_h,HDT_h,HEB_h,HER_h,HGH_h,HNF_h,HOL_h,HOO_h,HVL_h,IND_h,INT_h,KAY_h,KMS_h,KMT_h,LAY_h,LEH_h,LIN_h,MAP_h,MID_h,MLC_h,MMT_h,MRG_h,MSL_h,MUR_h,MWY_h,NOG_h,NSL_h,OGD_h,OKL_h,ORM_h,PAY_h,PGR_h,PLN_h,PRK_h,PRY_h,PVO_h,PVW_h,ROY_h,RVD_h,RVT_h,SAN_h,SAQ_h,SAR_h,SFK_h,SJC_h,SLC_h,SLM_h,SOG_h,SPV_h,SSL_h,SUN_h,SWE_h,SYR_h,TAY_h,TOO_h,UIN_h,VIN_h,WAT_h,WBG_h,WDL_h,WEB_h,WHT_h,WHV_h,WIL_h,WJC_h,WPT_h,WVC_h,WXC_h,TTL_h,AFK_w,ALA_w,ALP_w,BDL_w,BGM_w,BNT_w,BRT_w,CDF_w,CEN_w,CHA_w,CHL_w,CLF_w,CLI_w,CMT_w,COA_w,CWH_w,DAN_w,DRA_w,EAG_w,ELK_w,EMT_w,FAR_w,FCS_w,FFD_w,FRR_w,FTH_w,GLA_w,GOS_w,GRL_w,HAR_w,HDT_w,HEB_w,HER_w,HGH_w,HNF_w,HOL_w,HOO_w,HVL_w,IND_w,INT_w,KAY_w,KMS_w,KMT_w,LAY_w,LEH_w,LIN_w,MAP_w,MID_w,MLC_w,MMT_w,MRG_w,MSL_w,MUR_w,MWY_w,NOG_w,NSL_w,OGD_w,OKL_w,ORM_w,PAY_w,PGR_w,PLN_w,PRK_w,PRY_w,PVO_w,PVW_w,ROY_w,RVD_w,RVT_w,SAN_w,SAQ_w,SAR_w,SFK_w,SJC_w,SLC_w,SLM_w,SOG_w,SPV_w,SSL_w,SUN_w,SWE_w,SYR_w,TAY_w,TOO_w,UIN_w,VIN_w,WAT_w,WBG_w,WDL_w,WEB_w,WHT_w,WHV_w,WIL_w,WJC_w,WPT_w,WVC_w,WXC_w,TTL_w,z03_h,z03_w,z11_h,z11_w,z23_h,z23_w,z29_h,z29_w,z35_h,z35_w,z43_h,z43_w,z45_h,z45_w,z49_h,z49_w,z51_h,z51_w,z57_h,z57_w
0,z01,2.442373e-02,0.000000,0.000000e+00,3.085467e-02,3.055279e-02,1.518295e-02,0.000000,0.000000e+00,1.311616e-08,0.000000e+00,4.524812e-02,0.015150,1.001403e-02,0.000000,0.000000,1.822275e-02,0.184162,2.270563e-02,0.013803,6.982660e-07,0.000000,2.172975e-02,0.000001,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,1.955417e-02,0.000000e+00,0.000000e+00,1.247039e-02,0.029642,1.292347e-02,0.000000e+00,0.014031,1.905944e-07,0.000000,0.000000,0.000000,7.878358e-03,4.623317e-09,2.137637e-02,1.894890e-02,0.013854,0.000000e+00,4.355342e-02,2.986256e-02,3.034697e-02,0.034669,5.399276e-02,0.000000e+00,1.199171e-02,0.000000e+00,1.101079e-02,3.636804e-02,7.578358e-03,1.766784e-01,1.127074e-02,2.056323e-02,6.289716e-03,3.278531e-02,1.577148e-07,3.798006e-02,6.063427e-02,0.000000e+00,1.606061e-02,2.373042e-02,4.247806e-03,0.029420,5.366773e-02,0.014583,3.038374e-02,9.327853e-03,2.031712e-02,0.048768,2.441685e-08,1.811820e-02,3.455677e-02,3.840246e-02,0.000000e+00,1.648896e-02,2.947416e-02,2.981677e-02,0.000000,0.000000,2.220742e-02,0.000000e+00,0.120192,1.395412e-09,0.000000e+00,0.056538,9.842520e-02,0.017181,2.132538e-02,1.905342e-02,1.893404e-08,0.187315,0.031210,1.355896e-01,0.143626,2.271271e-02,5.574802e-07,1.748442e-02,0.000000,0.000000,1.773050e-02,0.529083,0.000000e+00,1.808468e-02,0.000000e+00,0.000000,0.000000e+00,0.014734,0.000000e+00,0.013270,0.000000,0.000000,0.000000,5.465078e-02,0.000000,0.000000,3.244646e-02,0.224215,0.000000,0.000000e+00,0.059313,0.000000,0.000000,0.051510,8.984726e-02,3.302560e-02,0.000000,2.965267e-02,0.000000e+00,0.000000,0.000000,0.0,0.049789,0.086730,2.326664e-02,0.003709,0.015059,0.031957,0.052060,0.032750,0.028823,3.651181e-07,7.974051e-02,5.491192e-02,0.047285,0.000000,3.581662e-02,0.013517,0.015483,3.968254e-01,0.033600,0.176112,0.018718,0.000000e+00,5.136972e-02,0.000000,0.061628,0.049677,3.947851e-02,3.866536e-02,1.960823e-02,0.029204,0.093639,4.283188e-07,0.039302,6.690525e-03,0.037343,2.754775e-06,0.014719,0.023905,0.049941,0.000000,0.000000e+00,2.657483e-02,0.019268,2.591973e-02,0.000000,0.000003,9.918460e-07,0.000000,0.000015,4.610470e-02,0.000000e+00,2.956859e-02,0.000000,0.033542,0.000000e+00,0.043842,4.207912e-02,0.162406,3.712499e-02,4.971687e-07,0.015182,0.021074,5.366773e-02,0.093639,5.399276e-02,7.974051e-02,0.021103,0.033669,1.445522e-02,0.057979,2.741843e-02,3.190797e-02,0.024024,0.040605,1.704014e-02,0.053777,0.012047,0.020960
1,z03,2.116763e-01,0.000000,6.758279e-02,6.170935e-02,3.977983e+01,2.783564e-01,0.000000,0.000000e+00,2.327767e-01,0.000000e+00,2.941129e-01,1.060566,7.211079e-01,0.000000,0.527080,1.518605e-01,0.000002,1.498563e-01,0.089717,6.435006e-02,0.135687,1.629940e-01,0.000000,0.000

In [145]:
df_CountySum           ['WFCounty'] = "Outside Wasatch Front"
df_CountySum_Percent_SA['WFCounty'] = "Outside Wasatch Front"

df_CountySum           .loc[df_CountySum.zCOFIPS == 'z11', 'WFCounty'] = "Davis County"
df_CountySum           .loc[df_CountySum.zCOFIPS == 'z57', 'WFCounty'] = "Weber County"
df_CountySum           .loc[df_CountySum.zCOFIPS == 'z35', 'WFCounty'] = "Salt Lake County"
df_CountySum           .loc[df_CountySum.zCOFIPS == 'z49', 'WFCounty'] = "Utah County"

df_CountySum_Percent_SA.loc[df_CountySum.zCOFIPS == 'z11', 'WFCounty'] = "Davis County"
df_CountySum_Percent_SA.loc[df_CountySum.zCOFIPS == 'z57', 'WFCounty'] = "Weber County"
df_CountySum_Percent_SA.loc[df_CountySum.zCOFIPS == 'z35', 'WFCounty'] = "Salt Lake County"
df_CountySum_Percent_SA.loc[df_CountySum.zCOFIPS == 'z49', 'WFCounty'] = "Utah County"

display(df_CountySum)
display(df_CountySum_Percent_SA)

,zCOFIPS,AFK_h,ALA_h,ALP_h,BDL_h,BGM_h,BNT_h,BRT_h,CDF_h,CEN_h,CHA_h,CHL_h,CLF_h,CLI_h,CMT_h,COA_h,CWH_h,DAN_h,DRA_h,EAG_h,ELK_h,EMT_h,FAR_h,FCS_h,FFD_h,FRR_h,FTH_h,GLA_h,GOS_h,GRL_h,HAR_h,HDT_h,HEB_h,HER_h,HGH_h,HNF_h,HOL_h,HOO_h,HVL_h,IND_h,INT_h,KAY_h,KMS_h,KMT_h,LAY_h,LEH_h,LIN_h,MAP_h,MID_h,MLC_h,MMT_h,MRG_h,MSL_h,MUR_h,MWY_h,NOG_h,NSL_h,OGD_h,OKL_h,ORM_h,PAY_h,PGR_h,PLN_h,PRK_h,PRY_h,PVO_h,PVW_h,ROY_h,RVD_h,RVT_h,SAN_h,SAQ_h,SAR_h,SFK_h,SJC_h,SLC_h,SLM_h,SOG_h,SPV_h,SSL_h,SUN_h,SWE_h,SYR_h,TAY_h,TOO_h,UIN_h,VIN_h,WAT_h,WBG_h,WDL_h,WEB_h,WHT_h,WHV_h,WIL_h,WJC_h,WPT_h,WVC_h,WXC_h,TTL_h,AFK_w,ALA_w,ALP_w,BDL_w,BGM_w,BNT_w,BRT_w,CDF_w,CEN_w,CHA_w,CHL_w,CLF_w,CLI_w,CMT_w,COA_w,CWH_w,DAN_w,DRA_w,EAG_w,ELK_w,EMT_w,FAR_w,FCS_w,FFD_w,FRR_w,FTH_w,GLA_w,GOS_w,GRL_w,HAR_w,HDT_w,HEB_w,HER_w,HGH_w,HNF_w,HOL_w,HOO_w,HVL_w,IND_w,INT_w,KAY_w,KMS_w,KMT_w,LAY_w,LEH_w,LIN_w,MAP_w,MID_w,MLC_w,MMT_w,MRG_w,MSL_w,MUR_w,MWY_w,NOG_w,NSL_w,OGD_w,OKL_w,ORM_w,PAY_w,PGR_w,PLN_w,PRK_w,PRY_w,PVO_w,PVW_w,ROY_w,RVD_w,RVT_w,SAN_w,SAQ_w,SAR_w,SFK_w,SJC_w,SLC_w,SLM_w,SOG_w,SPV_w,SSL_w,SUN_w,SWE_w,SYR_w,TAY_w,TOO_w,UIN_w,VIN_w,WAT_w,WBG_w,WDL_w,WEB_w,WHT_w,WHV_w,WIL_w,WJC_w,WPT_w,WVC_w,WXC_w,TTL_w,z03_h,z03_w,z11_h,z11_w,z23_h,z23_w,z29_h,z29_w,z35_h,z35_w,z43_h,z43_w,z45_h,z45_w,z49_h,z49_w,z51_h,z51_w,z57_h,z57_w,WFCounty
0,z01,2,0,0,2,2,3,0,0,0,0,1,1,1,0,0,3,1,4,1,0,0,2,0,0,0,0,0,0,1,0,0,1,3,1,0,2,0,0,0,0,1,0,3,5,3,0,1,3,7,3,0,0,2,0,1,3,2,1,4,1,1,0,0,1,22,0,2,1,0,13,3,1,4,2,16,1,0,2,3,1,0,1,8,4,0,0,1,0,1,0,0,2,1,9,0,11,0,2602,5,1,3,1,0,2,0,0,1,0,0,3,0,0,0,3,0,5,0,0,0,5,0,0,1,1,0,0,1,0,0,1,4,1,0,2,0,0,0,0,4,1,1,1,4,4,1,6,13,0,0,0,20,0,1,2,8,1,15,9,2,0,5,0,34,1,2,3,2,13,1,0,4,2,87,0,1,1,15,0,0,1,4,1,0,0,0,0,0,1,0,1,0,10,0,28,2,2290,4,0,21,23,3,1,0,0,112,228,1,7,5,2,58,93,2,2,12,19,Outside Wasatch Front
1,z03,21,0,2,3,3892,46,0,0,10,0,10,129,62,0,2,21,0,28,9,1,1,10,0,0,83,12,4,1,27,117,2,25,11,11,2,28,58,3,0,0,50,4,29,221,32,5,5,29,34,19,20,19,40,5,339,20,976,1,68,10,23,78,14,883,61,164,302,78,28,73,5,14,23,36,169,2,99,35,17,19,12,93,48,87,3,0,74,0,1,4,6,99,242,70,34,104,15,19043,122,1,11,15,5106,109,0,0,58,1,6,501,52,0,2,77,2,223,14,1,3,81,0,0,196,2,3,0,8,79,0,49,11,18,0,40,9,1,0,0,110,5,19,562,114,42,4,99,190,15,14,99,314,4,146,172,2743,0,275,28,40,44,26,546,256,132,260,213,59,295,2,12,66,133,1555,3,171,65,185,20,12,45,143,59,3,3,104,1,0,8,5,123,80,243,23,597,49,26452,5040,5757,832,1924,5,2,20,14,878,4416,27,37,119,70,397,1189,34,61,2610,4466,Outside Wasatch Front
2,z05,74,0,32,21,727,226,1,2,88,7,25,176,101,0,16,70,1,109,74,7,6,99,5,0,46,27,3,6,67,38,4,125,54,54,6,63,46,6,2,1,149,9,74,350,126,33,21,72,127,65,38,15,116,33,119,87,700,9,282,85,101,54,58,157,260,64,268,63,103,230,30,55,100,132,428,22,116,93,52,32,30,106,151,264,11,0,55,1,2,32,13,59,47,248,47,240,57,50124,310,3,27,26,969,208,1,0,112,2,14,328,95,1,3,155,4,406,21,7,0,123,1,0,67,4,2,0,18,96,0,155,34,21,1,74,10,1,0,0,117,4,53,756,322,103,12,214,390,42,26,37,688,14,96,246,1584,1,949,46,94,19,121,60,1063,109,209,301,154,557,12,40,116,263,2812,10,175,75,369,15,19,67,370,149,6,35,67,0,1,38,11,68,12,412,19,860,66,59032,935,1048,1627,2295,30,12,38,26,2406,8030,104,132,331,169,1473,3349,175,177,1681,2924,Outside Wasatch Front
3,z07,13,1,7,1,11,5,0,0,4,0,6,6,5,0,6,8,1,14,19,1,0,3,3,0,2,0,2,1,3,1,0,23,5,15,3,14,2,0,0,1,3,2,14,17,47,4,3,11,27,6,1,1,17,6,4,7,24,2,45,19,14,2,8,2,70,3,11,3,12,25,10,21,32,11,69,10,5,35,6,0,0,5,11,18,0,1,5,0,0,1,2,1,1,44,3,37,2,7702,93,1,16,7,13,10,0,1,3,1,6,12,4,0,0,31,0,141,7,1,0,3,0,0,3,0,2,0,2,0,1,36,12,7,0,18,0,0,0,0,8,4,2,62,68,41,19,55,64,2,0,8,93,2,5,28,110,0,161,44,15,0,12,1,273,0,8,18,25,96,11,9,55,65,569,5,5,44,43,4,1,7,108,23,0,6,0,0,0,2,1,14,1,63,3,180,5,9531,14,15,70,175,10,11,1,0,364,1636,25,16,21,26,403,919,31,44,68,188,Outside Wasatch Front
4,z09,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,3,0,1,0,214,1,0,0,0,0,1,0,0,0,0,0

,zCOFIPS,AFK_h,ALA_h,ALP_h,BDL_h,BGM_h,BNT_h,BRT_h,CDF_h,CEN_h,CHA_h,CHL_h,CLF_h,CLI_h,CMT_h,COA_h,CWH_h,DAN_h,DRA_h,EAG_h,ELK_h,EMT_h,FAR_h,FCS_h,FFD_h,FRR_h,FTH_h,GLA_h,GOS_h,GRL_h,HAR_h,HDT_h,HEB_h,HER_h,HGH_h,HNF_h,HOL_h,HOO_h,HVL_h,IND_h,INT_h,KAY_h,KMS_h,KMT_h,LAY_h,LEH_h,LIN_h,MAP_h,MID_h,MLC_h,MMT_h,MRG_h,MSL_h,MUR_h,MWY_h,NOG_h,NSL_h,OGD_h,OKL_h,ORM_h,PAY_h,PGR_h,PLN_h,PRK_h,PRY_h,PVO_h,PVW_h,ROY_h,RVD_h,RVT_h,SAN_h,SAQ_h,SAR_h,SFK_h,SJC_h,SLC_h,SLM_h,SOG_h,SPV_h,SSL_h,SUN_h,SWE_h,SYR_h,TAY_h,TOO_h,UIN_h,VIN_h,WAT_h,WBG_h,WDL_h,WEB_h,WHT_h,WHV_h,WIL_h,WJC_h,WPT_h,WVC_h,WXC_h,TTL_h,AFK_w,ALA_w,ALP_w,BDL_w,BGM_w,BNT_w,BRT_w,CDF_w,CEN_w,CHA_w,CHL_w,CLF_w,CLI_w,CMT_w,COA_w,CWH_w,DAN_w,DRA_w,EAG_w,ELK_w,EMT_w,FAR_w,FCS_w,FFD_w,FRR_w,FTH_w,GLA_w,GOS_w,GRL_w,HAR_w,HDT_w,HEB_w,HER_w,HGH_w,HNF_w,HOL_w,HOO_w,HVL_w,IND_w,INT_w,KAY_w,KMS_w,KMT_w,LAY_w,LEH_w,LIN_w,MAP_w,MID_w,MLC_w,MMT_w,MRG_w,MSL_w,MUR_w,MWY_w,NOG_w,NSL_w,OGD_w,OKL_w,ORM_w,PAY_w,PGR_w,PLN_w,PRK_w,PRY_w,PVO_w,PVW_w,ROY_w,RVD_w,RVT_w,SAN_w,SAQ_w,SAR_w,SFK_w,SJC_w,SLC_w,SLM_w,SOG_w,SPV_w,SSL_w,SUN_w,SWE_w,SYR_w,TAY_w,TOO_w,UIN_w,VIN_w,WAT_w,WBG_w,WDL_w,WEB_w,WHT_w,WHV_w,WIL_w,WJC_w,WPT_w,WVC_w,WXC_w,TTL_w,z03_h,z03_w,z11_h,z11_w,z23_h,z23_w,z29_h,z29_w,z35_h,z35_w,z43_h,z43_w,z45_h,z45_w,z49_h,z49_w,z51_h,z51_w,z57_h,z57_w,WFCounty
0,z01,2.442373e-02,0.000000,0.000000e+00,3.085467e-02,3.055279e-02,1.518295e-02,0.000000,0.000000e+00,1.311616e-08,0.000000e+00,4.524812e-02,0.015150,1.001403e-02,0.000000,0.000000,1.822275e-02,0.184162,2.270563e-02,0.013803,6.982660e-07,0.000000,2.172975e-02,0.000001,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,1.955417e-02,0.000000e+00,0.000000e+00,1.247039e-02,0.029642,1.292347e-02,0.000000e+00,0.014031,1.905944e-07,0.000000,0.000000,0.000000,7.878358e-03,4.623317e-09,2.137637e-02,1.894890e-02,0.013854,0.000000e+00,4.355342e-02,2.986256e-02,3.034697e-02,0.034669,5.399276e-02,0.000000e+00,1.199171e-02,0.000000e+00,1.101079e-02,3.636804e-02,7.578358e-03,1.766784e-01,1.127074e-02,2.056323e-02,6.289716e-03,3.278531e-02,1.577148e-07,3.798006e-02,6.063427e-02,0.000000e+00,1.606061e-02,2.373042e-02,4.247806e-03,0.029420,5.366773e-02,0.014583,3.038374e-02,9.327853e-03,2.031712e-02,0.048768,2.441685e-08,1.811820e-02,3.455677e-02,3.840246e-02,0.000000e+00,1.648896e-02,2.947416e-02,2.981677e-02,0.000000,0.000000,2.220742e-02,0.000000e+00,0.120192,1.395412e-09,0.000000e+00,0.056538,9.842520e-02,0.017181,2.132538e-02,1.905342e-02,1.893404e-08,0.187315,0.031210,1.355896e-01,0.143626,2.271271e-02,5.574802e-07,1.748442e-02,0.000000,0.000000,1.773050e-02,0.529083,0.000000e+00,1.808468e-02,0.000000e+00,0.000000,0.000000e+00,0.014734,0.000000e+00,0.013270,0.000000,0.000000,0.000000,5.465078e-02,0.000000,0.000000,3.244646e-02,0.224215,0.000000,0.000000e+00,0.059313,0.000000,0.000000,0.051510,8.984726e-02,3.302560e-02,0.000000,2.965267e-02,0.000000e+00,0.000000,0.000000,0.0,0.049789,0.086730,2.326664e-02,0.003709,0.015059,0.031957,0.052060,0.032750,0.028823,3.651181e-07,7.974051e-02,5.491192e-02,0.047285,0.000000,3.581662e-02,0.013517,0.015483,3.968254e-01,0.033600,0.176112,0.018718,0.000000e+00,5.136972e-02,0.000000,0.061628,0.049677,3.947851e-02,3.866536e-02,1.960823e-02,0.029204,0.093639,4.283188e-07,0.039302,6.690525e-03,0.037343,2.754775e-06,0.014719,0.023905,0.049941,0.000000,0.000000e+00,2.657483e-02,0.019268,2.591973e-02,0.000000,0.000003,9.918460e-07,0.000000,0.000015,4.610470e-02,0.000000e+00,2.956859e-02,0.000000,0.033542,0.000000e+00,0.043842,4.207912e-02,0.162406,3.712499e-02,4.971687e-07,0.015182,0.021074,5.366773e-02,0.093639,5.399276e-02,7.974051e-02,0.021103,0.033669,1.445522e-02,0.057979,2.741843e-02,3.190797e-02,0.024024,0.040605,1.704014e-02,0.053777,0.012047,0.020960,Outside Wasatch Front
1,z03,2.116763e-01,0.000000,6.758279e-02,6.170935e-02,3.977983e+01,2.783564e-01,0.000000,0.000000e+00,2.327767e-01,0.000000e+00,2.941129e-01,1.060566,7.211079e-01,0.000000,0.527080,1.518605e-01,0.000002,1.498563e-01,0.089717,6.435006e-02,0.135

In [146]:
df_CountySum_wOutWF            = df_CountySum           .groupby(['WFCounty'])[dColCodesWCounty].apply(lambda x : x.astype(int  ).sum())
df_CountySum_wOutWF_Percent_SA = df_CountySum_Percent_SA.groupby(['WFCounty'])[dColCodesWCounty].apply(lambda x : x.astype(float).sum())

df_CountySum_wOutWF            = df_CountySum_wOutWF.reset_index()
df_CountySum_wOutWF_Percent_SA = df_CountySum_wOutWF_Percent_SA.reset_index()

display(df_CountySum_wOutWF)
display(df_CountySum_wOutWF_Percent_SA)

,WFCounty,AFK_h,ALA_h,ALP_h,BDL_h,BGM_h,BNT_h,BRT_h,CDF_h,CEN_h,CHA_h,CHL_h,CLF_h,CLI_h,CMT_h,COA_h,CWH_h,DAN_h,DRA_h,EAG_h,ELK_h,EMT_h,FAR_h,FCS_h,FFD_h,FRR_h,FTH_h,GLA_h,GOS_h,GRL_h,HAR_h,HDT_h,HEB_h,HER_h,HGH_h,HNF_h,HOL_h,HOO_h,HVL_h,IND_h,INT_h,KAY_h,KMS_h,KMT_h,LAY_h,LEH_h,LIN_h,MAP_h,MID_h,MLC_h,MMT_h,MRG_h,MSL_h,MUR_h,MWY_h,NOG_h,NSL_h,OGD_h,OKL_h,ORM_h,PAY_h,PGR_h,PLN_h,PRK_h,PRY_h,PVO_h,PVW_h,ROY_h,RVD_h,RVT_h,SAN_h,SAQ_h,SAR_h,SFK_h,SJC_h,SLC_h,SLM_h,SOG_h,SPV_h,SSL_h,SUN_h,SWE_h,SYR_h,TAY_h,TOO_h,UIN_h,VIN_h,WAT_h,WBG_h,WDL_h,WEB_h,WHT_h,WHV_h,WIL_h,WJC_h,WPT_h,WVC_h,WXC_h,TTL_h,AFK_w,ALA_w,ALP_w,BDL_w,BGM_w,BNT_w,BRT_w,CDF_w,CEN_w,CHA_w,CHL_w,CLF_w,CLI_w,CMT_w,COA_w,CWH_w,DAN_w,DRA_w,EAG_w,ELK_w,EMT_w,FAR_w,FCS_w,FFD_w,FRR_w,FTH_w,GLA_w,GOS_w,GRL_w,HAR_w,HDT_w,HEB_w,HER_w,HGH_w,HNF_w,HOL_w,HOO_w,HVL_w,IND_w,INT_w,KAY_w,KMS_w,KMT_w,LAY_w,LEH_w,LIN_w,MAP_w,MID_w,MLC_w,MMT_w,MRG_w,MSL_w,MUR_w,MWY_w,NOG_w,NSL_w,OGD_w,OKL_w,ORM_w,PAY_w,PGR_w,PLN_w,PRK_w,PRY_w,PVO_w,PVW_w,ROY_w,RVD_w,RVT_w,SAN_w,SAQ_w,SAR_w,SFK_w,SJC_w,SLC_w,SLM_w,SOG_w,SPV_w,SSL_w,SUN_w,SWE_w,SYR_w,TAY_w,TOO_w,UIN_w,VIN_w,WAT_w,WBG_w,WDL_w,WEB_w,WHT_w,WHV_w,WIL_w,WJC_w,WPT_w,WVC_w,WXC_w,TTL_w,z03_h,z03_w,z11_h,z11_w,z23_h,z23_w,z29_h,z29_w,z35_h,z35_w,z43_h,z43_w,z45_h,z45_w,z49_h,z49_w,z51_h,z51_w,z57_h,z57_w
0,Davis County,184,2,66,114,710,6191,0,2,2536,0,67,5291,3530,10,24,415,8,592,245,22,12,3429,4,0,368,794,9,6,226,385,7,226,400,122,17,475,798,18,0,0,5223,11,615,12961,470,73,60,334,996,489,233,88,733,36,1035,2034,6985,9,747,161,270,354,72,181,686,517,4704,834,648,1260,78,185,329,856,4421,61,1477,261,442,853,1003,5056,930,790,93,1,872,3,2,1027,58,1060,83,1640,1806,2578,1543,111761,648,14,38,75,358,6356,0,1,2826,2,9,7625,993,3,14,1352,0,1538,16,3,0,5490,0,2,383,159,1,0,17,160,0,114,107,42,1,513,56,2,0,0,4950,4,104,13164,732,159,15,948,4048,113,108,203,2978,11,279,4533,11288,7,1285,54,139,66,181,18,1175,172,1880,1930,238,2261,5,39,107,1118,30791,10,1501,103,3015,467,206,1968,1187,232,29,17,639,0,0,893,7,576,6,1300,504,6105,2172,148169,1048,408,54691,53893,78,5,233,108,19349,59709,181,231,1054,268,4726,5541,322,143,20574,20260
1,Outside Wasatch Front,435,6,205,110,4827,664,3,13,259,102,165,552,330,6,348,652,269,765,570,64,39,283,234,3,164,59,59,43,1681,190,168,4716,390,350,164,626,167,12,48,17,433,531,566,1214,1180,174,164,460,1366,445,560,41,813,1067,619,267,2459,316,1602,601,518,173,2500,1091,1738,300,894,206,646,1663,652,479,919,834,3625,205,342,790,471,90,94,403,936,5206,19,1,212,63,37,74,94,248,309,1747,150,1813,144,269158,1851,34,555,213,6294,749,3,7,386,110,55,1295,226,3,390,1195,77,2583,136,39,6,526,34,8,337,20,25,5,1239,220,3,3918,179,138,21,536,30,3,6,0,520,813,244,2688,1958,869,151,1326,3179,211,771,199,3870,757,321,1395,6823,127,4350,984,667,73,7342,614,5742,288,743,915,734,3296,331,213,1312,1847,21341,138,727,1290,2533,67,52,232,1659,5058,18,85,277,16,8,143,24,342,98,2445,64,6515,336,326694,6271,7054,5679,9785,652,331,560,771,19601,56339,4237,8864,6936,6341,11548,22202,6631,4996,6546,12110
2,Salt Lake County,2933,92,1393,4887,1465,10540,40,95,3473,50,1276,3582,2427,300,178,13704,92,16359,6541,237,513,4288,75,20,438,800,137,69,2277,424,38,1579,13532,2871,55,11681,618,37,19,7,5054,162,15793,9803,11478,935,765,14235,24877,12107,360,102,21260,522,1465,4869,7176,85,8951,1912,3438,415,762,390,6608,619,3535,732,18716,39454,1088,5992,3875,26213,74606,791,1512,3319,9475,559,745,3326,26104,8287,109,10,757,32,160,1558,2296,878,154,49621,1127,54116,2611,665638,4441,1083,351,2600,279,2264,31,7,1041,9,95,1087,157,28,54,14366,13,24541,275,16,74,1175,3,26,111,47,5,0,167,59,3,385,2999,455,1,7444,27,4,1,0,576,66,3165,2906,8817,1718,67,12940,36092,2049,32,88,30499,79,98,5453,3998,26,7336,321,1325,19,2823,18,6710,61,349,763,6964,34283,38,590,655,21287,156359,71,470,944,23720,57,42,248,14893,1182,10,80,74,4,9,458,183,125,3,26860,51,47034,1118,563922,2131,307,57121,19029,1088,38,360,32,455558,475304,1491,3079,10730,1433,67150,37345,2552,570,20147,7281
3,Utah C

,WFCounty,AFK_h,ALA_h,ALP_h,BDL_h,BGM_h,BNT_h,BRT_h,CDF_h,CEN_h,CHA_h,CHL_h,CLF_h,CLI_h,CMT_h,COA_h,CWH_h,DAN_h,DRA_h,EAG_h,ELK_h,EMT_h,FAR_h,FCS_h,FFD_h,FRR_h,FTH_h,GLA_h,GOS_h,GRL_h,HAR_h,HDT_h,HEB_h,HER_h,HGH_h,HNF_h,HOL_h,HOO_h,HVL_h,IND_h,INT_h,KAY_h,KMS_h,KMT_h,LAY_h,LEH_h,LIN_h,MAP_h,MID_h,MLC_h,MMT_h,MRG_h,MSL_h,MUR_h,MWY_h,NOG_h,NSL_h,OGD_h,OKL_h,ORM_h,PAY_h,PGR_h,PLN_h,PRK_h,PRY_h,PVO_h,PVW_h,ROY_h,RVD_h,RVT_h,SAN_h,SAQ_h,SAR_h,SFK_h,SJC_h,SLC_h,SLM_h,SOG_h,SPV_h,SSL_h,SUN_h,SWE_h,SYR_h,TAY_h,TOO_h,UIN_h,VIN_h,WAT_h,WBG_h,WDL_h,WEB_h,WHT_h,WHV_h,WIL_h,WJC_h,WPT_h,WVC_h,WXC_h,TTL_h,AFK_w,ALA_w,ALP_w,BDL_w,BGM_w,BNT_w,BRT_w,CDF_w,CEN_w,CHA_w,CHL_w,CLF_w,CLI_w,CMT_w,COA_w,CWH_w,DAN_w,DRA_w,EAG_w,ELK_w,EMT_w,FAR_w,FCS_w,FFD_w,FRR_w,FTH_w,GLA_w,GOS_w,GRL_w,HAR_w,HDT_w,HEB_w,HER_w,HGH_w,HNF_w,HOL_w,HOO_w,HVL_w,IND_w,INT_w,KAY_w,KMS_w,KMT_w,LAY_w,LEH_w,LIN_w,MAP_w,MID_w,MLC_w,MMT_w,MRG_w,MSL_w,MUR_w,MWY_w,NOG_w,NSL_w,OGD_w,OKL_w,ORM_w,PAY_w,PGR_w,PLN_w,PRK_w,PRY_w,PVO_w,PVW_w,ROY_w,RVD_w,RVT_w,SAN_w,SAQ_w,SAR_w,SFK_w,SJC_w,SLC_w,SLM_w,SOG_w,SPV_w,SSL_w,SUN_w,SWE_w,SYR_w,TAY_w,TOO_w,UIN_w,VIN_w,WAT_w,WBG_w,WDL_w,WEB_w,WHT_w,WHV_w,WIL_w,WJC_w,WPT_w,WVC_w,WXC_w,TTL_w,z03_h,z03_w,z11_h,z11_w,z23_h,z23_w,z29_h,z29_w,z35_h,z35_w,z43_h,z43_w,z45_h,z45_w,z49_h,z49_w,z51_h,z51_w,z57_h,z57_w
0,Davis County,1.913190,2.469136,2.140114,2.360358,8.055957,31.928100,2.816734,0.931798,36.039108,0.000000,2.262409,40.966581,36.501671,3.246673,5.928874,2.964171,3.683213,3.028867,2.077241,2.187819,2.849712,38.482794,2.934626,0.000000,14.055752,40.562825,1.739107,2.685853,5.259983,14.824959,4.141994,3.491575,2.768506,2.222729,8.379787,3.816369,24.134439,12.230513,1.680672,2.000000,42.109708,2.433846,3.687398,41.333981,1.891011,2.034338,1.829260,2.371097,3.621329,3.806598,15.929508,13.499242,3.253695,2.755070,12.464773,25.843328,17.953733,2.650192,1.852931,2.148845,2.012644,13.804170,2.859530,8.659464,1.925163,14.377242,25.810554,21.833164,3.083886,2.853729,1.770994,1.676967,1.975028,2.935046,5.115263,1.926272,20.037321,1.932569,4.397285,35.906409,35.463806,42.682921,3.327266,5.164325,20.842500,1.111111,21.142050,4.819426,0.841352,35.418916,2.864849,21.655633,11.516249,2.994706,40.691074,4.253628,32.297505,8.049672,3.415080,2.372932,2.585354,3.111590,5.080198,56.686033,3.333333,1.923153,52.390459,1.587745,3.313588,46.755160,43.805796,6.578947,3.460141,7.273026,1.091106,4.421472,1.847408,1.612903,2.962436,61.425944,1.666403,1.980198,15.479348,53.808503,3.084164,0.000000,2.253792,14.976685,0.000000,3.279431,3.795954,2.774076,6.521739,6.157459,14.990656,14.864542,7.692308,0.0,63.268084,1.474495,3.862051,49.314430,2.581061,1.949350,1.717861,5.878349,8.601110,5.796408,13.159780,14.938852,6.997810,3.082014,13.039124,31.505628,19.648477,4.365111,2.973503,1.504897,2.030841,12.311308,2.362994,4.194400,2.233923,12.221948,26.375238,26.422625,3.284208,4.989526,1.123597,2.629795,1.375576,4.157848,12.976090,1.061170,23.951266,1.482072,9.235695,47.572349,44.338771,55.514705,6.324391,4.250715,21.929746,3.692879,22.325484,1.851852,1.613054,46.286349,2.590793,19.931185,8.197909,3.991336,57.102407,9.110004,48.343958,10.509792,8.434981,5.057151,37.838568,49.514537,1.770994,1.123597,15.929508,13.159780,3.605339,8.761053,3.411481,2.384415,5.186680,3.892620,1.937895,2.425938,3.297082,3.145867,19.192840,20.360721
1,Outside Wasatch Front,4.197337,6.172826,5.842146,2.175307,49.883204,3.817713,5.633925,5.279542,4.259444,49.602294,4.822798,4.762961,3.921137,2.164501,50.065419,4.392930,55.800025,3.897207,4.563516,5.840600,6.920399,3.600616,60.269706,5.063191,6.649459,3.784359,10.073957,10.836969,34.066025,7.362402,60.058096,60.057154,2.759889,5.497333,53.630835,4.944724,5.460379,6.494492,55.461444,57.999783,4.034234,60.691079,3.512186,4.154680,4.524643,4.229080,4.630501,3.168641,4.962952,3.690165,32.288584,5.939859,3.593356,49.554930,7.452619,3.809404,6.496130,61.306630,3.891913,7.078696,3.829821,6.787732,66.675161,42.916760,4.683312,8.460045,5.262068,5.766706,3.119005,3.765

In [147]:
# bu map unit
# block group
df_CountySum_wOutWF_Percent_MU = df_CountySum_wOutWF.copy()

for x in dColCodesWCounty_x:   
    #display(x)
    df_CountySum_wOutWF_Percent_MU[x] = df_CountySum_wOutWF_Percent_MU[x] / df_CountySum_wOutWF_Percent_MU['TTL_h'] * 100

for y in dColCodesWCounty_y:   
    #display(y)
    df_CountySum_wOutWF_Percent_MU[y] = df_CountySum_wOutWF_Percent_MU[y] / df_CountySum_wOutWF_Percent_MU['TTL_w'] * 100

df_CountySum_wOutWF_Percent_MU

,WFCounty,AFK_h,ALA_h,ALP_h,BDL_h,BGM_h,BNT_h,BRT_h,CDF_h,CEN_h,CHA_h,CHL_h,CLF_h,CLI_h,CMT_h,COA_h,CWH_h,DAN_h,DRA_h,EAG_h,ELK_h,EMT_h,FAR_h,FCS_h,FFD_h,FRR_h,FTH_h,GLA_h,GOS_h,GRL_h,HAR_h,HDT_h,HEB_h,HER_h,HGH_h,HNF_h,HOL_h,HOO_h,HVL_h,IND_h,INT_h,KAY_h,KMS_h,KMT_h,LAY_h,LEH_h,LIN_h,MAP_h,MID_h,MLC_h,MMT_h,MRG_h,MSL_h,MUR_h,MWY_h,NOG_h,NSL_h,OGD_h,OKL_h,ORM_h,PAY_h,PGR_h,PLN_h,PRK_h,PRY_h,PVO_h,PVW_h,ROY_h,RVD_h,RVT_h,SAN_h,SAQ_h,SAR_h,SFK_h,SJC_h,SLC_h,SLM_h,SOG_h,SPV_h,SSL_h,SUN_h,SWE_h,SYR_h,TAY_h,TOO_h,UIN_h,VIN_h,WAT_h,WBG_h,WDL_h,WEB_h,WHT_h,WHV_h,WIL_h,WJC_h,WPT_h,WVC_h,WXC_h,TTL_h,AFK_w,ALA_w,ALP_w,BDL_w,BGM_w,BNT_w,BRT_w,CDF_w,CEN_w,CHA_w,CHL_w,CLF_w,CLI_w,CMT_w,COA_w,CWH_w,DAN_w,DRA_w,EAG_w,ELK_w,EMT_w,FAR_w,FCS_w,FFD_w,FRR_w,FTH_w,GLA_w,GOS_w,GRL_w,HAR_w,HDT_w,HEB_w,HER_w,HGH_w,HNF_w,HOL_w,HOO_w,HVL_w,IND_w,INT_w,KAY_w,KMS_w,KMT_w,LAY_w,LEH_w,LIN_w,MAP_w,MID_w,MLC_w,MMT_w,MRG_w,MSL_w,MUR_w,MWY_w,NOG_w,NSL_w,OGD_w,OKL_w,ORM_w,PAY_w,PGR_w,PLN_w,PRK_w,PRY_w,PVO_w,PVW_w,ROY_w,RVD_w,RVT_w,SAN_w,SAQ_w,SAR_w,SFK_w,SJC_w,SLC_w,SLM_w,SOG_w,SPV_w,SSL_w,SUN_w,SWE_w,SYR_w,TAY_w,TOO_w,UIN_w,VIN_w,WAT_w,WBG_w,WDL_w,WEB_w,WHT_w,WHV_w,WIL_w,WJC_w,WPT_w,WVC_w,WXC_w,TTL_w,z03_h,z03_w,z11_h,z11_w,z23_h,z23_w,z29_h,z29_w,z35_h,z35_w,z43_h,z43_w,z45_h,z45_w,z49_h,z49_w,z51_h,z51_w,z57_h,z57_w
0,Davis County,0.164637,0.001790,0.059055,0.102003,0.635284,5.539499,0.000000,0.001790,2.269128,0.000000,0.059949,4.734210,3.158526,0.008948,0.021474,0.371328,0.007158,0.529702,0.219218,0.019685,0.010737,3.068154,0.003579,0.000000,0.329274,0.710445,0.008053,0.005369,0.202217,0.344485,0.006263,0.202217,0.357907,0.109162,0.015211,0.425014,0.714024,0.016106,0.000000,0.000000,4.673365,0.009842,0.550281,11.597069,0.420540,0.065318,0.053686,0.298852,0.891187,0.437541,0.208481,0.078739,0.655864,0.032212,0.926083,1.819955,6.249944,0.008053,0.668391,0.144057,0.241587,0.316747,0.064423,0.161953,0.613810,0.462594,4.208982,0.746235,0.579809,1.127406,0.069792,0.165532,0.294378,0.765920,3.955763,0.054581,1.321570,0.233534,0.395487,0.763236,0.897451,4.523939,0.832133,0.706866,0.083213,0.000895,0.780236,0.002684,0.001790,0.918925,0.051896,0.948453,0.074266,1.467417,1.615948,2.306708,1.380625,100.0,0.437338,0.009449,0.025646,0.050618,0.241616,4.289696,0.000000,0.000675,1.907282,0.001350,0.006074,5.146151,0.670181,0.002025,0.009449,0.912472,0.000000,1.038004,0.010798,0.002025,0.000000,3.705228,0.000000,0.001350,0.258489,0.107310,0.000675,0.000000,0.011473,0.107985,0.000000,0.076939,0.072215,0.028346,0.000675,0.346226,0.037795,0.001350,0.000000,0.0,3.340780,0.002700,0.070190,8.884450,0.494030,0.107310,0.010124,0.639810,2.732015,0.076264,0.072890,0.137006,2.009867,0.007424,0.188298,3.059344,7.618328,0.004724,0.867253,0.036445,0.093812,0.044544,0.122158,0.012148,0.793013,0.116084,1.268821,1.302567,0.160627,1.525960,0.003375,0.026321,0.072215,0.754544,20.781000,0.006749,1.013032,0.069515,2.034839,0.315181,0.139030,1.328213,0.801112,0.156578,0.019572,0.011473,0.431264,0.000000,0.000000,0.602690,0.004724,0.388745,0.004049,0.877377,0.340152,4.120295,1.465894,100.0,0.937715,0.275361,48.935675,36.372656,0.069792,0.003375,0.208481,0.072890,17.312837,40.297903,0.161953,0.155903,0.943084,0.180875,4.228667,3.739649,0.288115,0.096511,18.408926,13.673575
1,Outside Wasatch Front,0.161615,0.002229,0.076163,0.040868,1.793370,0.246695,0.001115,0.004830,0.096226,0.037896,0.061302,0.205084,0.122605,0.002229,0.129292,0.242237,0.099941,0.284220,0.211772,0.023778,0.014490,0.105143,0.086938,0.001115,0.060931,0.021920,0.021920,0.015976,0.624540,0.070591,0.062417,1.752131,0.144896,0.130035,0.060931,0.232577,0.062045,0.004458,0.017833,0.006316,0.160872,0.197282,0.210285,0.451036,0.438404,0.064646,0.060931,0.170903,0.507509,0.165330,0.208056,0.015233,0.302053,0.396421,0.229976,0.099198,0.913590,0.117403,0.595189,0.223289,0.192452,0.064275,0.928822,0.405338,0.645717,0.111459,0.332147,0.076535,0.240008,0.617853,0.242237,0.177962,0.341435,0.309855,1.346793,0.076163,0.127063,0.293508,0.174990,0.0

In [148]:
df_CountySum_wOutWF           .to_json(os.path.join(dirResults, "countydata_number.json"    ),orient='records')
df_CountySum_wOutWF_Percent_SA.to_json(os.path.join(dirResults, "countydata_percent_sa.json"),orient='records')
df_CountySum_wOutWF_Percent_MU.to_json(os.path.join(dirResults, "countydata_percent_mu.json"),orient='records')